In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import json
import os
import zipfile
import numpy as np
from collections import defaultdict, Counter
from scipy.ndimage import label, binary_dilation, binary_erosion, convolve
from itertools import product, permutations, combinations
import math

# ==================== SECTION 1: 1D PATTERNS AND TRANSFORMATIONS ====================

def handle_1d_raster_transform(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    h, w = inp.shape
    
    for order in ['row_major', 'col_major', 'zigzag', 'spiral']:
        arr_in = to_1d_raster(inp.tolist(), order)
        arr_out = to_1d_raster(out.tolist(), order)
        
        if arr_out == arr_in[::-1]:
            return f"""def p(g):
 h,w=len(g),len(g[0])
 arr={''.join([
  f"[g[i][j] for i in range(h) for j in range(w)]" if order=='row_major' else
  f"[g[i][j] for j in range(w) for i in range(h)]" if order=='col_major' else
  f"[g[i][j] for i in range(h-1,-1,-1) for j in range(w-1,-1,-1)]" if order=='row_major_rev' else
  "[]"
 ])}
 arr=arr[::-1]
 res=[[0]*w for _ in range(h)]
 for idx,val in enumerate(arr):
  {"i,j=idx//w,idx%w;res[i][j]=val" if order=='row_major' else
   "j,i=idx//h,idx%h;res[i][j]=val" if order=='col_major' else
   ""}
 return res
"""
        
        if sorted(arr_out) == sorted(arr_in) and arr_out == sorted(arr_in):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 arr=sorted([g[i][j] for i in range(h) for j in range(w)])
 res=[[0]*w for _ in range(h)]
 for idx,val in enumerate(arr):
  i,j=idx//w,idx%w
  res[i][j]=val
 return res
"""
    return None

def handle_1d_pattern_repeat(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    arr_in = to_1d_raster(inp.tolist(), 'row_major')
    arr_out = to_1d_raster(out.tolist(), 'row_major')
    
    for repeat in [2, 3, 4]:
        if arr_out == arr_in * repeat:
            h_out, w_out = out.shape
            return f"""def p(g):
 arr=[g[i][j] for i in range(len(g)) for j in range(len(g[0]))]
 arr=arr*{repeat}
 h,w={h_out},{w_out}
 res=[[0]*w for _ in range(h)]
 for idx,val in enumerate(arr[:h*w]):
  res[idx//w][idx%w]=val
 return res
"""
    return None

def handle_1d_sort_colors(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    arr_in = to_1d_raster(inp.tolist(), 'row_major')
    arr_out = to_1d_raster(out.tolist(), 'row_major')
    
    color_groups = defaultdict(list)
    for i, c in enumerate(arr_in):
        color_groups[c].append(i)
    
    sorted_arr = []
    for color in sorted(color_groups.keys()):
        sorted_arr.extend([color] * len(color_groups[color]))
    
    if arr_out == sorted_arr:
        return """def p(g):
 h,w=len(g),len(g[0])
 arr=[g[i][j] for i in range(h) for j in range(w)]
 counts={}
 for c in arr:
  counts[c]=counts.get(c,0)+1
 res=[]
 for c in sorted(counts.keys()):
  res.extend([c]*counts[c])
 out=[[0]*w for _ in range(h)]
 for i,v in enumerate(res):
  out[i//w][i%w]=v
 return out
"""
    return None

def handle_1d_run_length_decode(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    arr_in = to_1d_raster(inp.tolist(), 'row_major')
    
    decoded = []
    i = 0
    while i < len(arr_in) - 1:
        value = arr_in[i]
        count = arr_in[i+1]
        if count > 0 and count < 10:
            decoded.extend([value] * count)
        i += 2
    
    if len(decoded) > 0:
        oh, ow = out.shape
        if len(decoded) == oh * ow:
            test_grid = from_1d_raster(decoded, oh, ow, 'row_major')
            if test_grid and np.array_equal(np.array(test_grid), out):
                return f"""def p(g):
 arr=[g[i][j]for i in range(len(g))for j in range(len(g[0]))]
 d=[]
 i=0
 while i<len(arr)-1:
  v,c=arr[i],arr[i+1]
  if c>0 and c<10:
   d.extend([v]*c)
  i+=2
 h,w={oh},{ow}
 res=[[0]*w for _ in range(h)]
 for idx,val in enumerate(d[:h*w]):
  res[idx//w][idx%w]=val
 return res
"""
    return None

def handle_1d_reverse_segments(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    arr_in = to_1d_raster(inp.tolist(), 'row_major')
    arr_out = to_1d_raster(out.tolist(), 'row_major')
    
    for seg_size in range(2, min(10, len(arr_in)//2)):
        test = []
        for i in range(0, len(arr_in), seg_size):
            segment = arr_in[i:i+seg_size]
            test.extend(segment[::-1])
        
        if test[:len(arr_out)] == arr_out:
            h, w = out.shape
            return f"""def p(g):
 arr=[g[i][j] for i in range(len(g)) for j in range(len(g[0]))]
 res=[]
 for i in range(0,len(arr),{seg_size}):
  res.extend(arr[i:i+{seg_size}][::-1])
 h,w={h},{w}
 out=[[0]*w for _ in range(h)]
 for i,v in enumerate(res[:h*w]):
  out[i//w][i%w]=v
 return out
"""
    return None

def handle_1d_shuffle_pattern(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    arr_in = to_1d_raster(inp.tolist(), 'row_major')
    arr_out = to_1d_raster(out.tolist(), 'row_major')
    
    odd_even = []
    for i in range(1, len(arr_in), 2):
        odd_even.append(arr_in[i])
    for i in range(0, len(arr_in), 2):
        odd_even.append(arr_in[i])
    
    if odd_even == arr_out:
        h, w = out.shape
        return f"""def p(g):
 arr=[g[i][j] for i in range(len(g)) for j in range(len(g[0]))]
 res=[]
 for i in range(1,len(arr),2):
  res.append(arr[i])
 for i in range(0,len(arr),2):
  res.append(arr[i])
 h,w={h},{w}
 out=[[0]*w for _ in range(h)]
 for i,v in enumerate(res[:h*w]):
  out[i//w][i%w]=v
 return out
"""
    return None

def handle_1d_wave_transform(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    arr_in = to_1d_raster(inp.tolist(), 'row_major')
    arr_out = to_1d_raster(out.tolist(), 'row_major')
    
    n = len(arr_in)
    indices = list(range(n))
    wave_indices = sorted(indices, key=lambda i: math.sin(2 * math.pi * i / n))
    
    test = [arr_in[i] for i in wave_indices]
    if test == arr_out:
        h, w = out.shape
        return f"""def p(g):
 import math
 arr=[g[i][j] for i in range(len(g)) for j in range(len(g[0]))]
 n=len(arr)
 idx=sorted(range(n),key=lambda i:math.sin(2*math.pi*i/n))
 res=[arr[i] for i in idx]
 h,w={h},{w}
 out=[[0]*w for _ in range(h)]
 for i,v in enumerate(res[:h*w]):
  out[i//w][i%w]=v
 return out
"""
    return None

def handle_1d_compression(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    arr_in = to_1d_raster(inp.tolist(), 'row_major')
    
    compressed = []
    if arr_in:
        compressed.append(arr_in[0])
        for i in range(1, len(arr_in)):
            if arr_in[i] != arr_in[i-1]:
                compressed.append(arr_in[i])
    
    if len(compressed) <= out.shape[0] * out.shape[1]:
        h, w = out.shape
        test_grid = from_1d_raster(compressed + [0]*(h*w-len(compressed)), h, w, 'row_major')
        if test_grid and np.array_equal(np.array(test_grid), out):
            return f"""def p(g):
 arr=[g[i][j] for i in range(len(g)) for j in range(len(g[0]))]
 c=[]
 if arr:
  c.append(arr[0])
  for i in range(1,len(arr)):
   if arr[i]!=arr[i-1]:
    c.append(arr[i])
 h,w={h},{w}
 c.extend([0]*(h*w-len(c)))
 out=[[0]*w for _ in range(h)]
 for i,v in enumerate(c[:h*w]):
  out[i//w][i%w]=v
 return out
"""
    return None

def handle_1d_periodic_transform(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    arr_in = to_1d_raster(inp.tolist(), 'row_major')
    arr_out = to_1d_raster(out.tolist(), 'row_major')
    
    for period in range(2, min(10, len(arr_in)//2)):
        test = []
        for i in range(len(arr_in)):
            if i % period == 0:
                test.append(arr_in[i])
            elif i % period == 1 and i < len(arr_in):
                test.append((arr_in[i] + 1) % 10)
            else:
                test.append(arr_in[i])
        
        if test == arr_out:
            h, w = out.shape
            return f"""def p(g):
 arr=[g[i][j] for i in range(len(g)) for j in range(len(g[0]))]
 res=[]
 for i in range(len(arr)):
  if i%{period}==0:
   res.append(arr[i])
  elif i%{period}==1:
   res.append((arr[i]+1)%10)
  else:
   res.append(arr[i])
 h,w={h},{w}
 out=[[0]*w for _ in range(h)]
 for i,v in enumerate(res[:h*w]):
  out[i//w][i%w]=v
 return out
"""
    return None

def handle_1d_fibonacci_transform(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    arr_in = to_1d_raster(inp.tolist(), 'row_major')
    arr_out = to_1d_raster(out.tolist(), 'row_major')
    
    fibs = [0, 1]
    while fibs[-1] < len(arr_in):
        fibs.append(fibs[-1] + fibs[-2])
    
    test = [0] * len(arr_in)
    for f in fibs:
        if f < len(arr_in):
            test[f] = arr_in[f]
    
    if test == arr_out:
        h, w = out.shape
        return f"""def p(g):
 arr=[g[i][j] for i in range(len(g)) for j in range(len(g[0]))]
 f=[0,1]
 while f[-1]<len(arr):
  f.append(f[-1]+f[-2])
 res=[0]*len(arr)
 for i in f:
  if i<len(arr):
   res[i]=arr[i]
 h,w={h},{w}
 out=[[0]*w for _ in range(h)]
 for i,v in enumerate(res[:h*w]):
  out[i//w][i%w]=v
 return out
"""
    return None

def handle_1d_prime_positions(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    def is_prime(n):
        if n < 2: return False
        for i in range(2, int(n**0.5) + 1):
            if n % i == 0: return False
        return True
    
    arr_in = to_1d_raster(inp.tolist(), 'row_major')
    arr_out = to_1d_raster(out.tolist(), 'row_major')
    
    test = []
    for i in range(len(arr_in)):
        if is_prime(i):
            test.append(arr_in[i])
        else:
            test.append(0)
    
    if test == arr_out:
        h, w = out.shape
        return f"""def p(g):
 def ip(n):
  if n<2:return False
  for i in range(2,int(n**0.5)+1):
   if n%i==0:return False
  return True
 arr=[g[i][j] for i in range(len(g)) for j in range(len(g[0]))]
 res=[]
 for i in range(len(arr)):
  res.append(arr[i] if ip(i) else 0)
 h,w={h},{w}
 out=[[0]*w for _ in range(h)]
 for i,v in enumerate(res[:h*w]):
  out[i//w][i%w]=v
 return out
"""
    return None

def handle_1d_alternating_ops(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    arr_in = to_1d_raster(inp.tolist(), 'row_major')
    arr_out = to_1d_raster(out.tolist(), 'row_major')
    
    test = []
    for i, val in enumerate(arr_in):
        if i % 2 == 0:
            test.append((val + 1) % 10)
        else:
            test.append((val - 1) % 10)
    
    if test == arr_out:
        h, w = out.shape
        return f"""def p(g):
 arr=[g[i][j] for i in range(len(g)) for j in range(len(g[0]))]
 res=[]
 for i,v in enumerate(arr):
  if i%2==0:
   res.append((v+1)%10)
  else:
   res.append((v-1)%10)
 h,w={h},{w}
 out=[[0]*w for _ in range(h)]
 for i,v in enumerate(res[:h*w]):
  out[i//w][i%w]=v
 return out
"""
    return None

# ==================== SECTION 2: 2D LINEARIZATION PATTERNS ====================

def handle_diagonal_linearization(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    h, w = inp.shape
    
    diagonals = []
    for d in range(w):
        diag = []
        i, j = 0, d
        while i < h and j < w:
            diag.append(inp[i, j])
            i += 1
            j += 1
        diagonals.extend(diag)
    
    for d in range(1, h):
        diag = []
        i, j = d, 0
        while i < h and j < w:
            diag.append(inp[i, j])
            i += 1
            j += 1
        diagonals.extend(diag)
    
    oh, ow = out.shape
    if len(diagonals) == oh * ow:
        test_grid = from_1d_raster(diagonals, oh, ow, 'row_major')
        if test_grid and np.array_equal(np.array(test_grid), out):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 d=[]
 for s in range(w):
  i,j=0,s
  while i<h and j<w:
   d.append(g[i][j])
   i+=1;j+=1
 for s in range(1,h):
  i,j=s,0
  while i<h and j<w:
   d.append(g[i][j])
   i+=1;j+=1
 oh,ow={oh},{ow}
 res=[[0]*ow for _ in range(oh)]
 for idx,v in enumerate(d[:oh*ow]):
  res[idx//ow][idx%ow]=v
 return res
"""
    return None

def handle_hilbert_curve(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape == (2, 2) and out.shape == (1, 4):
        hilbert_order = [inp[1,0], inp[0,0], inp[0,1], inp[1,1]]
        if hilbert_order == out[0].tolist():
            return """def p(g):
 if len(g)==2 and len(g[0])==2:
  return[[g[1][0],g[0][0],g[0][1],g[1][1]]]
 return g
"""
    return None

def handle_morton_order(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    h, w = inp.shape
    if h == w and h & (h-1) == 0:
        morton = []
        
        def interleave_bits(x, y):
            result = 0
            for i in range(16):
                result |= ((x & (1 << i)) << i) | ((y & (1 << i)) << (i + 1))
            return result
        
        coords = []
        for i in range(h):
            for j in range(w):
                coords.append((interleave_bits(i, j), i, j))
        
        coords.sort()
        morton = [inp[i, j] for _, i, j in coords]
        
        oh, ow = out.shape
        if len(morton) == oh * ow:
            test_grid = from_1d_raster(morton, oh, ow, 'row_major')
            if test_grid and np.array_equal(np.array(test_grid), out):
                return """def p(g):
 h,w=len(g),len(g[0])
 def ib(x,y):
  r=0
  for i in range(16):
   r|=((x&(1<<i))<<i)|((y&(1<<i))<<(i+1))
  return r
 c=[]
 for i in range(h):
  for j in range(w):
   c.append((ib(i,j),i,j))
 c.sort()
 m=[g[i][j]for _,i,j in c]
 return[m]
"""
    return None

def handle_block_linearization(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    h, w = inp.shape
    for bh in range(1, min(h+1, 4)):
        for bw in range(1, min(w+1, 4)):
            if h % bh == 0 and w % bw == 0:
                blocks = []
                for i in range(0, h, bh):
                    for j in range(0, w, bw):
                        for bi in range(bh):
                            for bj in range(bw):
                                blocks.append(inp[i+bi, j+bj])
                
                oh, ow = out.shape
                if len(blocks) == oh * ow:
                    test_grid = from_1d_raster(blocks, oh, ow, 'row_major')
                    if test_grid and np.array_equal(np.array(test_grid), out):
                        return f"""def p(g):
 h,w=len(g),len(g[0])
 b=[]
 for i in range(0,h,{bh}):
  for j in range(0,w,{bw}):
   for bi in range({bh}):
    for bj in range({bw}):
     b.append(g[i+bi][j+bj])
 oh,ow={oh},{ow}
 res=[[0]*ow for _ in range(oh)]
 for idx,v in enumerate(b[:oh*ow]):
  res[idx//ow][idx%ow]=v
 return res
"""
    return None

def handle_snake_pattern(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    arr_out = to_1d_raster(out.tolist(), 'row_major')
    arr_snake = to_1d_raster(inp.tolist(), 'zigzag')
    
    if arr_snake == arr_out:
        h, w = out.shape
        return f"""def p(g):
 res=[]
 for i in range(len(g)):
  if i%2==0:
   res.extend(g[i])
  else:
   res.extend(g[i][::-1])
 h,w={h},{w}
 out=[[0]*w for _ in range(h)]
 for i,v in enumerate(res[:h*w]):
  out[i//w][i%w]=v
 return out
"""
    return None

def handle_radial_linearization(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    h, w = inp.shape
    cx, cy = w // 2, h // 2
    
    pixels = []
    for i in range(h):
        for j in range(w):
            dist = ((i - cy)**2 + (j - cx)**2)**0.5
            angle = math.atan2(i - cy, j - cx)
            pixels.append((dist, angle, inp[i, j]))
    
    pixels.sort()
    radial = [p[2] for p in pixels]
    
    oh, ow = out.shape
    if len(radial) == oh * ow:
        test_grid = from_1d_raster(radial, oh, ow, 'row_major')
        if test_grid and np.array_equal(np.array(test_grid), out):
            return f"""def p(g):
 import math
 h,w=len(g),len(g[0])
 cx,cy=w//2,h//2
 px=[]
 for i in range(h):
  for j in range(w):
   d=((i-cy)**2+(j-cx)**2)**0.5
   a=math.atan2(i-cy,j-cx)
   px.append((d,a,g[i][j]))
 px.sort()
 r=[p[2]for p in px]
 oh,ow={oh},{ow}
 res=[[0]*ow for _ in range(oh)]
 for idx,v in enumerate(r[:oh*ow]):
  res[idx//ow][idx%ow]=v
 return res
"""
    return None

# ==================== SECTION 3: PARITY AND MODULAR PATTERNS ====================

def handle_odd_even_rows(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    test = np.copy(inp)
    
    for i in range(0, len(inp), 2):
        test[i] = (test[i] + 1) % 10
    
    if np.array_equal(test, out):
        return """def p(g):
 res=[r[:]for r in g]
 for i in range(0,len(g),2):
  res[i]=[(x+1)%10 for x in g[i]]
 return res
"""
    
    test = np.copy(inp)
    for i in range(1, len(inp), 2):
        test[i] = 0
    
    if np.array_equal(test, out):
        return """def p(g):
 res=[r[:]for r in g]
 for i in range(1,len(g),2):
  res[i]=[0]*len(g[0])
 return res
"""
    return None

def handle_odd_even_cols(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    test = np.copy(inp)
    w = inp.shape[1]
    for j in range(0, w-1, 2):
        test[:, j], test[:, j+1] = inp[:, j+1].copy(), inp[:, j].copy()
    
    if np.array_equal(test, out):
        return """def p(g):
 res=[r[:]for r in g]
 w=len(g[0])
 for i in range(len(g)):
  for j in range(0,w-1,2):
   res[i][j],res[i][j+1]=g[i][j+1],g[i][j]
 return res
"""
    return None

def handle_checkerboard_transform(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    test = np.copy(inp)
    h, w = inp.shape
    
    for i in range(h):
        for j in range(w):
            if (i + j) % 2 == 0:
                test[i, j] = (inp[i, j] * 2) % 10
            else:
                test[i, j] = (inp[i, j] + 5) % 10
    
    if np.array_equal(test, out):
        return """def p(g):
 res=[r[:]for r in g]
 for i in range(len(g)):
  for j in range(len(g[0])):
   if(i+j)%2==0:
    res[i][j]=(g[i][j]*2)%10
   else:
    res[i][j]=(g[i][j]+5)%10
 return res
"""
    return None

def handle_parity_coloring(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    test = np.zeros_like(inp)
    h, w = inp.shape
    
    for i in range(h):
        for j in range(w):
            if inp[i, j] != 0:
                test[i, j] = 1 if (i + j) % 2 == 0 else 2
    
    if np.array_equal(test, out):
        return """def p(g):
 res=[[0]*len(g[0])for _ in g]
 for i in range(len(g)):
  for j in range(len(g[0])):
   if g[i][j]!=0:
    res[i][j]=1 if(i+j)%2==0 else 2
 return res
"""
    return None

def handle_alternating_blocks(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    if h % 2 == 0 and w % 2 == 0:
        test = np.copy(inp)
        
        for i in range(0, h, 2):
            for j in range(0, w, 2):
                block_idx = (i//2 + j//2) % 2
                if block_idx == 0:
                    block = inp[i:i+2, j:j+2]
                    test[i:i+2, j:j+2] = np.rot90(block, 1)
                else:
                    block = inp[i:i+2, j:j+2]
                    test[i:i+2, j:j+2] = np.fliplr(block)
        
        if np.array_equal(test, out):
            return """def p(g):
 import numpy as np
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(0,h,2):
  for j in range(0,w,2):
   idx=(i//2+j//2)%2
   b=[g[i+x][j:j+2]for x in range(2)]
   if idx==0:
    r=np.rot90(b,1).tolist()
   else:
    r=np.fliplr(b).tolist()
   for x in range(2):
    for y in range(2):
     res[i+x][j+y]=r[x][y]
 return res
"""
    return None

def handle_modular_arithmetic(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    for mod in range(2, 10):
        for add in range(0, 10):
            test = (inp + add) % mod
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[(x+{add})%{mod} for x in r]for r in g]
"""
    
    for mod in range(2, 10):
        for mult in range(1, 10):
            test = (inp * mult) % mod
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[(x*{mult})%{mod} for x in r]for r in g]
"""
    return None

# ==================== SECTION 4: BINARY AND ARITHMETIC PATTERNS ====================

def handle_xor_patterns(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    for const in range(1, 16):
        test = inp ^ const
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[x^{const} for x in r]for r in g]
"""
    
    test = np.zeros_like(inp)
    h, w = inp.shape
    for i in range(h):
        for j in range(w):
            test[i, j] = inp[i, j] ^ ((i + j) % 10)
    
    if np.array_equal(test, out):
        return """def p(g):
 return[[g[i][j]^((i+j)%10)for j in range(len(g[0]))]for i in range(len(g))]
"""
    return None

def handle_binary_operations(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    test = np.zeros_like(inp)
    for i in range(inp.shape[0]):
        for j in range(inp.shape[1]):
            test[i, j] = bin(inp[i, j]).count('1')
    
    if np.array_equal(test, out):
        return """def p(g):
 return[[bin(x).count('1')for x in r]for r in g]
"""
    
    test = np.zeros_like(inp)
    for i in range(inp.shape[0]):
        for j in range(inp.shape[1]):
            val = inp[i, j]
            if val < 8:
                rev = int(bin(val)[2:].zfill(3)[::-1], 2)
                test[i, j] = rev
    
    if np.array_equal(test, out):
        return """def p(g):
 res=[]
 for r in g:
  row=[]
  for x in r:
   if x<8:
    row.append(int(bin(x)[2:].zfill(3)[::-1],2))
   else:
    row.append(x)
  res.append(row)
 return res
"""
    return None

def handle_bit_manipulation(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    for shift in range(1, 4):
        test = (inp << shift) % 10
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[(x<<{shift})%10 for x in r]for r in g]
"""
    
    for shift in range(1, 4):
        test = inp >> shift
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[x>>{shift} for x in r]for r in g]
"""
    
    for bit in range(3):
        test = inp ^ (1 << bit)
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[x^(1<<{bit})for x in r]for r in g]
"""
    return None

def handle_arithmetic_sequences(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    arr_out = out.flatten()
    if len(arr_out) > 2:
        diff = arr_out[1] - arr_out[0]
        is_arithmetic = all(arr_out[i+1] - arr_out[i] == diff for i in range(len(arr_out)-1))
        
        if is_arithmetic:
            start = arr_out[0]
            h, w = out.shape
            return f"""def p(g):
 s,d={start},{diff}
 h,w={h},{w}
 res=[]
 for i in range(h):
  row=[]
  for j in range(w):
   row.append(s+(i*w+j)*d)
  res.append(row)
 return res
"""
    return None

def handle_geometric_sequences(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    arr_out = out.flatten()
    if len(arr_out) > 2 and arr_out[0] != 0:
        ratio = arr_out[1] / arr_out[0] if arr_out[0] != 0 else 0
        is_geometric = all(
            abs(arr_out[i+1] - arr_out[i] * ratio) < 0.001 
            for i in range(len(arr_out)-1) 
            if arr_out[i] != 0
        )
        
        if is_geometric:
            start = arr_out[0]
            h, w = out.shape
            return f"""def p(g):
 s,r={start},{ratio}
 h,w={h},{w}
 res=[]
 for i in range(h):
  row=[]
  for j in range(w):
   row.append(int(s*(r**(i*w+j))))
  res.append(row)
 return res
"""
    return None

# ==================== SECTION 5: TEMPLATE AND CONVOLUTION PATTERNS ====================

def handle_template_matching(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    for th in range(2, 4):
        for tw in range(2, 4):
            if th > inp.shape[0] or tw > inp.shape[1]:
                continue
            
            templates = {}
            for i in range(inp.shape[0] - th + 1):
                for j in range(inp.shape[1] - tw + 1):
                    template = tuple(inp[i:i+th, j:j+tw].flatten())
                    if template not in templates:
                        templates[template] = len(templates) + 1
            
            test = np.copy(inp)
            for i in range(inp.shape[0] - th + 1):
                for j in range(inp.shape[1] - tw + 1):
                    template = tuple(inp[i:i+th, j:j+tw].flatten())
                    tid = templates[template]
                    test[i + th//2, j + tw//2] = tid
            
            if np.array_equal(test, out):
                return f"""def p(g):
 h,w=len(g),len(g[0])
 th,tw={th},{tw}
 t={{}}
 for i in range(h-th+1):
  for j in range(w-tw+1):
   tmpl=tuple(g[i+x][j+y]for x in range(th)for y in range(tw))
   if tmpl not in t:
    t[tmpl]=len(t)+1
 res=[r[:]for r in g]
 for i in range(h-th+1):
  for j in range(w-tw+1):
   tmpl=tuple(g[i+x][j+y]for x in range(th)for y in range(tw))
   res[i+th//2][j+tw//2]=t[tmpl]
 return res
"""
    return None

def handle_convolution_patterns(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    kernels = [
        np.array([[1, 1], [1, 1]]),
        np.array([[1, 0], [0, 1]]),
        np.array([[0, 1], [1, 0]]),
        np.array([[1, -1], [-1, 1]]),
    ]
    
    for kernel in kernels:
        from scipy.ndimage import convolve
        test = convolve(inp.astype(float), kernel, mode='constant')
        test = np.abs(test).astype(int) % 10
        
        if np.array_equal(test, out):
            kernel_str = str(kernel.tolist()).replace(' ', '')
            return f"""def p(g):
 from scipy.ndimage import convolve
 import numpy as np
 k=np.array({kernel_str})
 t=convolve(np.array(g,float),k,mode='constant')
 return(np.abs(t).astype(int)%10).tolist()
"""
    return None

def handle_morphological_ops(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    binary_inp = inp > 0
    
    from scipy.ndimage import binary_erosion
    test = binary_erosion(binary_inp).astype(int)
    if np.array_equal(test * inp.max(), out):
        return """def p(g):
 from scipy.ndimage import binary_erosion
 import numpy as np
 b=np.array(g)>0
 e=binary_erosion(b).astype(int)
 return(e*np.max(g)).tolist()
"""
    
    from scipy.ndimage import binary_dilation
    test = binary_dilation(binary_inp).astype(int)
    if np.array_equal(test * inp.max(), out):
        return """def p(g):
 from scipy.ndimage import binary_dilation
 import numpy as np
 b=np.array(g)>0
 d=binary_dilation(b).astype(int)
 return(d*np.max(g)).tolist()
"""
    return None

def handle_edge_detection_advanced(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    edges = np.zeros_like(inp)
    
    for i in range(1, h-1):
        for j in range(1, w-1):
            gx = abs(inp[i,j+1] - inp[i,j-1])
            gy = abs(inp[i+1,j] - inp[i-1,j])
            edges[i,j] = min(gx + gy, 9)
    
    if np.array_equal(edges, out):
        return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    edge=False
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if not(0<=ni<h and 0<=nj<w)or g[ni][nj]==0:
      edge=True
      break
    if edge:
     res[i][j]=g[i][j]
 return res
"""
    return None

def handle_corner_detection(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    corners = np.zeros_like(inp)
    
    for i in range(1, h-1):
        for j in range(1, w-1):
            if inp[i,j] != 0:
                neighbors = [
                    inp[i-1,j], inp[i+1,j],
                    inp[i,j-1], inp[i,j+1]
                ]
                if sum(n != 0 for n in neighbors) == 2:
                    if (neighbors[0] != 0 and neighbors[2] != 0) or \
                       (neighbors[0] != 0 and neighbors[3] != 0) or \
                       (neighbors[1] != 0 and neighbors[2] != 0) or \
                       (neighbors[1] != 0 and neighbors[3] != 0):
                        corners[i,j] = inp[i,j]
    
    if np.array_equal(corners, out):
        return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(1,h-1):
  for j in range(1,w-1):
   if g[i][j]!=0:
    n=[g[i-1][j],g[i+1][j],g[i][j-1],g[i][j+1]]
    c=sum(1 for x in n if x!=0)
    if c==2:
     if(n[0]!=0 and n[2]!=0)or(n[0]!=0 and n[3]!=0)or(n[1]!=0 and n[2]!=0)or(n[1]!=0 and n[3]!=0):
      res[i][j]=g[i][j]
 return res
"""
    return None

def handle_pattern_completion_advanced(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape == out.shape:
        h, w = inp.shape
        
        for period in range(1, w//2):
            test = np.copy(inp)
            for i in range(h):
                row = inp[i]
                for j in range(w):
                    if row[j] == 0 and j >= period:
                        test[i,j] = row[j % period]
            
            if np.array_equal(test, out):
                return f"""def p(g):
 res=[r[:]for r in g]
 h,w=len(g),len(g[0])
 for i in range(h):
  for j in range(w):
   if g[i][j]==0 and j>={period}:
    res[i][j]=g[i][j%{period}]
 return res
"""
    return None

# ==================== SECTION 6: GRAPH AND COMPONENT PATTERNS ====================

def handle_graph_coloring(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    colors = np.zeros_like(inp)
    color_id = 1
    
    visited = set()
    
    for i in range(h):
        for j in range(w):
            if inp[i,j] != 0 and (i,j) not in visited:
                queue = [(i,j)]
                component = []
                
                while queue:
                    y, x = queue.pop(0)
                    if (y,x) in visited:
                        continue
                    visited.add((y,x))
                    component.append((y,x))
                    
                    for dy, dx in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ny, nx = y + dy, x + dx
                        if (0 <= ny < h and 0 <= nx < w and 
                            inp[ny,nx] != 0 and (ny,nx) not in visited):
                            queue.append((ny,nx))
                
                for y, x in component:
                    colors[y,x] = color_id
                color_id = (color_id % 9) + 1
    
    if np.array_equal(colors, out):
        return """def p(g):
 h,w=len(g),len(g[0])
 c=[[0]*w for _ in range(h)]
 cid=1
 v=set()
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0 and(i,j)not in v:
    q=[(i,j)]
    comp=[]
    while q:
     y,x=q.pop(0)
     if(y,x)in v:continue
     v.add((y,x))
     comp.append((y,x))
     for dy,dx in[(0,1),(1,0),(0,-1),(-1,0)]:
      ny,nx=y+dy,x+dx
      if 0<=ny<h and 0<=nx<w and g[ny][nx]!=0 and(ny,nx)not in v:
       q.append((ny,nx))
    for y,x in comp:
     c[y][x]=cid
    cid=(cid%9)+1
 return c
"""
    return None

def handle_tree_traversal(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape == out.shape:
        h, w = inp.shape
        
        root = None
        for i in range(h):
            for j in range(w):
                if inp[i,j] != 0:
                    root = (i,j)
                    break
            if root:
                break
        
        if root:
            levels = np.zeros_like(inp)
            queue = [(root[0], root[1], 1)]
            visited = set()
            
            while queue:
                y, x, level = queue.pop(0)
                if (y,x) in visited:
                    continue
                visited.add((y,x))
                levels[y,x] = level
                
                for dy, dx in [(1,0), (0,1), (0,-1)]:
                    ny, nx = y + dy, x + dx
                    if (0 <= ny < h and 0 <= nx < w and 
                        inp[ny,nx] != 0 and (ny,nx) not in visited):
                        queue.append((ny,nx, level + 1))
            
            if np.array_equal(levels, out):
                return """def p(g):
 h,w=len(g),len(g[0])
 r=None
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    r=(i,j)
    break
  if r:break
 if r:
  l=[[0]*w for _ in range(h)]
  q=[(r[0],r[1],1)]
  v=set()
  while q:
   y,x,lv=q.pop(0)
   if(y,x)in v:continue
   v.add((y,x))
   l[y][x]=lv
   for dy,dx in[(1,0),(0,1),(0,-1)]:
    ny,nx=y+dy,x+dx
    if 0<=ny<h and 0<=nx<w and g[ny][nx]!=0 and(ny,nx)not in v:
     q.append((ny,nx,lv+1))
  return l
 return g
"""
    return None

def handle_component_labeling(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    from scipy.ndimage import label
    labeled, n = label(inp > 0)
    
    if np.array_equal(labeled, out):
        return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 l,_=label(np.array(g)>0)
 return l.tolist()
"""
    return None

def handle_minimum_spanning_tree(task_data, analysis):
    return None

def handle_graph_isomorphism(task_data, analysis):
    return None

# ==================== SECTION 7: COLOR MANIPULATION PATTERNS ====================

def handle_color_gradients(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    h, w = out.shape
    
    test = np.zeros((h, w), dtype=int)
    for j in range(w):
        test[:, j] = (j * 9 // (w-1)) if w > 1 else 0
    
    if np.array_equal(test, out):
        return f"""def p(g):
 h,w={h},{w}
 return[[(j*9//(w-1))if w>1 else 0 for j in range(w)]for _ in range(h)]
"""
    
    test = np.zeros((h, w), dtype=int)
    for i in range(h):
        test[i, :] = (i * 9 // (h-1)) if h > 1 else 0
    
    if np.array_equal(test, out):
        return f"""def p(g):
 h,w={h},{w}
 return[[(i*9//(h-1))if h>1 else 0 for j in range(w)]for i in range(h)]
"""
    
    cx, cy = w // 2, h // 2
    max_dist = max(h, w)
    test = np.zeros((h, w), dtype=int)
    
    for i in range(h):
        for j in range(w):
            dist = int(((i-cy)**2 + (j-cx)**2)**0.5)
            test[i,j] = min(dist * 9 // max_dist, 9)
    
    if np.array_equal(test, out):
        return f"""def p(g):
 h,w={h},{w}
 cx,cy={cx},{cy}
 md={max_dist}
 res=[]
 for i in range(h):
  row=[]
  for j in range(w):
   d=int(((i-cy)**2+(j-cx)**2)**0.5)
   row.append(min(d*9//md,9))
  res.append(row)
 return res
"""
    return None

def handle_color_mixing(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    test = np.zeros_like(inp)
    
    for i in range(h):
        for j in range(w):
            neighbors = []
            for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                ni, nj = i + di, j + dj
                if 0 <= ni < h and 0 <= nj < w:
                    neighbors.append(inp[ni,nj])
            
            if neighbors:
                test[i,j] = sum(neighbors) // len(neighbors)
    
    if np.array_equal(test, out):
        return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   n=[]
   for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
    ni,nj=i+di,j+dj
    if 0<=ni<h and 0<=nj<w:
     n.append(g[ni][nj])
   if n:
    res[i][j]=sum(n)//len(n)
 return res
"""
    return None

def handle_palette_reduction(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    unique_out = np.unique(out)
    if len(unique_out) < len(np.unique(inp)):
        color_map = {}
        for val in np.unique(inp):
            nearest = min(unique_out, key=lambda x: abs(x - val))
            color_map[int(val)] = int(nearest)
        
        test = np.vectorize(lambda x: color_map.get(x, x))(inp)
        
        if np.array_equal(test, out):
            map_str = ','.join(f'{k}:{v}' for k,v in color_map.items())
            return f"""def p(g):
 m={{{map_str}}}
 return[[m.get(x,x)for x in r]for r in g]
"""
    return None

def handle_color_quantization(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    for levels in [2, 3, 4, 5]:
        test = (inp * levels // 10) * (10 // levels)
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[(x*{levels}//10)*{10//levels} for x in r]for r in g]
"""
    return None

def handle_color_spreading(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    spread = np.copy(inp)
    
    for iteration in range(3):
        new_spread = np.copy(spread)
        
        for i in range(h):
            for j in range(w):
                if spread[i,j] == 0:
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w and spread[ni,nj] != 0:
                            new_spread[i,j] = spread[ni,nj]
                            break
        
        spread = new_spread
        
        if np.array_equal(spread, out):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 s=[r[:]for r in g]
 for _ in range({iteration+1}):
  ns=[r[:]for r in s]
  for i in range(h):
   for j in range(w):
    if s[i][j]==0:
     for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
      ni,nj=i+di,j+dj
      if 0<=ni<h and 0<=nj<w and s[ni][nj]!=0:
       ns[i][j]=s[ni][nj]
       break
  s=ns
 return s
"""
    return None

def handle_color_waves(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    h, w = out.shape
    
    test = np.zeros((h, w), dtype=int)
    for i in range(h):
        for j in range(w):
            val = int(4.5 + 4.5 * math.sin(2 * math.pi * j / w))
            test[i,j] = val
    
    if np.array_equal(test, out):
        return f"""def p(g):
 import math
 h,w={h},{w}
 res=[]
 for i in range(h):
  row=[]
  for j in range(w):
   v=int(4.5+4.5*math.sin(2*math.pi*j/w))
   row.append(v)
  res.append(row)
 return res
"""
    return None

def handle_color_count(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    unique_colors = len(np.unique(inp))
    
    if np.all(out == unique_colors):
        return f"""def p(g):
 colors=set()
 for r in g:
  for c in r:
   colors.add(c)
 n=len(colors)
 return[[n]*len(g[0])for _ in g]
"""
    return None

def handle_color_frequency_sort(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    flat = inp.flatten()
    counts = Counter(flat)
    
    sorted_colors = sorted(counts.keys(), key=lambda x: (-counts[x], x))
    color_map = {c: i+1 for i, c in enumerate(sorted_colors) if c != 0}
    color_map[0] = 0
    
    test = np.vectorize(lambda x: color_map.get(x, x))(inp)
    
    if np.array_equal(test, out):
        map_str = ','.join(f'{k}:{v}' for k,v in color_map.items())
        return f"""def p(g):
 from collections import Counter
 c=Counter(x for r in g for x in r)
 s=sorted(c.keys(),key=lambda x:(-c[x],x))
 m={{c:i+1 for i,c in enumerate(s)if c!=0}}
 m[0]=0
 return[[m.get(x,x)for x in r]for r in g]
"""
    return None

def handle_replace_most_common(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    counts = Counter(inp.flatten())
    most_common = counts.most_common(1)[0][0]
    
    for new_color in range(10):
        test = np.where(inp == most_common, new_color, inp)
        if np.array_equal(test, out):
            return f"""def p(g):
 from collections import Counter
 c=Counter(x for r in g for x in r)
 mc=c.most_common(1)[0][0]
 return[[{new_color} if x==mc else x for x in r]for r in g]
"""
    return None

def handle_color_propagation(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    h, w = inp.shape
    
    test = np.copy(inp)
    for i in range(h):
        last_color = 0
        for j in range(w):
            if inp[i,j] != 0:
                last_color = inp[i,j]
            elif last_color != 0:
                test[i,j] = last_color
    
    if np.array_equal(test, out):
        return """def p(g):
 res=[r[:]for r in g]
 for i in range(len(g)):
  lc=0
  for j in range(len(g[0])):
   if g[i][j]!=0:
    lc=g[i][j]
   elif lc!=0:
    res[i][j]=lc
 return res
"""
    
    test = np.copy(inp)
    for j in range(w):
        last_color = 0
        for i in range(h):
            if inp[i,j] != 0:
                last_color = inp[i,j]
            elif last_color != 0:
                test[i,j] = last_color
    
    if np.array_equal(test, out):
        return """def p(g):
 res=[r[:]for r in g]
 for j in range(len(g[0])):
  lc=0
  for i in range(len(g)):
   if g[i][j]!=0:
    lc=g[i][j]
   elif lc!=0:
    res[i][j]=lc
 return res
"""
    return None

def handle_flood_fill(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp[0,0] == 0 and out[0,0] != 0:
        fill_color = out[0,0]
        
        h, w = inp.shape
        filled = np.copy(inp)
        to_fill = [(0, 0)]
        filled[0,0] = fill_color
        
        while to_fill:
            y, x = to_fill.pop()
            for dy, dx in [(0,1), (1,0), (0,-1), (-1,0)]:
                ny, nx = y + dy, x + dx
                if 0 <= ny < h and 0 <= nx < w and filled[ny,nx] == 0:
                    filled[ny,nx] = fill_color
                    to_fill.append((ny, nx))
        
        if np.array_equal(filled, out):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 fc={fill_color}
 q=[(0,0)]
 res[0][0]=fc
 while q:
  y,x=q.pop()
  for dy,dx in[(0,1),(1,0),(0,-1),(-1,0)]:
   ny,nx=y+dy,x+dx
   if 0<=ny<h and 0<=nx<w and res[ny][nx]==0:
    res[ny][nx]=fc
    q.append((ny,nx))
 return res
"""
    return None

def handle_color_mapping(task_data, analysis):
    color_map = {}
    for pos, changes in analysis['color_changes'].items():
        if len(changes) == 1:
            src, dest = next(iter(changes))
            if src not in color_map:
                color_map[src] = dest
            elif color_map[src] != dest:
                return None
    
    if not color_map:
        return None
    
    cases = ','.join([f'{int(s)}:{int(d)}' for s,d in color_map.items()])
    return f"""def p(g):
 m={{{cases}}}
 return[[m.get(x,x)for x in r]for r in g]
"""

# ==================== SECTION 8: GEOMETRIC TRANSFORMATIONS ====================

def handle_rotate_90(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if np.array_equal(np.rot90(inp, -1), out):
        return """def p(g):
 return[list(r)for r in zip(*g[::-1])]
"""
    return None

def handle_rotate_180(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if np.array_equal(np.rot90(inp, 2), out):
        return """def p(g):
 return[r[::-1]for r in g[::-1]]
"""
    return None

def handle_rotate_270(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if np.array_equal(np.rot90(inp, 1), out):
        return """def p(g):
 return[list(r)for r in zip(*g)][::-1]
"""
    return None

def handle_flip_horizontal(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if np.array_equal(np.fliplr(inp), out):
        return """def p(g):
 return[r[::-1]for r in g]
"""
    return None

def handle_flip_vertical(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if np.array_equal(np.flipud(inp), out):
        return """def p(g):
 return g[::-1]
"""
    return None

def handle_transpose(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    if np.array_equal(inp.T, out):
        return """def p(g):
 return[list(r)for r in zip(*g)]
"""
    return None

def handle_rotate_45(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    h, w = inp.shape
    
    diagonals = defaultdict(list)
    for i in range(h):
        for j in range(w):
            diagonals[i+j].append(inp[i,j])
    
    if len(diagonals) == out.shape[0]:
        out_rows = []
        for d in sorted(diagonals.keys()):
            out_rows.append(diagonals[d])
        
        max_len = max(len(row) for row in out_rows)
        for row in out_rows:
            row.extend([0] * (max_len - len(row)))
        
        if np.array_equal(np.array(out_rows), out):
            return """def p(g):
 h,w=len(g),len(g[0])
 d={}
 for i in range(h):
  for j in range(w):
   k=i+j
   if k not in d:d[k]=[]
   d[k].append(g[i][j])
 res=[]
 for k in sorted(d.keys()):
  res.append(d[k])
 m=max(len(r)for r in res)
 for r in res:
  r.extend([0]*(m-len(r)))
 return res
"""
    return None

def handle_shear_transform(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    h, w = inp.shape
    
    for shear in range(1, 4):
        test = np.zeros_like(out)
        for i in range(h):
            for j in range(w):
                new_j = (j + i * shear) % out.shape[1]
                if new_j < out.shape[1]:
                    test[i, new_j] = inp[i, j]
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*{out.shape[1]} for _ in range(h)]
 for i in range(h):
  for j in range(w):
   nj=(j+i*{shear})%{out.shape[1]}
   res[i][nj]=g[i][j]
 return res
"""
    return None

def handle_perspective_transform(task_data, analysis):
    return None

def handle_upscale(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    for h_scale in range(1, 4):
        for w_scale in range(1, 4):
            if (inp.shape[0]*h_scale == out.shape[0] and 
                inp.shape[1]*w_scale == out.shape[1]):
                test = np.repeat(np.repeat(inp, h_scale, axis=0), w_scale, axis=1)
                if np.array_equal(test, out):
                    return f"""def p(g):
 return[[x for x in row for _ in range({w_scale})]
        for row in g for _ in range({h_scale})]
"""
    return None

def handle_downscale(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    for h_scale in range(1, 4):
        for w_scale in range(1, 4):
            if (inp.shape[0]//h_scale == out.shape[0] and 
                inp.shape[1]//w_scale == out.shape[1]):
                test = inp[::h_scale, ::w_scale]
                if np.array_equal(test, out):
                    return f"""def p(g):
 return[row[::{w_scale}]for row in g[::{h_scale}]]
"""
    return None

def handle_pad_to_square(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    h, w = inp.shape
    
    if h != w and out.shape[0] == out.shape[1]:
        size = max(h, w)
        if out.shape == (size, size):
            pad_h = (size - h) // 2
            pad_w = (size - w) // 2
            
            test = np.zeros((size, size), dtype=int)
            test[pad_h:pad_h+h, pad_w:pad_w+w] = inp
            
            if np.array_equal(test, out):
                return f"""def p(g):
 h,w=len(g),len(g[0])
 s=max(h,w)
 ph,pw=(s-h)//2,(s-w)//2
 res=[[0]*s for _ in range(s)]
 for i in range(h):
  for j in range(w):
   res[ph+i][pw+j]=g[i][j]
 return res
"""
    return None

def handle_crop_to_content(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    rows, cols = np.where(inp > 0)
    if len(rows) == 0:
        return None
        
    min_row, max_row = rows.min(), rows.max()
    min_col, max_col = cols.min(), cols.max()
    
    cropped = inp[min_row:max_row+1, min_col:max_col+1]
    
    if np.array_equal(cropped, out):
        return """def p(g):
 rows,cols=[],[]
 for i in range(len(g)):
  for j in range(len(g[0])):
   if g[i][j]>0:
    rows.append(i)
    cols.append(j)
 if not rows:return g
 r0,r1=min(rows),max(rows)+1
 c0,c1=min(cols),max(cols)+1
 return[r[c0:c1]for r in g[r0:r1]]
"""
    return None

def handle_zoom_center(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    h, w = inp.shape
    oh, ow = out.shape
    
    if oh < h and ow < w:
        y = (h - oh) // 2
        x = (w - ow) // 2
        
        if np.array_equal(inp[y:y+oh, x:x+ow], out):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 oh,ow={oh},{ow}
 y,x=(h-oh)//2,(w-ow)//2
 return[r[x:x+ow]for r in g[y:y+oh]]
"""
    return None

def handle_aspect_ratio_change(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    ih, iw = inp.shape
    oh, ow = out.shape
    
    if oh != ih or ow != iw:
        test = np.zeros((oh, ow), dtype=int)
        for i in range(oh):
            for j in range(ow):
                src_i = i * ih // oh
                src_j = j * iw // ow
                test[i,j] = inp[src_i, src_j]
        
        if np.array_equal(test, out):
            return f"""def p(g):
 ih,iw=len(g),len(g[0])
 oh,ow={oh},{ow}
 res=[]
 for i in range(oh):
  row=[]
  for j in range(ow):
   si=i*ih//oh
   sj=j*iw//ow
   row.append(g[si][sj])
  res.append(row)
 return res
"""
    return None

# ==================== SECTION 9: PATTERN REPETITION AND TILING ====================

def handle_repeat_pattern(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    for h in range(1, min(5, inp.shape[0]+1)):
        for w in range(1, min(5, inp.shape[1]+1)):
            pattern = inp[:h,:w]
            test = np.tile(pattern, (out.shape[0]//h + 1, out.shape[1]//w + 1))
            test = test[:out.shape[0], :out.shape[1]]
            if np.array_equal(out, test):
                oh, ow = out.shape
                return f"""def p(g):
 pat=[r[:{w}]for r in g[:{h}]]
 res=[]
 for i in range({oh}):
  row=[]
  for j in range({ow}):
   row.append(pat[i%{h}][j%{w}])
  res.append(row)
 return res
"""
    return None

def handle_tile_pattern(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    ih, iw = inp.shape
    oh, ow = out.shape
    
    for th in range(1, min(ih+1, 5)):
        for tw in range(1, min(iw+1, 5)):
            if oh % th == 0 and ow % tw == 0:
                tile = inp[:th, :tw]
                tiled = np.tile(tile, (oh//th, ow//tw))
                
                if np.array_equal(tiled, out):
                    return f"""def p(g):
 tile=[r[:{tw}]for r in g[:{th}]]
 h,w={oh},{ow}
 res=[]
 for i in range(h):
  row=[]
  for j in range(w):
   row.append(tile[i%{th}][j%{tw}])
  res.append(row)
 return res
"""
    return None

def handle_mirror_pattern(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    mirrored = np.hstack([inp, np.fliplr(inp)])
    if np.array_equal(mirrored, out):
        return """def p(g):
 return[r+r[::-1]for r in g]
"""
    
    mirrored = np.vstack([inp, np.flipud(inp)])
    if np.array_equal(mirrored, out):
        return """def p(g):
 return g+g[::-1]
"""
    
    top = np.hstack([inp, np.fliplr(inp)])
    bottom = np.hstack([np.flipud(inp), np.flipud(np.fliplr(inp))])
    mirrored = np.vstack([top, bottom])
    if np.array_equal(mirrored, out):
        return """def p(g):
 t=[r+r[::-1]for r in g]
 b=[r+r[::-1]for r in g[::-1]]
 return t+b
"""
    return None

def handle_extract_pattern(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    ih, iw = inp.shape
    oh, ow = out.shape
    
    if oh <= ih and ow <= iw:
        for y in range(ih - oh + 1):
            for x in range(iw - ow + 1):
                if np.array_equal(inp[y:y+oh, x:x+ow], out):
                    if y == 0 and x == 0:
                        return f"""def p(g):
 return[r[:{ow}]for r in g[:{oh}]]
"""
                    if y == (ih-oh)//2 and x == (iw-ow)//2:
                        return f"""def p(g):
 h,w=len(g),len(g[0])
 oh,ow={oh},{ow}
 y,x=(h-oh)//2,(w-ow)//2
 return[r[x:x+ow]for r in g[y:y+oh]]
"""
    return None

def handle_apply_mask(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    mask = (out != 0)
    
    checkerboard = np.indices(inp.shape).sum(axis=0) % 2
    if np.array_equal(mask, checkerboard):
        test = np.where(checkerboard, inp, 0)
        if np.array_equal(test, out):
            return """def p(g):
 return[[g[i][j]if(i+j)%2 else 0 for j in range(len(g[0]))]for i in range(len(g))]
"""
    
    diag_mask = np.eye(inp.shape[0], inp.shape[1], dtype=bool)
    if np.array_equal(mask, diag_mask):
        test = np.where(diag_mask, inp, 0)
        if np.array_equal(test, out):
            return """def p(g):
 return[[g[i][j]if i==j else 0 for j in range(len(g[0]))]for i in range(len(g))]
"""
    return None

def handle_pattern_substitution(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape[0] >= 2 and inp.shape[1] >= 2:
        patterns = {
            ((0,0),(0,0)): ((1,1),(1,1)),
            ((1,0),(0,1)): ((2,2),(2,2)),
            ((0,1),(1,0)): ((3,3),(3,3)),
        }
        
        test = np.copy(inp)
        for i in range(inp.shape[0]-1):
            for j in range(inp.shape[1]-1):
                pattern = ((inp[i,j], inp[i,j+1]), 
                          (inp[i+1,j], inp[i+1,j+1]))
                
                if pattern in patterns:
                    replacement = patterns[pattern]
                    test[i,j] = replacement[0][0]
                    test[i,j+1] = replacement[0][1]
                    test[i+1,j] = replacement[1][0]
                    test[i+1,j+1] = replacement[1][1]
        
        if np.array_equal(test, out):
            return """def p(g):
 res=[r[:]for r in g]
 return res
"""
    return None

# ==================== SECTION 10: OBJECT MANIPULATION PATTERNS ====================

def handle_count_objects(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    labeled, n = label(inp > 0)
    
    if out.shape == (1,1) and out[0,0] == n:
        return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 _,n=label(np.array(g)>0)
 return[[n]]
"""
    
    if np.all(out == n):
        return f"""def p(g):
 from scipy.ndimage import label
 import numpy as np
 _,n=label(np.array(g)>0)
 return[[n]*len(g[0])for _ in g]
"""
    return None

def handle_largest_object(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    labeled, n = label(inp > 0)
    if n == 0:
        return None
    
    sizes = [(labeled == i).sum() for i in range(1, n+1)]
    largest = np.argmax(sizes) + 1
    
    result = np.where(labeled == largest, inp, 0)
    
    if np.array_equal(result, out):
        return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 arr=np.array(g)
 l,n=label(arr>0)
 if n==0:return g
 sizes=[(l==i).sum()for i in range(1,n+1)]
 largest=np.argmax(sizes)+1
 return np.where(l==largest,arr,0).tolist()
"""
    return None

def handle_move_objects(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    for dy in range(-3, 4):
        for dx in range(-3, 4):
            if dy == 0 and dx == 0:
                continue
            
            test = np.zeros_like(inp)
            h, w = inp.shape
            
            for i in range(h):
                for j in range(w):
                    if inp[i,j] != 0:
                        ni, nj = i + dy, j + dx
                        if 0 <= ni < h and 0 <= nj < w:
                            test[ni,nj] = inp[i,j]
            
            if np.array_equal(test, out):
                return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    ni,nj=i+{dy},j+{dx}
    if 0<=ni<h and 0<=nj<w:
     res[ni][nj]=g[i][j]
 return res
"""
    return None

def handle_gravity(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    
    test = np.zeros_like(inp)
    for j in range(w):
        col = inp[:,j]
        non_zero = col[col != 0]
        test[h-len(non_zero):,j] = non_zero
    
    if np.array_equal(test, out):
        return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for j in range(w):
  nz=[g[i][j]for i in range(h)if g[i][j]!=0]
  for k,v in enumerate(nz):
   res[h-len(nz)+k][j]=v
 return res
"""
    
    test = np.zeros_like(inp)
    for j in range(w):
        col = inp[:,j]
        non_zero = col[col != 0]
        test[:len(non_zero),j] = non_zero
    
    if np.array_equal(test, out):
        return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for j in range(w):
  nz=[g[i][j]for i in range(h)if g[i][j]!=0]
  for k,v in enumerate(nz):
   res[k][j]=v
 return res
"""
    return None

def handle_connect_same_color(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    test = np.copy(inp)
    
    for color in range(1, 10):
        positions = np.argwhere(inp == color)
        if len(positions) < 2:
            continue
        
        for i in range(len(positions)):
            for j in range(i+1, len(positions)):
                y1, x1 = positions[i]
                y2, x2 = positions[j]
                
                if y1 == y2:
                    for x in range(min(x1,x2), max(x1,x2)+1):
                        test[y1,x] = color
                elif x1 == x2:
                    for y in range(min(y1,y2), max(y1,y2)+1):
                        test[y,x1] = color
    
    if np.array_equal(test, out):
        return """def p(g):
 res=[r[:]for r in g]
 for c in range(1,10):
  pos=[(i,j)for i in range(len(g))for j in range(len(g[0]))if g[i][j]==c]
  for i in range(len(pos)):
   for j in range(i+1,len(pos)):
    y1,x1=pos[i]
    y2,x2=pos[j]
    if y1==y2:
     for x in range(min(x1,x2),max(x1,x2)+1):
      res[y1][x]=c
    elif x1==x2:
     for y in range(min(y1,y2),max(y1,y2)+1):
      res[y][x1]=c
 return res
"""
    return None

def handle_outline_objects(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    edges = np.zeros_like(inp)
    
    for i in range(h):
        for j in range(w):
            if inp[i,j] != 0:
                is_edge = False
                for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                    ni, nj = i + di, j + dj
                    if not (0 <= ni < h and 0 <= nj < w) or inp[ni,nj] == 0:
                        is_edge = True
                        break
                if is_edge:
                    edges[i,j] = inp[i,j]
    
    if np.array_equal(edges, out):
        return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    edge=False
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if not(0<=ni<h and 0<=nj<w)or g[ni][nj]==0:
      edge=True
      break
    if edge:
     res[i][j]=g[i][j]
 return res
"""
    return None

def handle_fill_objects(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    from scipy.ndimage import binary_fill_holes
    
    test = np.copy(inp)
    for color in range(1, 10):
        mask = inp == color
        if np.any(mask):
            filled = binary_fill_holes(mask)
            test[filled] = color
    
    if np.array_equal(test, out):
        return """def p(g):
 from scipy.ndimage import binary_fill_holes
 import numpy as np
 t=np.array(g)
 res=t.copy()
 for c in range(1,10):
  m=t==c
  if np.any(m):
   f=binary_fill_holes(m)
   res[f]=c
 return res.tolist()
"""
    return None

def handle_object_intersection(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    intersections = np.zeros_like(inp)
    
    for i in range(h):
        for j in range(w):
            colors = set()
            for di in range(-1, 2):
                for dj in range(-1, 2):
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w and inp[ni,nj] != 0:
                        colors.add(inp[ni,nj])
            
            if len(colors) > 1:
                intersections[i,j] = len(colors)
    
    if np.array_equal(intersections, out):
        return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   c=set()
   for di in range(-1,2):
    for dj in range(-1,2):
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w and g[ni][nj]!=0:
      c.add(g[ni][nj])
   if len(c)>1:
    res[i][j]=len(c)
 return res
"""
    return None

# ==================== SECTION 11: LINE AND SYMMETRY PATTERNS ====================

def handle_draw_lines(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    
    ch, cw = h // 2, w // 2
    test = np.copy(inp)
    
    if h > 0:
        test[ch,:] = 1
    if w > 0:
        test[:,cw] = 1
    
    if np.array_equal(test, out):
        return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 if h>0:
  res[h//2]=[1]*w
 if w>0:
  for i in range(h):
   res[i][w//2]=1
 return res
"""
    
    test = np.copy(inp)
    for i in range(min(h,w)):
        test[i,i] = 1
    
    if np.array_equal(test, out):
        return """def p(g):
 res=[r[:]for r in g]
 for i in range(min(len(g),len(g[0]))):
  res[i][i]=1
 return res
"""
    return None

def handle_detect_lines(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    test = np.zeros_like(inp)
    
    for i in range(h):
        if np.all(inp[i,:] == inp[i,0]) and inp[i,0] != 0:
            test[i,:] = inp[i,0]
    
    for j in range(w):
        if np.all(inp[:,j] == inp[0,j]) and inp[0,j] != 0:
            test[:,j] = inp[0,j]
    
    if np.array_equal(test, out):
        return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  if all(g[i][j]==g[i][0]for j in range(w))and g[i][0]!=0:
   res[i]=[g[i][0]]*w
 for j in range(w):
  if all(g[i][j]==g[0][j]for i in range(h))and g[0][j]!=0:
   for i in range(h):
    res[i][j]=g[0][j]
 return res
"""
    return None

def handle_extend_lines(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    test = np.copy(inp)
    
    for i in range(h):
        segments = []
        start = None
        
        for j in range(w):
            if inp[i,j] != 0:
                if start is None:
                    start = j
            else:
                if start is not None:
                    segments.append((start, j-1, inp[i,start]))
                    start = None
        
        if start is not None:
            segments.append((start, w-1, inp[i,start]))
        
        for s, e, color in segments:
            if e - s >= 1:
                test[i,:] = color
    
    if np.array_equal(test, out):
        return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(h):
  segs=[]
  s=None
  for j in range(w):
   if g[i][j]!=0:
    if s is None:
     s=j
   else:
    if s is not None:
     segs.append((s,j-1,g[i][s]))
     s=None
  if s is not None:
   segs.append((s,w-1,g[i][s]))
  for s,e,c in segs:
   if e-s>=1:
    res[i]=[c]*w
 return res
"""
    return None

def handle_line_intersection(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    intersections = np.zeros_like(inp)
    
    for i in range(1, h-1):
        for j in range(1, w-1):
            if inp[i,j] != 0:
                if (inp[i-1,j] != 0 and inp[i+1,j] != 0 and
                    inp[i,j-1] != 0 and inp[i,j+1] != 0):
                    intersections[i,j] = 9
    
    if np.array_equal(intersections, out):
        return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(1,h-1):
  for j in range(1,w-1):
   if g[i][j]!=0:
    if g[i-1][j]!=0 and g[i+1][j]!=0 and g[i][j-1]!=0 and g[i][j+1]!=0:
     res[i][j]=9
 return res
"""
    return None

def handle_perpendicular_lines(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    test = np.copy(inp)
    
    for i in range(h):
        for j in range(w):
            if inp[i,j] != 0:
                test[i,:] = inp[i,j]
                test[:,j] = inp[i,j]
    
    if np.array_equal(test, out):
        return """def p(g):
 res=[r[:]for r in g]
 h,w=len(g),len(g[0])
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    for k in range(w):
     res[i][k]=g[i][j]
    for k in range(h):
     res[k][j]=g[i][j]
 return res
"""
    return None

def handle_make_symmetric(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    
    test = np.copy(inp)
    for i in range(h//2):
        test[h-1-i,:] = test[i,:]
    
    if np.array_equal(test, out):
        return """def p(g):
 h=len(g)
 res=[r[:]for r in g]
 for i in range(h//2):
  res[h-1-i]=res[i][:]
 return res
"""
    
    test = np.copy(inp)
    for j in range(w//2):
        test[:,w-1-j] = test[:,j]
    
    if np.array_equal(test, out):
        return """def p(g):
 w=len(g[0])
 res=[r[:]for r in g]
 for i in range(len(g)):
  for j in range(w//2):
   res[i][w-1-j]=res[i][j]
 return res
"""
    return None

def handle_complete_pattern(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape == out.shape:
        h, w = inp.shape
        
        test = np.copy(inp)
        for i in range(h):
            for j in range(w):
                if i < j and inp[i,j] != 0:
                    test[j,i] = inp[i,j]
                elif i > j and inp[i,j] != 0:
                    test[j,i] = inp[i,j]
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(h):
  for j in range(w):
   if i<j and g[i][j]!=0:
    res[j][i]=g[i][j]
   elif i>j and g[i][j]!=0:
    res[j][i]=g[i][j]
 return res
"""
    return None

def handle_find_symmetry_axis(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    
    is_vsym = True
    for i in range(h):
        for j in range(w//2):
            if inp[i,j] != inp[i,w-1-j]:
                is_vsym = False
                break
    
    if is_vsym:
        test = np.copy(inp)
        test[:,w//2] = 9
        if np.array_equal(test, out):
            return """def p(g):
 res=[r[:]for r in g]
 w=len(g[0])
 for i in range(len(g)):
  res[i][w//2]=9
 return res
"""
    return None

def handle_rotational_symmetry(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape[0] == inp.shape[1] and out.shape == inp.shape:
        n = inp.shape[0]
        test = np.copy(inp)
        
        for i in range(n):
            for j in range(n):
                if inp[i,j] != 0:
                    test[i,j] = inp[i,j]
                    test[j,n-1-i] = inp[i,j]
                    test[n-1-i,n-1-j] = inp[i,j]
                    test[n-1-j,i] = inp[i,j]
        
        if np.array_equal(test, out):
            return """def p(g):
 n=len(g)
 res=[r[:]for r in g]
 for i in range(n):
  for j in range(n):
   if g[i][j]!=0:
    res[i][j]=g[i][j]
    res[j][n-1-i]=g[i][j]
    res[n-1-i][n-1-j]=g[i][j]
    res[n-1-j][i]=g[i][j]
 return res
"""
    return None

# ==================== SECTION 12: CELLULAR AUTOMATA PATTERNS ====================

def handle_conway_step(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    test = np.zeros_like(inp)
    
    for i in range(h):
        for j in range(w):
            neighbors = 0
            for di in [-1, 0, 1]:
                for dj in [-1, 0, 1]:
                    if di == 0 and dj == 0:
                        continue
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w and inp[ni,nj] != 0:
                        neighbors += 1
            
            if inp[i,j] != 0:
                if neighbors in [2, 3]:
                    test[i,j] = inp[i,j]
            else:
                if neighbors == 3:
                    test[i,j] = 1
    
    if np.array_equal(test, out):
        return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   n=0
   for di in[-1,0,1]:
    for dj in[-1,0,1]:
     if di==0 and dj==0:continue
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w and g[ni][nj]!=0:
      n+=1
   if g[i][j]!=0:
    if n in[2,3]:
     res[i][j]=g[i][j]
   else:
    if n==3:
     res[i][j]=1
 return res
"""
    return None

def handle_cellular_automaton(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    
    test = np.zeros_like(inp)
    
    for i in range(h):
        for j in range(w):
            color_counts = defaultdict(int)
            for di in [-1, 0, 1]:
                for dj in [-1, 0, 1]:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w:
                        color_counts[inp[ni,nj]] += 1
            
            most_common = max(color_counts.items(), key=lambda x: x[1])[0]
            test[i,j] = most_common
    
    if np.array_equal(test, out):
        return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   cc={}
   for di in[-1,0,1]:
    for dj in[-1,0,1]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w:
      c=g[ni][nj]
      cc[c]=cc.get(c,0)+1
   mc=max(cc.items(),key=lambda x:x[1])[0]
   res[i][j]=mc
 return res
"""
    return None

def handle_voronoi_regions(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    seeds = []
    h, w = inp.shape
    for i in range(h):
        for j in range(w):
            if inp[i,j] != 0:
                seeds.append((i, j, inp[i,j]))
    
    if not seeds:
        return None
    
    voronoi = np.zeros_like(inp)
    for i in range(h):
        for j in range(w):
            min_dist = float('inf')
            nearest_color = 0
            
            for si, sj, color in seeds:
                dist = abs(i - si) + abs(j - sj)
                if dist < min_dist:
                    min_dist = dist
                    nearest_color = color
            
            voronoi[i,j] = nearest_color
    
    if np.array_equal(voronoi, out):
        return """def p(g):
 h,w=len(g),len(g[0])
 s=[]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    s.append((i,j,g[i][j]))
 v=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   md=999
   nc=0
   for si,sj,c in s:
    d=abs(i-si)+abs(j-sj)
    if d<md:
     md=d
     nc=c
   v[i][j]=nc
 return v
"""
    return None

def handle_fractal_generation(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if out.shape[0] == out.shape[1]:
        n = out.shape[0]
        test = np.zeros((n, n), dtype=int)
        
        for i in range(n):
            for j in range(i+1):
                if j == 0 or j == i:
                    test[i,j] = 1
                else:
                    test[i,j] = (test[i-1,j-1] + test[i-1,j]) % 2
        
        if np.array_equal(test, out):
            return f"""def p(g):
 n={n}
 t=[[0]*n for _ in range(n)]
 for i in range(n):
  for j in range(i+1):
   if j==0 or j==i:
    t[i][j]=1
   else:
    t[i][j]=(t[i-1][j-1]+t[i-1][j])%2
 return t
"""
    return None

def handle_maze_solve(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    start = np.argwhere(inp == 2)
    end = np.argwhere(inp == 3)
    
    if len(start) == 1 and len(end) == 1:
        sy, sx = start[0]
        ey, ex = end[0]
        
        h, w = inp.shape
        visited = set()
        queue = [(sy, sx, [(sy, sx)])]
        path = None
        
        while queue and not path:
            y, x, current_path = queue.pop(0)
            
            if (y, x) == (ey, ex):
                path = current_path
                break
            
            if (y, x) in visited:
                continue
            visited.add((y, x))
            
            for dy, dx in [(0,1), (1,0), (0,-1), (-1,0)]:
                ny, nx = y + dy, x + dx
                if (0 <= ny < h and 0 <= nx < w and 
                    inp[ny,nx] != 1 and (ny,nx) not in visited):
                    queue.append((ny, nx, current_path + [(ny, nx)]))
        
        if path:
            test = np.copy(inp)
            for y, x in path[1:-1]:
                test[y,x] = 4
            
            if np.array_equal(test, out):
                return """def p(g):
 import numpy as np
 h,w=len(g),len(g[0])
 inp=np.array(g)
 start=np.argwhere(inp==2)
 end=np.argwhere(inp==3)
 if len(start)==1 and len(end)==1:
  sy,sx=start[0]
  ey,ex=end[0]
  visited=set()
  queue=[(sy,sx,[(sy,sx)])]
  path=None
  while queue and not path:
   y,x,cp=queue.pop(0)
   if(y,x)==(ey,ex):
    path=cp
    break
   if(y,x)in visited:
    continue
   visited.add((y,x))
   for dy,dx in[(0,1),(1,0),(0,-1),(-1,0)]:
    ny,nx=y+dy,x+dx
    if 0<=ny<h and 0<=nx<w and g[ny][nx]!=1 and(ny,nx)not in visited:
     queue.append((ny,nx,cp+[(ny,nx)]))
  if path:
   res=[r[:]for r in g]
   for y,x in path[1:-1]:
    res[y][x]=4
   return res
 return g
"""
    return None

def handle_path_finding(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    colors = defaultdict(list)
    h, w = inp.shape
    
    for i in range(h):
        for j in range(w):
            if inp[i,j] != 0:
                colors[inp[i,j]].append((i,j))
    
    test = np.copy(inp)
    
    for color, points in colors.items():
        if len(points) == 2:
            (y1, x1), (y2, x2) = points
            
            if y1 == y2:
                for x in range(min(x1,x2), max(x1,x2)+1):
                    test[y1,x] = color
            elif x1 == x2:
                for y in range(min(y1,y2), max(y1,y2)+1):
                    test[y,x1] = color
            else:
                for y in range(min(y1,y2), max(y1,y2)+1):
                    test[y,x1] = color
                for x in range(min(x1,x2), max(x1,x2)+1):
                    test[y2,x] = color
    
    if np.array_equal(test, out):
        return """def p(g):
 from collections import defaultdict
 h,w=len(g),len(g[0])
 colors=defaultdict(list)
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    colors[g[i][j]].append((i,j))
 res=[r[:]for r in g]
 for c,pts in colors.items():
  if len(pts)==2:
   (y1,x1),(y2,x2)=pts
   if y1==y2:
    for x in range(min(x1,x2),max(x1,x2)+1):
     res[y1][x]=c
   elif x1==x2:
    for y in range(min(y1,y2),max(y1,y2)+1):
     res[y][x1]=c
   else:
    for y in range(min(y1,y2),max(y1,y2)+1):
     res[y][x1]=c
    for x in range(min(x1,x2),max(x1,x2)+1):
     res[y2][x]=c
 return res
"""
    return None

# ==================== SECTION 13: GRID OPERATIONS ====================

def handle_split_grid(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    h, w = inp.shape
    oh, ow = out.shape
    
    if oh == h//2 and ow == w//2:
        if np.array_equal(inp[:h//2, :w//2], out):
            return """def p(g):
 h,w=len(g),len(g[0])
 return[r[:w//2]for r in g[:h//2]]
"""
        if np.array_equal(inp[:h//2, w//2:], out):
            return """def p(g):
 h,w=len(g),len(g[0])
 return[r[w//2:]for r in g[:h//2]]
"""
        if np.array_equal(inp[h//2:, :w//2], out):
            return """def p(g):
 h,w=len(g),len(g[0])
 return[r[:w//2]for r in g[h//2:]]
"""
        if np.array_equal(inp[h//2:, w//2:], out):
            return """def p(g):
 h,w=len(g),len(g[0])
 return[r[w//2:]for r in g[h//2:]]
"""
    return None

def handle_merge_grids(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    h, w = inp.shape
    oh, ow = out.shape
    
    if oh == 2*h and ow == 2*w:
        test = np.vstack([
            np.hstack([inp, inp]),
            np.hstack([inp, inp])
        ])
        if np.array_equal(test, out):
            return """def p(g):
 return[r+r for r in g]+[r+r for r in g]
"""
    return None

def handle_overlay_grids(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    
    test = np.copy(inp)
    for j in range(0, w, 3):
        test[:,j] = np.where(test[:,j] == 0, 1, test[:,j])
    for i in range(0, h, 3):
        test[i,:] = np.where(test[i,:] == 0, 1, test[i,:])
    
    if np.array_equal(test, out):
        return """def p(g):
 res=[r[:]for r in g]
 h,w=len(g),len(g[0])
 for j in range(0,w,3):
  for i in range(h):
   if res[i][j]==0:
    res[i][j]=1
 for i in range(0,h,3):
  for j in range(w):
   if res[i][j]==0:
    res[i][j]=1
 return res
"""
    return None

def handle_subtract_grids(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    for color in range(1, 10):
        test = np.where(inp == color, 0, inp)
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[0 if x=={color} else x for x in r]for r in g]
"""
    
    return None

# ==================== SECTION 14: COMPLEX TRANSFORMATIONS ====================

def handle_multi_step_transform(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    for rotation in [1, 2, 3]:
        rotated = np.rot90(inp, rotation)
        
        for scale in [2, 3]:
            if (rotated.shape[0] * scale == out.shape[0] and
                rotated.shape[1] * scale == out.shape[1]):
                
                scaled = np.repeat(np.repeat(rotated, scale, axis=0), scale, axis=1)
                
                if np.array_equal(scaled, out):
                    return f"""def p(g):
 import numpy as np
 r=np.rot90(g,{rotation})
 return np.repeat(np.repeat(r,{scale},axis=0),{scale},axis=1).tolist()
"""
    return None

def handle_conditional_transform(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    if inp.shape != out.shape:
        return None
    
    h, w = inp.shape
    test = np.zeros_like(inp)
    
    for i in range(h):
        for j in range(w):
            if inp[i,j] != 0:
                count = 0
                for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w and inp[ni,nj] != 0:
                        count += 1
                
                test[i,j] = count + 1
    
    if np.array_equal(test, out):
        return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    c=0
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w and g[ni][nj]!=0:
      c+=1
    res[i][j]=c+1
 return res
"""
    return None

def handle_recursive_transform(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    h, w = inp.shape
    
    test = np.copy(inp)
    changed = True
    
    while changed:
        changed = False
        for i in range(1, h-1):
            for j in range(1, w-1):
                if test[i,j] == 0:
                    neighbors = [test[i+di,j+dj] for di,dj in [(0,1),(1,0),(0,-1),(-1,0)]]
                    
                    if all(n != 0 for n in neighbors):
                        color_counts = Counter(neighbors)
                        test[i,j] = color_counts.most_common(1)[0][0]
                        changed = True
    
    if np.array_equal(test, out):
        return """def p(g):
 from collections import Counter
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 changed=True
 while changed:
  changed=False
  for i in range(1,h-1):
   for j in range(1,w-1):
    if res[i][j]==0:
     n=[res[i+di][j+dj]for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]]
     if all(x!=0 for x in n):
      c=Counter(n)
      res[i][j]=c.most_common(1)[0][0]
      changed=True
 return res
"""
    return None

def handle_learned_transform(task_data, analysis):
    if len(task_data['train']) < 2:
        return None
    return None

# ==================== SECTION 15: HELPER FUNCTIONS ====================

def to_1d_raster(grid, order='row_major'):
    h, w = len(grid), len(grid[0]) if grid else 0
    
    if order == 'row_major':
        return [grid[i][j] for i in range(h) for j in range(w)]
    elif order == 'col_major':
        return [grid[i][j] for j in range(w) for i in range(h)]
    elif order == 'row_major_rev':
        return [grid[i][j] for i in range(h-1,-1,-1) for j in range(w-1,-1,-1)]
    elif order == 'spiral':
        result = []
        top, bottom, left, right = 0, h-1, 0, w-1
        while top <= bottom and left <= right:
            for j in range(left, right+1):
                result.append(grid[top][j])
            top += 1
            for i in range(top, bottom+1):
                result.append(grid[i][right])
            right -= 1
            if top <= bottom:
                for j in range(right, left-1, -1):
                    result.append(grid[bottom][j])
                bottom -= 1
            if left <= right:
                for i in range(bottom, top-1, -1):
                    result.append(grid[i][left])
                left += 1
        return result
    elif order == 'zigzag':
        result = []
        for i in range(h):
            if i % 2 == 0:
                result.extend(grid[i])
            else:
                result.extend(grid[i][::-1])
        return result
    return []

def from_1d_raster(arr, h, w, order='row_major'):
    if len(arr) != h * w:
        return None
        
    grid = [[0]*w for _ in range(h)]
    
    if order == 'row_major':
        for idx, val in enumerate(arr):
            i, j = idx // w, idx % w
            grid[i][j] = val
    elif order == 'col_major':
        for idx, val in enumerate(arr):
            j, i = idx // h, idx % h
            grid[i][j] = val
    elif order == 'row_major_rev':
        for idx, val in enumerate(arr):
            i, j = (h-1) - idx // w, (w-1) - idx % w
            grid[i][j] = val
    elif order == 'spiral':
        idx = 0
        top, bottom, left, right = 0, h-1, 0, w-1
        while top <= bottom and left <= right and idx < len(arr):
            for j in range(left, right+1):
                if idx < len(arr):
                    grid[top][j] = arr[idx]
                    idx += 1
            top += 1
            for i in range(top, bottom+1):
                if idx < len(arr):
                    grid[i][right] = arr[idx]
                    idx += 1
            right -= 1
            if top <= bottom:
                for j in range(right, left-1, -1):
                    if idx < len(arr):
                        grid[bottom][j] = arr[idx]
                        idx += 1
                bottom -= 1
            if left <= right:
                for i in range(bottom, top-1, -1):
                    if idx < len(arr):
                        grid[i][left] = arr[idx]
                        idx += 1
                left += 1
    elif order == 'zigzag':
        idx = 0
        for i in range(h):
            if i % 2 == 0:
                for j in range(w):
                    grid[i][j] = arr[idx]
                    idx += 1
            else:
                for j in range(w-1, -1, -1):
                    grid[i][j] = arr[idx]
                    idx += 1
    
    return grid

# ==================== SECTION 16: BASIC PATTERNS ====================

def handle_empty_input(task_data, analysis):
    inp = np.array(task_data['train'][0]['input'])
    out = np.array(task_data['train'][0]['output'])
    
    if inp.size == 0 or inp.shape[0] == 0:
        return f"""def p(g):
 return {out.tolist()}
"""
    return None

def handle_identity(task_data, analysis):
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    if np.array_equal(inp, out):
        return """def p(g):
 return g
"""
    return None



In [ ]:
import json
import os
import zipfile
import numpy as np
from collections import defaultdict, Counter
from scipy.ndimage import label, binary_dilation, binary_erosion, convolve
from itertools import product, permutations, combinations
import math
import time
from functools import lru_cache
from typing import Dict, List, Tuple, Optional, Callable

# Configuration
DEBUG_MODE = True
DEBUG_LOG = []
PERFORMANCE_STATS = defaultdict(lambda: {'calls': 0, 'successes': 0, 'total_time': 0})
HANDLER_CACHE = {}

def debug_print(msg, level="INFO"):
    """Enhanced debug printing with levels and timing"""
    if DEBUG_MODE:
        timestamp = time.strftime("%H:%M:%S")
        formatted_msg = f"[{timestamp}] [{level}] {msg}"
        print(formatted_msg)
        DEBUG_LOG.append(formatted_msg)

def cache_key(task_data):
    """Generate a cache key for task data"""
    inp = np.array(task_data['train'][0]['input'])
    out = np.array(task_data['train'][0]['output'])
    return f"{inp.shape}_{out.shape}_{hash(inp.tobytes())}_{hash(out.tobytes())}"

def extract_function_body(solution_code):
    """Extract the function body from a solution string"""
    lines = solution_code.strip().split('\n')
    if not lines or not lines[0].startswith('def p(g):'):
        return ""
    body_lines = []
    for line in lines[1:]:
        if line.startswith(' '):
            body_lines.append(line[1:])
    return '\n'.join(body_lines)

def indent_code(code, spaces):
    """Indent code by specified spaces"""
    return '\n'.join(' ' * spaces + line for line in code.split('\n'))

def timed_handler(func):
    """Decorator to time handler execution"""
    def wrapper(task_data, analysis):
        start_time = time.time()
        result = None
        try:
            result = func(task_data, analysis)
            success = result is not None
        except Exception as e:
            success = False
            debug_print(f"Handler {func.__name__} failed: {str(e)}", "ERROR")
        
        elapsed = time.time() - start_time
        PERFORMANCE_STATS[func.__name__]['calls'] += 1
        PERFORMANCE_STATS[func.__name__]['total_time'] += elapsed
        if success:
            PERFORMANCE_STATS[func.__name__]['successes'] += 1
        
        return result
    return wrapper

def analyze_task(task_data):
    """Analyze task to extract patterns and features"""
    all_examples = task_data['train'] + task_data['test']
    analysis = {
        'color_changes': defaultdict(set),
        'shape_changes': set(),
        'symmetry': None,
        'arithmetic': None,
        'pattern_types': set(),
        'color_count': set(),
        'has_objects': False,
        'has_lines': False,
        'has_repeating_patterns': False,
    }
    
    for example in all_examples:
        in_grid = np.array(example['input'])
        out_grid = np.array(example['output'])
        
        analysis['shape_changes'].add((in_grid.shape, out_grid.shape))
        analysis['color_count'].add(len(np.unique(in_grid)))
        
        if len(np.unique(in_grid)) > 2:
            analysis['has_objects'] = True
        
        for i in range(in_grid.shape[0]):
            if len(np.unique(in_grid[i,:])) == 1 and in_grid[i,0] != 0:
                analysis['has_lines'] = True
        
        for (i,j), val in np.ndenumerate(in_grid):
            if i < out_grid.shape[0] and j < out_grid.shape[1]:
                if in_grid[i,j] != out_grid[i,j]:
                    analysis['color_changes'][(i,j)].add((int(in_grid[i,j]), int(out_grid[i,j])))
    
    return analysis

def verify_solution(solution_code, task_data):
    """Verify solution against all training examples"""
    try:
        local_namespace = {}
        exec(solution_code, globals(), local_namespace)
        
        if 'p' not in local_namespace:
            return False
        
        p = local_namespace['p']
        
        for idx, example in enumerate(task_data['train']):
            input_grid = example['input']
            expected = example['output']
            try:
                actual = p(input_grid)
                if actual != expected:
                    return False
            except Exception as e:
                return False
        return True
    except Exception as e:
        return False

# Basic transformation handlers
@timed_handler
def handle_identity(task_data, analysis):
    """Check if output equals input"""
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    if np.array_equal(inp, out):
        return """def p(g):
 return g
"""
    return None

@timed_handler
def handle_rotate_90(task_data, analysis):
    """Rotate 90 degrees clockwise"""
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    if np.array_equal(np.rot90(inp, -1), out):
        return """def p(g):
 return[list(r)for r in zip(*g[::-1])]
"""
    return None

@timed_handler
def handle_rotate_180(task_data, analysis):
    """Rotate 180 degrees"""
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    if np.array_equal(np.rot90(inp, 2), out):
        return """def p(g):
 return[r[::-1]for r in g[::-1]]
"""
    return None

@timed_handler
def handle_rotate_270(task_data, analysis):
    """Rotate 270 degrees clockwise"""
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    if np.array_equal(np.rot90(inp, 1), out):
        return """def p(g):
 return[list(r)for r in zip(*g)][::-1]
"""
    return None

@timed_handler
def handle_flip_horizontal(task_data, analysis):
    """Flip horizontally"""
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    if np.array_equal(np.fliplr(inp), out):
        return """def p(g):
 return[r[::-1]for r in g]
"""
    return None

@timed_handler
def handle_flip_vertical(task_data, analysis):
    """Flip vertically"""
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    if np.array_equal(np.flipud(inp), out):
        return """def p(g):
 return g[::-1]
"""
    return None

@timed_handler
def handle_transpose(task_data, analysis):
    """Transpose matrix"""
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    if np.array_equal(inp.T, out):
        return """def p(g):
 return[list(r)for r in zip(*g)]
"""
    return None

@timed_handler
def handle_color_mapping(task_data, analysis):
    """Map colors based on consistent changes"""
    color_map = {}
    for pos, changes in analysis['color_changes'].items():
        if len(changes) == 1:
            src, dest = next(iter(changes))
            if src not in color_map:
                color_map[src] = dest
            elif color_map[src] != dest:
                return None
    
    if not color_map:
        return None
    
    cases = ','.join([f'{int(s)}:{int(d)}' for s,d in color_map.items()])
    return f"""def p(g):
 m={{{cases}}}
 return[[m.get(x,x)for x in r]for r in g]
"""

@timed_handler
def handle_upscale(task_data, analysis):
    """Scale up the grid"""
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    for h_scale in range(1, 4):
        for w_scale in range(1, 4):
            if (inp.shape[0]*h_scale == out.shape[0] and 
                inp.shape[1]*w_scale == out.shape[1]):
                test = np.repeat(np.repeat(inp, h_scale, axis=0), w_scale, axis=1)
                if np.array_equal(test, out):
                    return f"""def p(g):
 return[[x for x in row for _ in range({w_scale})]
        for row in g for _ in range({h_scale})]
"""
    return None

@timed_handler
def handle_downscale(task_data, analysis):
    """Scale down the grid"""
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    for h_scale in range(2, 5):
        for w_scale in range(2, 5):
            if (inp.shape[0]//h_scale == out.shape[0] and 
                inp.shape[1]//w_scale == out.shape[1]):
                test = inp[::h_scale, ::w_scale]
                if np.array_equal(test, out):
                    return f"""def p(g):
 return[row[::{w_scale}]for row in g[::{h_scale}]]
"""
    return None

@timed_handler
def handle_tile_pattern(task_data, analysis):
    """Tile a pattern to create output"""
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    ih, iw = inp.shape
    oh, ow = out.shape
    
    for th in range(1, min(ih+1, 5)):
        for tw in range(1, min(iw+1, 5)):
            if oh % th == 0 and ow % tw == 0:
                tile = inp[:th, :tw]
                tiled = np.tile(tile, (oh//th, ow//tw))
                
                if np.array_equal(tiled, out):
                    return f"""def p(g):
 tile=[r[:{tw}]for r in g[:{th}]]
 h,w={oh},{ow}
 res=[]
 for i in range(h):
  row=[]
  for j in range(w):
   row.append(tile[i%{th}][j%{tw}])
  res.append(row)
 return res
"""
    return None

@timed_handler
def handle_crop_to_content(task_data, analysis):
    """Crop grid to non-zero content"""
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    rows, cols = np.where(inp > 0)
    if len(rows) == 0:
        return None
        
    min_row, max_row = rows.min(), rows.max()
    min_col, max_col = cols.min(), cols.max()
    
    cropped = inp[min_row:max_row+1, min_col:max_col+1]
    
    if np.array_equal(cropped, out):
        return """def p(g):
 rows,cols=[],[]
 for i in range(len(g)):
  for j in range(len(g[0])):
   if g[i][j]>0:
    rows.append(i)
    cols.append(j)
 if not rows:return g
 r0,r1=min(rows),max(rows)+1
 c0,c1=min(cols),max(cols)+1
 return[r[c0:c1]for r in g[r0:r1]]
"""
    return None

# Enhanced pattern detection
class PatternLearner:
    """Learn which handlers work for which patterns"""
    def __init__(self):
        self.handler_success_patterns = defaultdict(lambda: defaultdict(int))
        self.pattern_features = {}
        self.global_successes = defaultdict(int)
        
    def extract_features(self, task_data):
        """Extract features from task"""
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        features = {
            'shape_change': inp.shape != out.shape,
            'size_ratio': (out.size / inp.size) if inp.size > 0 else 1,
            'color_count_in': len(np.unique(inp)),
            'color_count_out': len(np.unique(out)),
            'has_symmetry': self.check_symmetry(inp),
            'has_pattern': self.check_pattern(inp),
            'density': np.sum(inp != 0) / inp.size if inp.size > 0 else 0,
            'shape_aspect': inp.shape[0] / inp.shape[1] if inp.shape[1] > 0 else 1,
        }
        
        return features
    
    def check_symmetry(self, grid):
        """Check if grid has symmetry"""
        return (np.array_equal(grid, np.fliplr(grid)) or 
                np.array_equal(grid, np.flipud(grid)) or
                (grid.shape[0] == grid.shape[1] and np.array_equal(grid, grid.T)))
    
    def check_pattern(self, grid):
        """Check if grid has repeating pattern"""
        h, w = grid.shape
        for ph in range(1, min(h//2 + 1, 5)):
            for pw in range(1, min(w//2 + 1, 5)):
                if h % ph == 0 and w % pw == 0:
                    pattern = grid[:ph, :pw]
                    tiled = np.tile(pattern, (h//ph, w//pw))
                    if np.array_equal(grid, tiled):
                        return True
        return False
    
    def update_success(self, handler_name, features, success):
        """Update handler success for given features"""
        feature_key = tuple(sorted(features.items()))
        self.handler_success_patterns[handler_name][feature_key] += (1 if success else -0.1)
        if success:
            self.global_successes[handler_name] += 1
    
    def get_handler_priority(self, features, handlers):
        """Get handlers prioritized by past success"""
        feature_key = tuple(sorted(features.items()))
        
        scores = []
        for handler in handlers:
            # Combine feature-specific score with global success rate
            feature_score = self.handler_success_patterns[handler.__name__][feature_key]
            global_score = self.global_successes[handler.__name__] * 0.1
            total_score = feature_score + global_score + np.random.normal(0, 0.05)
            scores.append((total_score, handler))
        
        scores.sort(reverse=True, key=lambda x: x[0])
        return [handler for _, handler in scores]

def get_all_pattern_handlers():
    """Get all available pattern handlers in priority order"""
    # Import handlers from the main file
    from paste import (
        handle_1d_raster_transform, handle_1d_pattern_repeat, handle_1d_sort_colors,
        handle_1d_run_length_decode, handle_1d_reverse_segments, handle_1d_shuffle_pattern,
        handle_1d_wave_transform, handle_1d_compression, handle_1d_periodic_transform,
        handle_1d_fibonacci_transform, handle_1d_prime_positions, handle_1d_alternating_ops,
        handle_diagonal_linearization, handle_hilbert_curve, handle_morton_order,
        handle_block_linearization, handle_snake_pattern, handle_radial_linearization,
        handle_odd_even_rows, handle_odd_even_cols, handle_checkerboard_transform,
        handle_parity_coloring, handle_alternating_blocks, handle_modular_arithmetic,
        handle_xor_patterns, handle_binary_operations, handle_bit_manipulation,
        handle_arithmetic_sequences, handle_geometric_sequences, handle_template_matching,
        handle_convolution_patterns, handle_morphological_ops, handle_edge_detection_advanced,
        handle_corner_detection, handle_pattern_completion_advanced, handle_graph_coloring,
        handle_tree_traversal, handle_component_labeling, handle_color_gradients,
        handle_color_mixing, handle_palette_reduction, handle_color_quantization,
        handle_color_spreading, handle_color_waves, handle_color_count,
        handle_color_frequency_sort, handle_replace_most_common, handle_color_propagation,
        handle_flood_fill, handle_repeat_pattern, handle_mirror_pattern,
        handle_extract_pattern, handle_apply_mask, handle_pattern_substitution,
        handle_count_objects, handle_largest_object, handle_move_objects,
        handle_gravity, handle_connect_same_color, handle_outline_objects,
        handle_fill_objects, handle_object_intersection, handle_draw_lines,
        handle_detect_lines, handle_extend_lines, handle_line_intersection,
        handle_perpendicular_lines, handle_make_symmetric, handle_complete_pattern,
        handle_find_symmetry_axis, handle_rotational_symmetry, handle_conway_step,
        handle_cellular_automaton, handle_voronoi_regions, handle_fractal_generation,
        handle_maze_solve, handle_path_finding, handle_multi_step_transform,
        handle_conditional_transform, handle_recursive_transform, handle_split_grid,
        handle_merge_grids, handle_overlay_grids, handle_subtract_grids,
        handle_empty_input, handle_rotate_45, handle_shear_transform,
        handle_pad_to_square, handle_zoom_center, handle_aspect_ratio_change
    )
    
    # Basic handlers implemented above
    basic_handlers = [
        handle_identity,
        handle_rotate_90,
        handle_rotate_180,
        handle_rotate_270,
        handle_flip_horizontal,
        handle_flip_vertical,
        handle_transpose,
        handle_color_mapping,
        handle_upscale,
        handle_downscale,
        handle_tile_pattern,
        handle_crop_to_content,
    ]
    
    # Import all handlers from paste file
    imported_handlers = [
        handle_1d_raster_transform, handle_1d_pattern_repeat, handle_1d_sort_colors,
        handle_1d_run_length_decode, handle_1d_reverse_segments, handle_1d_shuffle_pattern,
        handle_1d_wave_transform, handle_1d_compression, handle_1d_periodic_transform,
        handle_1d_fibonacci_transform, handle_1d_prime_positions, handle_1d_alternating_ops,
        handle_diagonal_linearization, handle_hilbert_curve, handle_morton_order,
        handle_block_linearization, handle_snake_pattern, handle_radial_linearization,
        handle_odd_even_rows, handle_odd_even_cols, handle_checkerboard_transform,
        handle_parity_coloring, handle_alternating_blocks, handle_modular_arithmetic,
        handle_xor_patterns, handle_binary_operations, handle_bit_manipulation,
        handle_arithmetic_sequences, handle_geometric_sequences, handle_template_matching,
        handle_convolution_patterns, handle_morphological_ops, handle_edge_detection_advanced,
        handle_corner_detection, handle_pattern_completion_advanced, handle_graph_coloring,
        handle_tree_traversal, handle_component_labeling, handle_color_gradients,
        handle_color_mixing, handle_palette_reduction, handle_color_quantization,
        handle_color_spreading, handle_color_waves, handle_color_count,
        handle_color_frequency_sort, handle_replace_most_common, handle_color_propagation,
        handle_flood_fill, handle_repeat_pattern, handle_mirror_pattern,
        handle_extract_pattern, handle_apply_mask, handle_pattern_substitution,
        handle_count_objects, handle_largest_object, handle_move_objects,
        handle_gravity, handle_connect_same_color, handle_outline_objects,
        handle_fill_objects, handle_object_intersection, handle_draw_lines,
        handle_detect_lines, handle_extend_lines, handle_line_intersection,
        handle_perpendicular_lines, handle_make_symmetric, handle_complete_pattern,
        handle_find_symmetry_axis, handle_rotational_symmetry, handle_conway_step,
        handle_cellular_automaton, handle_voronoi_regions, handle_fractal_generation,
        handle_maze_solve, handle_path_finding, handle_multi_step_transform,
        handle_conditional_transform, handle_recursive_transform, handle_split_grid,
        handle_merge_grids, handle_overlay_grids, handle_subtract_grids,
        handle_empty_input, handle_rotate_45, handle_shear_transform,
        handle_pad_to_square, handle_zoom_center, handle_aspect_ratio_change
    ]
    
    # Combine all handlers
    return basic_handlers + imported_handlers

def analyze_task_advanced(task_data):
    """Advanced task analysis with pattern detection"""
    analysis = analyze_task(task_data)
    
    inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
    analysis['transform_type'] = set()
    
    # Shape transformations
    if inp.shape != out.shape:
        if out.shape[0] > inp.shape[0] or out.shape[1] > inp.shape[1]:
            analysis['transform_type'].add('expansion')
        else:
            analysis['transform_type'].add('reduction')
    
    # Rotation detection
    if inp.shape == out.shape:
        for k in range(1, 4):
            if np.array_equal(np.rot90(inp, k), out):
                analysis['transform_type'].add(f'rotation_{k*90}')
        
        if np.array_equal(np.fliplr(inp), out):
            analysis['transform_type'].add('mirror_horizontal')
        if np.array_equal(np.flipud(inp), out):
            analysis['transform_type'].add('mirror_vertical')
    
    # Pattern detection
    if out.shape[0] % inp.shape[0] == 0 and out.shape[1] % inp.shape[1] == 0:
        analysis['transform_type'].add('tiling')
    
    # Color mapping detection
    unique_in = set(inp.flatten())
    unique_out = set(out.flatten())
    if len(unique_in) == len(unique_out) and unique_in != unique_out:
        analysis['transform_type'].add('color_mapping')
    
    return analysis

def try_handler_combinations(task_data, handlers, max_depth=2):
    """Try combining multiple handlers"""
    inp = np.array(task_data['train'][0]['input'])
    out = np.array(task_data['train'][0]['output'])
    
    # Try single handlers first
    for handler in handlers[:30]:
        try:
            solution = handler(task_data, {})
            if solution and verify_solution(solution, task_data):
                return solution
        except:
            pass
    
    # Try two-step combinations
    if max_depth >= 2:
        for h1 in handlers[:20]:
            try:
                sol1 = h1(task_data, {})
                if sol1:
                    # Apply first transformation
                    local_ns = {}
                    exec(sol1, globals(), local_ns)
                    if 'p' in local_ns:
                        intermediate = local_ns['p'](inp.tolist())
                        
                        # Create new task with intermediate result
                        new_task = {
                            'train': [{'input': intermediate, 'output': out.tolist()}],
                            'test': []
                        }
                        
                        # Try second transformation
                        for h2 in handlers[:20]:
                            try:
                                sol2 = h2(new_task, {})
                                if sol2:
                                    combined = create_combination(sol1, sol2, h1.__name__, h2.__name__)
                                    if verify_solution(combined, task_data):
                                        return combined
                            except:
                                pass
            except:
                pass
    
    return None

def create_combination(sol1, sol2, name1, name2):
    """Create combined solution"""
    body1 = extract_function_body(sol1)
    body2 = extract_function_body(sol2)
    
    return f"""def p(g):
 # {name1}
 def t1(g):
{indent_code(body1, 2)}
 # {name2}
 def t2(g):
{indent_code(body2, 2)}
 return t2(t1(g))
"""

def generate_solution_ultra(task_data, learner):
    """Ultra-enhanced solution generation"""
    analysis = analyze_task_advanced(task_data)
    
    # Get all handlers
    all_handlers = get_all_pattern_handlers()
    
    # Check cache
    key = cache_key(task_data)
    if key in HANDLER_CACHE:
        return HANDLER_CACHE[key]
    
    # Extract features for learning
    features = learner.extract_features(task_data)
    
    # Get prioritized handlers
    handlers = learner.get_handler_priority(features, all_handlers)
    
    # Phase 1: Try single handlers with priority
    debug_print("Phase 1: Trying prioritized single handlers", "INFO")
    for i, handler in enumerate(handlers):
        if i > 50 and PERFORMANCE_STATS[handler.__name__]['successes'] == 0:
            continue  # Skip handlers that have never worked
        
        try:
            solution = handler(task_data, analysis)
            if solution and verify_solution(solution, task_data):
                debug_print(f"✓ Found solution with {handler.__name__}", "SUCCESS")
                learner.update_success(handler.__name__, features, True)
                HANDLER_CACHE[key] = solution
                return solution
            elif solution:
                learner.update_success(handler.__name__, features, False)
        except:
            pass
    
    # Phase 2: Try combinations
    debug_print("Phase 2: Trying handler combinations", "INFO")
    combination = try_handler_combinations(task_data, handlers, max_depth=2)
    if combination:
        HANDLER_CACHE[key] = combination
        return combination
    
    # Fallback
    fallback = """def p(g):
 return g
"""
    HANDLER_CACHE[key] = fallback
    return fallback

def create_arc_solutions_ultra(input_dir=".", output_dir="submission"):
    """Main execution with learning and optimization"""
    solutions = {}
    os.makedirs(output_dir, exist_ok=True)
    
    # Initialize learner
    learner = PatternLearner()
    
    solved_count = 0
    total_count = 0
    start_time = time.time()
    
    # Process all tasks
    for task_num in range(1, 401):
        task_id = f"{task_num:03d}"
        task_file = os.path.join(input_dir, f"task{task_id}.json")
        
        try:
            with open(task_file) as f:
                task_data = json.load(f)
            
            # Generate solution
            solution = generate_solution_ultra(task_data, learner)
            solutions[task_id] = solution
            
            # Verify and update statistics
            if verify_solution(solution, task_data):
                print(f"✅ Task {task_id} solved")
                solved_count += 1
            else:
                print(f"❌ Task {task_id} fallback")
            
            total_count += 1
            
        except Exception as e:
            print(f"Error task {task_id}: {str(e)}")
            solutions[task_id] = """def p(g):
 return g
"""
            total_count += 1
    
    # Save solutions
    for task_id, code in solutions.items():
        output_file = os.path.join(output_dir, f"task{task_id}.py")
        with open(output_file, "w") as f:
            f.write(code)
    
    # Create submission
    with zipfile.ZipFile("submission.zip", "w") as zipf:
        for task_id in solutions:
            file_path = os.path.join(output_dir, f"task{task_id}.py")
            zipf.write(file_path, f"task{task_id}.py")
    
    # Print results
    total_time = time.time() - start_time
    print(f"\n{'='*60}")
    print(f"FINAL RESULTS")
    print(f"{'='*60}")
    print(f"✅ Solved: {solved_count}/{total_count} ({solved_count/total_count*100:.1f}%)")
    print(f"⏱️  Time: {total_time:.1f}s ({total_time/total_count:.2f}s per task)")
    
    # Performance statistics
    if PERFORMANCE_STATS:
        print(f"\nTop handlers by success:")
        sorted_handlers = sorted(
            [(k, v) for k, v in PERFORMANCE_STATS.items() if v['successes'] > 0],
            key=lambda x: x[1]['successes'],
            reverse=True
        )[:10]
        
        for name, stats in sorted_handlers:
            success_rate = stats['successes'] / stats['calls'] * 100 if stats['calls'] > 0 else 0
            print(f"  {name}: {stats['successes']} ({success_rate:.1f}%)")
    
    print(f"\n📦 Created: submission.zip")

if __name__ == "__main__":
    create_arc_solutions_ultra()

In [ ]:
# import time
# from functools import lru_cache
# from typing import Dict, List, Tuple, Optional, Callable
# import numpy as np
# from collections import defaultdict, Counter
# import json
# import os
# import zipfile

# DEBUG_MODE = True
# DEBUG_LOG = []
# PERFORMANCE_STATS = defaultdict(lambda: {'calls': 0, 'successes': 0, 'total_time': 0})
# HANDLER_CACHE = {}

# def debug_print(msg, level="INFO"):
#     if DEBUG_MODE:
#         timestamp = time.strftime("%H:%M:%S")
#         formatted_msg = f"[{timestamp}] [{level}] {msg}"
#         print(formatted_msg)
#         DEBUG_LOG.append(formatted_msg)

# def cache_key(task_data):
#     inp = np.array(task_data['train'][0]['input'])
#     out = np.array(task_data['train'][0]['output'])
#     return f"{inp.shape}_{out.shape}_{hash(inp.tobytes())}_{hash(out.tobytes())}"

# def extract_function_body(solution_code):
#     lines = solution_code.strip().split('\n')
#     if not lines or not lines[0].startswith('def p(g):'):
#         return ""
#     body_lines = []
#     for line in lines[1:]:
#         if line.startswith(' '):
#             body_lines.append(line[1:])
#     return '\n'.join(body_lines)

# def indent_code(code, spaces):
#     return '\n'.join(' ' * spaces + line for line in code.split('\n'))

# def timed_handler(func):
#     def wrapper(task_data, analysis):
#         start_time = time.time()
#         result = None
#         try:
#             result = func(task_data, analysis)
#             success = result is not None
#         except Exception as e:
#             success = False
#             debug_print(f"Handler {func.__name__} failed: {str(e)}", "ERROR")
        
#         elapsed = time.time() - start_time
#         PERFORMANCE_STATS[func.__name__]['calls'] += 1
#         PERFORMANCE_STATS[func.__name__]['total_time'] += elapsed
#         if success:
#             PERFORMANCE_STATS[func.__name__]['successes'] += 1
        
#         return result
#     return wrapper

# def analyze_task(task_data):
#     all_examples = task_data['train'] + task_data['test']
#     analysis = {
#         'color_changes': defaultdict(set),
#         'shape_changes': set(),
#         'symmetry': None,
#         'arithmetic': None,
#         'pattern_types': set(),
#         'color_count': set(),
#         'has_objects': False,
#         'has_lines': False,
#         'has_repeating_patterns': False,
#     }
    
#     for example in all_examples:
#         in_grid = np.array(example['input'])
#         out_grid = np.array(example['output'])
        
#         analysis['shape_changes'].add((in_grid.shape, out_grid.shape))
#         analysis['color_count'].add(len(np.unique(in_grid)))
        
#         if len(np.unique(in_grid)) > 2:
#             analysis['has_objects'] = True
        
#         for i in range(in_grid.shape[0]):
#             if len(np.unique(in_grid[i,:])) == 1 and in_grid[i,0] != 0:
#                 analysis['has_lines'] = True
        
#         for (i,j), val in np.ndenumerate(in_grid):
#             if i < out_grid.shape[0] and j < out_grid.shape[1]:
#                 if in_grid[i,j] != out_grid[i,j]:
#                     analysis['color_changes'][(i,j)].add((int(in_grid[i,j]), int(out_grid[i,j])))
    
#     return analysis

# def verify_solution(solution_code, task_data):
#     try:
#         local_namespace = {}
#         exec(solution_code, globals(), local_namespace)
        
#         if 'p' not in local_namespace:
#             return False
        
#         p = local_namespace['p']
        
#         for idx, example in enumerate(task_data['train']):
#             input_grid = example['input']
#             expected = example['output']
#             try:
#                 actual = p(input_grid)
#                 if actual != expected:
#                     debug_print(f"  Failed on training example {idx}", "WARNING")
#                     return False
#             except Exception as e:
#                 debug_print(f"  Exception on training example {idx}: {str(e)}", "ERROR")
#                 return False
#         return True
#     except Exception as e:
#         debug_print(f"  Verification failed: {str(e)}", "ERROR")
#         return False

# def get_existing_handlers():
#     return []

# def handle_rotate_90(task_data, analysis):
#     return None

# def handle_rotate_180(task_data, analysis):
#     return None

# def handle_rotate_270(task_data, analysis):
#     return None

# def handle_flip_horizontal(task_data, analysis):
#     return None

# def handle_flip_vertical(task_data, analysis):
#     return None

# def handle_tile_pattern(task_data, analysis):
#     return None

# def handle_repeat_pattern(task_data, analysis):
#     return None

# def handle_upscale(task_data, analysis):
#     return None

# def handle_pad_to_square(task_data, analysis):
#     return None

# def handle_border_operations(task_data, analysis):
#     return None

# def handle_downscale(task_data, analysis):
#     return None

# def handle_crop_to_content(task_data, analysis):
#     return None

# def handle_extract_pattern(task_data, analysis):
#     return None

# def handle_color_mapping(task_data, analysis):
#     return None

# def handle_layered_transform(task_data, analysis):
#     return None

# def handle_color_cycle(task_data, analysis):
#     return None

# def analyze_task_advanced(task_data):
#     analysis = analyze_task(task_data)
    
#     inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
#     analysis['transform_type'] = set()
    
#     if inp.shape != out.shape:
#         if out.shape[0] > inp.shape[0] or out.shape[1] > inp.shape[1]:
#             analysis['transform_type'].add('expansion')
#         else:
#             analysis['transform_type'].add('reduction')
            
#     if inp.shape == out.shape:
#         for k in range(1, 4):
#             if np.array_equal(np.rot90(inp, k), out):
#                 analysis['transform_type'].add(f'rotation_{k*90}')
                
#         if np.array_equal(np.fliplr(inp), out):
#             analysis['transform_type'].add('mirror_horizontal')
#         if np.array_equal(np.flipud(inp), out):
#             analysis['transform_type'].add('mirror_vertical')
            
#     if out.shape[0] % inp.shape[0] == 0 and out.shape[1] % inp.shape[1] == 0:
#         analysis['transform_type'].add('tiling')
        
#     unique_in = set(inp.flatten())
#     unique_out = set(out.flatten())
#     if len(unique_in) == len(unique_out) and unique_in != unique_out:
#         analysis['transform_type'].add('color_mapping')
        
#     analysis['recommended_handlers'] = recommend_handlers(analysis)
    
#     return analysis

# @timed_handler
# def handle_recursive_pattern_completion(task_data, analysis):
#     debug_print(f"Trying handle_recursive_pattern_completion")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             def apply_rules(grid):
#                 changed = False
#                 new_grid = np.copy(grid)
                
#                 for i in range(1, h-1):
#                     for j in range(1, w-1):
#                         if grid[i,j] == 0:
#                             neighbors = [grid[i-1,j], grid[i+1,j], grid[i,j-1], grid[i,j+1]]
#                             if len(set(neighbors)) == 1 and neighbors[0] != 0:
#                                 new_grid[i,j] = neighbors[0]
#                                 changed = True
                                
#                 for i in range(h-2):
#                     for j in range(w-2):
#                         pattern = grid[i:i+3, j:j+3]
#                         if np.sum(pattern == 0) == 1:
#                             non_zero = pattern[pattern != 0]
#                             if len(non_zero) > 0:
#                                 most_common = Counter(non_zero).most_common(1)[0][0]
#                                 zero_pos = np.where(pattern == 0)
#                                 new_grid[i+zero_pos[0][0], j+zero_pos[1][0]] = most_common
#                                 changed = True
                                
#                 return new_grid, changed
            
#             current = np.copy(inp)
#             max_iterations = 10
#             for _ in range(max_iterations):
#                 current, changed = apply_rules(current)
#                 if not changed:
#                     break
                    
#             if np.array_equal(current, out):
#                 debug_print(f"  ✓ Found recursive pattern completion")
#                 return """def p(g):
#  import numpy as np
#  from collections import Counter
#  h,w=len(g),len(g[0])
#  def ar(grid):
#   ch=False
#   ng=np.copy(grid)
#   for i in range(1,h-1):
#    for j in range(1,w-1):
#     if grid[i,j]==0:
#      n=[grid[i-1,j],grid[i+1,j],grid[i,j-1],grid[i,j+1]]
#      if len(set(n))==1 and n[0]!=0:
#       ng[i,j]=n[0]
#       ch=True
#   for i in range(h-2):
#    for j in range(w-2):
#     p=grid[i:i+3,j:j+3]
#     if np.sum(p==0)==1:
#      nz=p[p!=0]
#      if len(nz)>0:
#       mc=Counter(nz).most_common(1)[0][0]
#       zp=np.where(p==0)
#       ng[i+zp[0][0],j+zp[1][0]]=mc
#       ch=True
#   return ng,ch
#  c=np.array(g)
#  for _ in range(10):
#   c,ch=ar(c)
#   if not ch:break
#  return c.tolist()
# """
#         debug_print(f"  ✗ Not a recursive pattern completion")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_recursive_pattern_completion: {str(e)}")
#     return None

# @timed_handler
# def handle_recursive_growth(task_data, analysis):
#     debug_print(f"Trying handle_recursive_growth")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             def grow_step(grid, growth_rule):
#                 new_grid = np.copy(grid)
#                 for i in range(h):
#                     for j in range(w):
#                         if grid[i,j] != 0:
#                             for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
#                                 ni, nj = i + di, j + dj
#                                 if 0 <= ni < h and 0 <= nj < w and grid[ni,nj] == 0:
#                                     new_grid[ni,nj] = growth_rule(grid[i,j], i, j, ni, nj)
#                 return new_grid
            
#             for rule_type in ['same', 'increment', 'position_based']:
#                 if rule_type == 'same':
#                     growth_rule = lambda val, i, j, ni, nj: val
#                 elif rule_type == 'increment':
#                     growth_rule = lambda val, i, j, ni, nj: (val % 9) + 1
#                 else:
#                     growth_rule = lambda val, i, j, ni, nj: ((ni + nj) % 9) + 1
                
#                 current = np.copy(inp)
#                 for step in range(5):
#                     current = grow_step(current, growth_rule)
#                     if np.array_equal(current, out):
#                         debug_print(f"  ✓ Found recursive growth with {rule_type} rule")
#                         if rule_type == 'same':
#                             return """def p(g):
#  import numpy as np
#  h,w=len(g),len(g[0])
#  def gs(grid):
#   ng=np.copy(grid)
#   for i in range(h):
#    for j in range(w):
#     if grid[i,j]!=0:
#      for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
#       ni,nj=i+di,j+dj
#       if 0<=ni<h and 0<=nj<w and grid[ni,nj]==0:
#        ng[ni,nj]=grid[i,j]
#   return ng
#  c=np.array(g)
#  for _ in range(5):
#   c=gs(c)
#  return c.tolist()
# """
#         debug_print(f"  ✗ Not a recursive growth pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_recursive_growth: {str(e)}")
#     return None

# @timed_handler
# def handle_contextual_transformation(task_data, analysis):
#     debug_print(f"Trying handle_contextual_transformation")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.zeros_like(inp)
            
#             for i in range(h):
#                 for j in range(w):
#                     context = []
#                     for di in range(-1, 2):
#                         for dj in range(-1, 2):
#                             ni, nj = i + di, j + dj
#                             if 0 <= ni < h and 0 <= nj < w:
#                                 context.append(inp[ni,nj])
#                             else:
#                                 context.append(0)
                    
#                     context_hash = sum(c * (10 ** idx) for idx, c in enumerate(context)) % 10
#                     test[i,j] = context_hash
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found contextual transformation")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[[0]*w for _ in range(h)]
#  for i in range(h):
#   for j in range(w):
#    ctx=[]
#    for di in range(-1,2):
#     for dj in range(-1,2):
#      ni,nj=i+di,j+dj
#      if 0<=ni<h and 0<=nj<w:
#       ctx.append(g[ni][nj])
#      else:
#       ctx.append(0)
#    ch=sum(c*(10**idx)for idx,c in enumerate(ctx))%10
#    res[i][j]=ch
#  return res
# """
#         debug_print(f"  ✗ Not a contextual transformation")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_contextual_transformation: {str(e)}")
#     return None

# @timed_handler
# def handle_probabilistic_fill(task_data, analysis):
#     debug_print(f"Trying handle_probabilistic_fill")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             color_probs = defaultdict(lambda: defaultdict(int))
            
#             for i in range(h):
#                 for j in range(w):
#                     if inp[i,j] != 0:
#                         for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
#                             ni, nj = i + di, j + dj
#                             if 0 <= ni < h and 0 <= nj < w:
#                                 color_probs[inp[i,j]][inp[ni,nj]] += 1
            
#             test = np.copy(inp)
#             for i in range(h):
#                 for j in range(w):
#                     if inp[i,j] == 0:
#                         neighbor_colors = []
#                         for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
#                             ni, nj = i + di, j + dj
#                             if 0 <= ni < h and 0 <= nj < w and inp[ni,nj] != 0:
#                                 neighbor_colors.append(inp[ni,nj])
                        
#                         if neighbor_colors:
#                             color_scores = defaultdict(int)
#                             for nc in neighbor_colors:
#                                 for target, count in color_probs[nc].items():
#                                     color_scores[target] += count
                            
#                             if color_scores:
#                                 best_color = max(color_scores.items(), key=lambda x: x[1])[0]
#                                 test[i,j] = best_color
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found probabilistic fill")
#                 return """def p(g):
#  from collections import defaultdict
#  h,w=len(g),len(g[0])
#  res=[r[:]for r in g]
#  for i in range(h):
#   for j in range(w):
#    if g[i][j]==0:
#     nc=[]
#     for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
#      ni,nj=i+di,j+dj
#      if 0<=ni<h and 0<=nj<w and g[ni][nj]!=0:
#       nc.append(g[ni][nj])
#     if nc:
#      res[i][j]=max(set(nc),key=nc.count)
#  return res
# """
#         debug_print(f"  ✗ Not a probabilistic fill")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_probabilistic_fill: {str(e)}")
#     return None

# def try_adaptive_combinations(task_data, pattern_handlers, analysis):
#     debug_print("=== Trying adaptive pattern combinations ===", "INFO")
    
#     recommended = analysis.get('recommended_handlers', [])
#     if recommended:
#         debug_print(f"  Using {len(recommended)} recommended handlers", "INFO")
        
#         for handler in recommended:
#             solution = handler(task_data, analysis)
#             if solution and verify_solution(solution, task_data):
#                 debug_print(f"✓ Recommended handler {handler.__name__} solved it!", "SUCCESS")
#                 return solution
    
#     inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
#     intermediate_candidates = detect_intermediate_states(inp, out)
    
#     if intermediate_candidates:
#         debug_print(f"  Found {len(intermediate_candidates)} possible intermediate states", "INFO")
        
#         for intermediate in intermediate_candidates:
#             step1_handlers = find_transformation(inp, intermediate, pattern_handlers[:20])
            
#             if step1_handlers:
#                 step2_handlers = find_transformation(intermediate, out, pattern_handlers[:20])
                
#                 if step2_handlers:
#                     combined = create_multi_step_combination(
#                         step1_handlers[0][1], 
#                         step2_handlers[0][1],
#                         step1_handlers[0][0].__name__,
#                         step2_handlers[0][0].__name__
#                     )
                    
#                     if verify_solution(combined, task_data):
#                         debug_print(f"✓ Found multi-step combination!", "SUCCESS")
#                         return combined
    
#     return None

# def detect_intermediate_states(inp, out):
#     candidates = []
    
#     if inp.shape == out.shape:
#         for k in range(1, 4):
#             candidates.append(np.rot90(inp, k))
            
#         candidates.append(np.fliplr(inp))
#         candidates.append(np.flipud(inp))
        
#         unique_colors = np.unique(inp)
#         if len(unique_colors) <= 5:
#             shifted = np.copy(inp)
#             for color in unique_colors:
#                 if color != 0:
#                     shifted[inp == color] = (color % 9) + 1
#             candidates.append(shifted)
    
#     return candidates

# class PatternLearner:
#     def __init__(self):
#         self.handler_success_patterns = defaultdict(lambda: defaultdict(int))
#         self.pattern_features = {}
        
#     def extract_features(self, task_data):
#         inp = np.array(task_data['train'][0]['input'])
#         out = np.array(task_data['train'][0]['output'])
        
#         features = {
#             'shape_change': inp.shape != out.shape,
#             'size_ratio': (out.size / inp.size) if inp.size > 0 else 1,
#             'color_count_in': len(np.unique(inp)),
#             'color_count_out': len(np.unique(out)),
#             'has_symmetry': self.check_symmetry(inp),
#             'has_pattern': self.check_pattern(inp),
#             'density': np.sum(inp != 0) / inp.size,
#             'shape_aspect': inp.shape[0] / inp.shape[1] if inp.shape[1] > 0 else 1,
#         }
        
#         return features
    
#     def check_symmetry(self, grid):
#         return (np.array_equal(grid, np.fliplr(grid)) or 
#                 np.array_equal(grid, np.flipud(grid)) or
#                 (grid.shape[0] == grid.shape[1] and np.array_equal(grid, grid.T)))
    
#     def check_pattern(self, grid):
#         h, w = grid.shape
#         for ph in range(1, h//2 + 1):
#             for pw in range(1, w//2 + 1):
#                 if h % ph == 0 and w % pw == 0:
#                     pattern = grid[:ph, :pw]
#                     tiled = np.tile(pattern, (h//ph, w//pw))
#                     if np.array_equal(grid, tiled):
#                         return True
#         return False
    
#     def update_success(self, handler_name, features, success):
#         feature_key = tuple(sorted(features.items()))
#         self.handler_success_patterns[handler_name][feature_key] += (1 if success else -1)
    
#     def get_handler_priority(self, features, handlers):
#         feature_key = tuple(sorted(features.items()))
        
#         scores = []
#         for handler in handlers:
#             score = self.handler_success_patterns[handler.__name__][feature_key]
#             score += np.random.normal(0, 0.1)
#             scores.append((score, handler))
        
#         scores.sort(reverse=True, key=lambda x: x[0])
        
#         return [handler for _, handler in scores]

# def generate_solution_enhanced(task_data, pattern_learner=None):
#     global DEBUG_LOG, HANDLER_CACHE
#     DEBUG_LOG = []
    
#     analysis = analyze_task_advanced(task_data)
    
#     debug_print(f"\n{'='*60}", "INFO")
#     debug_print(f"Advanced Analysis Results:", "INFO")
#     debug_print(f"  Transform types: {analysis['transform_type']}", "INFO")
#     debug_print(f"  Recommended handlers: {len(analysis.get('recommended_handlers', []))}", "INFO")
    
#     pattern_handlers = get_all_handlers()
    
#     if analysis.get('recommended_handlers'):
#         debug_print("\n=== PHASE 0: Trying recommended handlers ===", "INFO")
#         for handler in analysis['recommended_handlers']:
#             solution = handler(task_data, analysis)
#             if solution and verify_solution(solution, task_data):
#                 debug_print(f"✓ Recommended handler {handler.__name__} solved it!", "SUCCESS")
#                 return solution
    
#     debug_print("\n=== PHASE 1: Intelligent handler selection ===", "INFO")
    
#     if pattern_learner:
#         features = pattern_learner.extract_features(task_data)
#         prioritized_handlers = pattern_learner.get_handler_priority(features, pattern_handlers)
#     else:
#         prioritized_handlers = pattern_handlers
    
#     for idx, handler in enumerate(prioritized_handlers[:50]):
#         solution = handler(task_data, analysis)
        
#         if solution and verify_solution(solution, task_data):
#             debug_print(f"✓ Handler {handler.__name__} solved it!", "SUCCESS")
            
#             if pattern_learner:
#                 features = pattern_learner.extract_features(task_data)
#                 pattern_learner.update_success(handler.__name__, features, True)
                
#             return solution
    
#     debug_print("\n=== PHASE 2: Adaptive combinations ===", "INFO")
    
#     combination_solution = try_adaptive_combinations(task_data, pattern_handlers, analysis)
#     if combination_solution:
#         return combination_solution
    
#     debug_print("\n=== PHASE 3: Recursive transformation search ===", "INFO")
    
#     solution = recursive_transformation_search(
#         task_data, 
#         pattern_handlers[:30], 
#         max_depth=3,
#         analysis=analysis
#     )
#     if solution:
#         return solution
    
#     debug_print("\n=== PHASE 4: Exhaustive search ===", "INFO")
    
#     for handler in prioritized_handlers[50:]:
#         solution = handler(task_data, analysis)
#         if solution and verify_solution(solution, task_data):
#             return solution
    
#     return """def p(g):
#  return g
# """

# def recursive_transformation_search(task_data, handlers, max_depth=3, current_depth=0, analysis=None):
#     if current_depth >= max_depth:
#         return None
        
#     inp = np.array(task_data['train'][0]['input'])
#     out = np.array(task_data['train'][0]['output'])
    
#     for handler in handlers[:10]:
#         try:
#             solution = handler(task_data, analysis or {})
#             if solution:
#                 local_namespace = {}
#                 exec(solution, globals(), local_namespace)
#                 if 'p' in local_namespace:
#                     intermediate = local_namespace['p'](inp.tolist())
                    
#                     if intermediate == out.tolist():
#                         return solution
                    
#                     new_task = {
#                         'train': [{'input': intermediate, 'output': out.tolist()}],
#                         'test': []
#                     }
                    
#                     next_solution = recursive_transformation_search(
#                         new_task, 
#                         handlers, 
#                         max_depth, 
#                         current_depth + 1,
#                         analysis
#                     )
                    
#                     if next_solution:
#                         return create_recursive_combination(solution, next_solution, handler.__name__)
#         except:
#             pass
            
#     return None

# def create_recursive_combination(sol1, sol2, name1):
#     body1 = extract_function_body(sol1)
#     body2 = extract_function_body(sol2)
    
#     return f"""def p(g):
#  def t1(g):
# {indent_code(body1, 2)}
 
#  def t2(g):
# {indent_code(body2, 2)}
 
#  intermediate = t1(g)
#  return t2(intermediate)
# """

# def get_all_handlers():
#     return [
#         handle_recursive_pattern_completion,
#         handle_recursive_growth,
#         handle_contextual_transformation,
#         handle_probabilistic_fill,
#     ] + get_existing_handlers()

# def find_transformation(inp, out, handlers):
#     mock_task = {
#         'train': [{'input': inp.tolist(), 'output': out.tolist()}],
#         'test': []
#     }
    
#     matches = []
#     for handler in handlers:
#         try:
#             solution = handler(mock_task, {})
#             if solution and verify_solution(solution, mock_task):
#                 matches.append((handler, solution))
#         except:
#             pass
            
#     return matches

# def create_multi_step_combination(sol1, sol2, name1, name2):
#     body1 = extract_function_body(sol1)
#     body2 = extract_function_body(sol2)
    
#     return f"""def p(g):
#  def t1(g):
# {indent_code(body1, 2)}
 
#  def t2(g):
# {indent_code(body2, 2)}
 
#  intermediate = t1(g)
#  return t2(intermediate)
# """

# def recommend_handlers(analysis):
#     recommendations = []
    
#     handler_patterns = {
#         'rotation_90': [handle_rotate_90],
#         'rotation_180': [handle_rotate_180],
#         'rotation_270': [handle_rotate_270],
#         'mirror_horizontal': [handle_flip_horizontal],
#         'mirror_vertical': [handle_flip_vertical],
#         'tiling': [handle_tile_pattern, handle_repeat_pattern],
#         'expansion': [handle_upscale, handle_pad_to_square, handle_border_operations],
#         'reduction': [handle_downscale, handle_crop_to_content, handle_extract_pattern],
#         'color_mapping': [handle_color_mapping, handle_layered_transform, handle_color_cycle],
#     }
    
#     for pattern_type in analysis['transform_type']:
#         if pattern_type in handler_patterns:
#             recommendations.extend(handler_patterns[pattern_type])
            
#     return recommendations

# learner = PatternLearner()

# def create_arc_solutions_ultra(input_dir=".", output_dir="submission"):
#     solutions = {}
    
#     os.makedirs(output_dir, exist_ok=True)
    
#     solved_count = 0
#     total_count = 0
    
#     for task_num in range(1, 401):
#         task_id = f"{task_num:03d}"
#         task_file = os.path.join(input_dir, f"task{task_id}.json")
        
#         try:
#             with open(task_file) as f:
#                 task_data = json.load(f)
            
#             solution = generate_solution_enhanced(task_data, learner)
#             solutions[task_id] = solution
            
#             if verify_solution(solution, task_data):
#                 print(f"✅ Task {task_id} solved")
#                 solved_count += 1
                
#                 features = learner.extract_features(task_data)
#                 for line in solution.split('\n'):
#                     if '# Step 1:' in line or '✓ Found' in DEBUG_LOG:
#                         handler_name = line.split(':')[1].strip()
#                         learner.update_success(handler_name, features, True)
#                         break
#             else:
#                 print(f"❌ Task {task_id} fallback")
            
#             total_count += 1
            
#         except Exception as e:
#             print(f"Error task {task_id}: {str(e)}")
#             solutions[task_id] = """def p(g):
#  return g
# """
#             total_count += 1
    
#     for task_id, code in solutions.items():
#         output_file = os.path.join(output_dir, f"task{task_id}.py")
#         with open(output_file, "w") as f:
#             f.write(code)
    
#     with zipfile.ZipFile("submission.zip", "w") as zipf:
#         for task_id in solutions:
#             file_path = os.path.join(output_dir, f"task{task_id}.py")
#             zipf.write(file_path, f"task{task_id}.py")
    
#     print(f"\n{'='*60}")
#     print(f"FINAL RESULTS")
#     print(f"{'='*60}")
#     print(f"✅ Solved: {solved_count}/{total_count} ({solved_count/total_count*100:.1f}%)")
#     print(f"\n📦 Created: submission.zip")

# if __name__ == "__main__":
#     create_arc_solutions_ultra(input_dir='/kaggle/input/google-code-golf-2025')

In [ ]:
# # ==================== SECTION 17: CORE FUNCTIONALITY ====================

# def analyze_task_advanced(task_data):
#     """Advanced task analysis with pattern detection and handler recommendations"""
#     analysis = analyze_task(task_data)  # Keep existing analysis
    
#     # Add pattern detection
#     inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
#     # Detect transformation types
#     analysis['transform_type'] = set()
    
#     # Shape transformations
#     if inp.shape != out.shape:
#         if out.shape[0] > inp.shape[0] or out.shape[1] > inp.shape[1]:
#             analysis['transform_type'].add('expansion')
#         else:
#             analysis['transform_type'].add('reduction')
            
#     # Symmetry detection
#     if inp.shape == out.shape:
#         # Check for rotational relationship
#         for k in range(1, 4):
#             if np.array_equal(np.rot90(inp, k), out):
#                 analysis['transform_type'].add(f'rotation_{k*90}')
                
#         # Check for mirror
#         if np.array_equal(np.fliplr(inp), out):
#             analysis['transform_type'].add('mirror_horizontal')
#         if np.array_equal(np.flipud(inp), out):
#             analysis['transform_type'].add('mirror_vertical')
            
#     # Pattern repetition detection
#     if out.shape[0] % inp.shape[0] == 0 and out.shape[1] % inp.shape[1] == 0:
#         analysis['transform_type'].add('tiling')
        
#     # Color transformation detection
#     unique_in = set(inp.flatten())
#     unique_out = set(out.flatten())
#     if len(unique_in) == len(unique_out) and unique_in != unique_out:
#         analysis['transform_type'].add('color_mapping')
        
#     # Recommend handlers based on analysis
#     analysis['recommended_handlers'] = recommend_handlers(analysis)
    
#     return analysis

# @timed_handler
# def handle_recursive_pattern_completion(task_data, analysis):
#     """Recursively complete patterns until no changes occur"""
#     debug_print(f"Trying handle_recursive_pattern_completion")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             # Define recursive completion rules
#             def apply_rules(grid):
#                 changed = False
#                 new_grid = np.copy(grid)
                
#                 # Rule 1: If a cell is surrounded by same color, fill it
#                 for i in range(1, h-1):
#                     for j in range(1, w-1):
#                         if grid[i,j] == 0:
#                             neighbors = [grid[i-1,j], grid[i+1,j], grid[i,j-1], grid[i,j+1]]
#                             if len(set(neighbors)) == 1 and neighbors[0] != 0:
#                                 new_grid[i,j] = neighbors[0]
#                                 changed = True
                                
#                 # Rule 2: Complete patterns
#                 for i in range(h-2):
#                     for j in range(w-2):
#                         # Check 3x3 patterns
#                         pattern = grid[i:i+3, j:j+3]
#                         if np.sum(pattern == 0) == 1:  # One empty cell
#                             # Find most common non-zero value
#                             non_zero = pattern[pattern != 0]
#                             if len(non_zero) > 0:
#                                 most_common = Counter(non_zero).most_common(1)[0][0]
#                                 zero_pos = np.where(pattern == 0)
#                                 new_grid[i+zero_pos[0][0], j+zero_pos[1][0]] = most_common
#                                 changed = True
                                
#                 return new_grid, changed
            
#             # Apply rules recursively
#             current = np.copy(inp)
#             max_iterations = 10
#             for _ in range(max_iterations):
#                 current, changed = apply_rules(current)
#                 if not changed:
#                     break
                    
#             if np.array_equal(current, out):
#                 debug_print(f"  ✓ Found recursive pattern completion")
#                 return """def p(g):
#  import numpy as np
#  from collections import Counter
#  h,w=len(g),len(g[0])
#  def ar(grid):
#   ch=False
#   ng=np.copy(grid)
#   for i in range(1,h-1):
#    for j in range(1,w-1):
#     if grid[i,j]==0:
#      n=[grid[i-1,j],grid[i+1,j],grid[i,j-1],grid[i,j+1]]
#      if len(set(n))==1 and n[0]!=0:
#       ng[i,j]=n[0]
#       ch=True
#   for i in range(h-2):
#    for j in range(w-2):
#     p=grid[i:i+3,j:j+3]
#     if np.sum(p==0)==1:
#      nz=p[p!=0]
#      if len(nz)>0:
#       mc=Counter(nz).most_common(1)[0][0]
#       zp=np.where(p==0)
#       ng[i+zp[0][0],j+zp[1][0]]=mc
#       ch=True
#   return ng,ch
#  c=np.array(g)
#  for _ in range(10):
#   c,ch=ar(c)
#   if not ch:break
#  return c.tolist()
# """
#         debug_print(f"  ✗ Not a recursive pattern completion")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_recursive_pattern_completion: {str(e)}")
#     return None

# @timed_handler
# def handle_recursive_growth(task_data, analysis):
#     """Recursively grow patterns from seed points"""
#     debug_print(f"Trying handle_recursive_growth")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             def grow_step(grid, growth_rule):
#                 new_grid = np.copy(grid)
#                 for i in range(h):
#                     for j in range(w):
#                         if grid[i,j] != 0:
#                             # Apply growth rule to neighbors
#                             for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
#                                 ni, nj = i + di, j + dj
#                                 if 0 <= ni < h and 0 <= nj < w and grid[ni,nj] == 0:
#                                     new_grid[ni,nj] = growth_rule(grid[i,j], i, j, ni, nj)
#                 return new_grid
            
#             # Try different growth rules
#             for rule_type in ['same', 'increment', 'position_based']:
#                 if rule_type == 'same':
#                     growth_rule = lambda val, i, j, ni, nj: val
#                 elif rule_type == 'increment':
#                     growth_rule = lambda val, i, j, ni, nj: (val % 9) + 1
#                 else:
#                     growth_rule = lambda val, i, j, ni, nj: ((ni + nj) % 9) + 1
                
#                 current = np.copy(inp)
#                 for step in range(5):
#                     current = grow_step(current, growth_rule)
#                     if np.array_equal(current, out):
#                         debug_print(f"  ✓ Found recursive growth with {rule_type} rule")
#                         if rule_type == 'same':
#                             return """def p(g):
#  import numpy as np
#  h,w=len(g),len(g[0])
#  def gs(grid):
#   ng=np.copy(grid)
#   for i in range(h):
#    for j in range(w):
#     if grid[i,j]!=0:
#      for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
#       ni,nj=i+di,j+dj
#       if 0<=ni<h and 0<=nj<w and grid[ni,nj]==0:
#        ng[ni,nj]=grid[i,j]
#   return ng
#  c=np.array(g)
#  for _ in range(5):
#   c=gs(c)
#  return c.tolist()
# """
#         debug_print(f"  ✗ Not a recursive growth pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_recursive_growth: {str(e)}")
#     return None

# def analyze_task_advanced(task_data):
#     """Advanced task analysis with pattern detection and handler recommendations"""
#     analysis = analyze_task(task_data)  # Keep existing analysis
    
#     # Add pattern detection
#     inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
#     # Detect transformation types
#     analysis['transform_type'] = set()
    
#     # Shape transformations
#     if inp.shape != out.shape:
#         if out.shape[0] > inp.shape[0] or out.shape[1] > inp.shape[1]:
#             analysis['transform_type'].add('expansion')
#         else:
#             analysis['transform_type'].add('reduction')
            
#     # Symmetry detection
#     if inp.shape == out.shape:
#         # Check for rotational relationship
#         for k in range(1, 4):
#             if np.array_equal(np.rot90(inp, k), out):
#                 analysis['transform_type'].add(f'rotation_{k*90}')
                
#         # Check for mirror
#         if np.array_equal(np.fliplr(inp), out):
#             analysis['transform_type'].add('mirror_horizontal')
#         if np.array_equal(np.flipud(inp), out):
#             analysis['transform_type'].add('mirror_vertical')
            
#     # Pattern repetition detection
#     if out.shape[0] % inp.shape[0] == 0 and out.shape[1] % inp.shape[1] == 0:
#         analysis['transform_type'].add('tiling')
        
#     # Color transformation detection
#     unique_in = set(inp.flatten())
#     unique_out = set(out.flatten())
#     if len(unique_in) == len(unique_out) and unique_in != unique_out:
#         analysis['transform_type'].add('color_mapping')
        
#     # Recommend handlers based on analysis
#     analysis['recommended_handlers'] = recommend_handlers(analysis)
    
#     return analysis

# def try_adaptive_combinations(task_data, pattern_handlers, analysis):
#     """Try adaptive combinations based on pattern analysis"""
#     debug_print("=== Trying adaptive pattern combinations ===", "INFO")
    
#     # Use recommended handlers first
#     recommended = analysis.get('recommended_handlers', [])
#     if recommended:
#         debug_print(f"  Using {len(recommended)} recommended handlers", "INFO")
        
#         for handler in recommended:
#             solution = handler(task_data, analysis)
#             if solution and verify_solution(solution, task_data):
#                 debug_print(f"✓ Recommended handler {handler.__name__} solved it!", "SUCCESS")
#                 return solution
    
#     # Try multi-step combinations (3+ transformations)
#     inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
    
#     # Detect intermediate states
#     intermediate_candidates = detect_intermediate_states(inp, out)
    
#     if intermediate_candidates:
#         debug_print(f"  Found {len(intermediate_candidates)} possible intermediate states", "INFO")
        
#         for intermediate in intermediate_candidates:
#             # Find transformation from input to intermediate
#             step1_handlers = find_transformation(inp, intermediate, pattern_handlers[:20])
            
#             if step1_handlers:
#                 # Find transformation from intermediate to output
#                 step2_handlers = find_transformation(intermediate, out, pattern_handlers[:20])
                
#                 if step2_handlers:
#                     combined = create_multi_step_combination(
#                         step1_handlers[0][1], 
#                         step2_handlers[0][1],
#                         step1_handlers[0][0].__name__,
#                         step2_handlers[0][0].__name__
#                     )
                    
#                     if verify_solution(combined, task_data):
#                         debug_print(f"✓ Found multi-step combination!", "SUCCESS")
#                         return combined
    
#     return None

# def detect_intermediate_states(inp, out):
#     """Detect possible intermediate transformation states"""
#     candidates = []
    
#     # Common intermediate states
#     if inp.shape == out.shape:
#         # Rotations as intermediate
#         for k in range(1, 4):
#             candidates.append(np.rot90(inp, k))
            
#         # Flips as intermediate
#         candidates.append(np.fliplr(inp))
#         candidates.append(np.flipud(inp))
        
#         # Color transformations as intermediate
#         unique_colors = np.unique(inp)
#         if len(unique_colors) <= 5:
#             # Try shifting all colors
#             shifted = np.copy(inp)
#             for color in unique_colors:
#                 if color != 0:
#                     shifted[inp == color] = (color % 9) + 1
#             candidates.append(shifted)
    
#     return candidates

# class PatternLearner:
#     """Learn which handlers work for which patterns"""
#     def __init__(self):
#         self.handler_success_patterns = defaultdict(lambda: defaultdict(int))
#         self.pattern_features = {}
        
#     def extract_features(self, task_data):
#         """Extract features from task"""
#         inp = np.array(task_data['train'][0]['input'])
#         out = np.array(task_data['train'][0]['output'])
        
#         features = {
#             'shape_change': inp.shape != out.shape,
#             'size_ratio': (out.size / inp.size) if inp.size > 0 else 1,
#             'color_count_in': len(np.unique(inp)),
#             'color_count_out': len(np.unique(out)),
#             'has_symmetry': self.check_symmetry(inp),
#             'has_pattern': self.check_pattern(inp),
#             'density': np.sum(inp != 0) / inp.size,
#             'shape_aspect': inp.shape[0] / inp.shape[1] if inp.shape[1] > 0 else 1,
#         }
        
#         return features
    
#     def check_symmetry(self, grid):
#         """Check if grid has symmetry"""
#         return (np.array_equal(grid, np.fliplr(grid)) or 
#                 np.array_equal(grid, np.flipud(grid)) or
#                 (grid.shape[0] == grid.shape[1] and np.array_equal(grid, grid.T)))
    
#     def check_pattern(self, grid):
#         """Check if grid has repeating pattern"""
#         h, w = grid.shape
#         for ph in range(1, h//2 + 1):
#             for pw in range(1, w//2 + 1):
#                 if h % ph == 0 and w % pw == 0:
#                     pattern = grid[:ph, :pw]
#                     tiled = np.tile(pattern, (h//ph, w//pw))
#                     if np.array_equal(grid, tiled):
#                         return True
#         return False
    
#     def update_success(self, handler_name, features, success):
#         """Update handler success for given features"""
#         feature_key = tuple(sorted(features.items()))
#         self.handler_success_patterns[handler_name][feature_key] += (1 if success else -1)
    
#     def get_handler_priority(self, features, handlers):
#         """Get handlers prioritized by past success"""
#         feature_key = tuple(sorted(features.items()))
        
#         scores = []
#         for handler in handlers:
#             score = self.handler_success_patterns[handler.__name__][feature_key]
#             # Add some randomness to explore
#             score += np.random.normal(0, 0.1)
#             scores.append((score, handler))
        
#         # Sort by score descending
#         scores.sort(reverse=True, key=lambda x: x[0])
        
#         return [handler for _, handler in scores]

# def generate_solution_enhanced(task_data, pattern_learner=None):
#     """Enhanced solution generation with intelligent selection and combinations"""
#     global DEBUG_LOG, HANDLER_CACHE
#     DEBUG_LOG = []
    
#     # Advanced analysis
#     analysis = analyze_task_advanced(task_data)
    
#     debug_print(f"\n{'='*60}", "INFO")
#     debug_print(f"Advanced Analysis Results:", "INFO")
#     debug_print(f"  Transform types: {analysis['transform_type']}", "INFO")
#     debug_print(f"  Recommended handlers: {len(analysis.get('recommended_handlers', []))}", "INFO")
    
#     # Get all handlers (including new recursive ones)
#     pattern_handlers = get_all_handlers()
    
#     # Phase 0: Try recommended handlers first
#     if analysis.get('recommended_handlers'):
#         debug_print("\n=== PHASE 0: Trying recommended handlers ===", "INFO")
#         for handler in analysis['recommended_handlers']:
#             solution = handler(task_data, analysis)
#             if solution and verify_solution(solution, task_data):
#                 debug_print(f"✓ Recommended handler {handler.__name__} solved it!", "SUCCESS")
#                 return solution
    
#     # Phase 1: Intelligent handler selection
#     debug_print("\n=== PHASE 1: Intelligent handler selection ===", "INFO")
    
#     if pattern_learner:
#         features = pattern_learner.extract_features(task_data)
#         prioritized_handlers = pattern_learner.get_handler_priority(features, pattern_handlers)
#     else:
#         prioritized_handlers = pattern_handlers
    
#     # Try handlers with early stopping
#     for idx, handler in enumerate(prioritized_handlers[:50]):  # Limit first pass
#         solution = handler(task_data, analysis)
        
#         if solution and verify_solution(solution, task_data):
#             debug_print(f"✓ Handler {handler.__name__} solved it!", "SUCCESS")
            
#             if pattern_learner:
#                 features = pattern_learner.extract_features(task_data)
#                 pattern_learner.update_success(handler.__name__, features, True)
                
#             return solution
    
#     # Phase 2: Adaptive combinations
#     debug_print("\n=== PHASE 2: Adaptive combinations ===", "INFO")
    
#     combination_solution = try_adaptive_combinations(task_data, pattern_handlers, analysis)
#     if combination_solution:
#         return combination_solution
    
#     # Phase 3: Recursive search with backtracking
#     debug_print("\n=== PHASE 3: Recursive transformation search ===", "INFO")
    
#     solution = recursive_transformation_search(
#         task_data, 
#         pattern_handlers[:30], 
#         max_depth=3,
#         analysis=analysis
#     )
#     if solution:
#         return solution
    
#     # Phase 4: Try remaining handlers
#     debug_print("\n=== PHASE 4: Exhaustive search ===", "INFO")
    
#     for handler in prioritized_handlers[50:]:
#         solution = handler(task_data, analysis)
#         if solution and verify_solution(solution, task_data):
#             return solution
    
#     # Fallback
#     return """def p(g):
#  return g
# """

# def recursive_transformation_search(task_data, handlers, max_depth=3, current_depth=0, analysis=None):
#     """Recursively search for transformation combinations"""
#     if current_depth >= max_depth:
#         return None
        
#     inp = np.array(task_data['train'][0]['input'])
#     out = np.array(task_data['train'][0]['output'])
    
#     # Try each handler
#     for handler in handlers[:10]:  # Limit to prevent explosion
#         try:
#             solution = handler(task_data, analysis or {})
#             if solution:
#                 # Create a mock execution to get intermediate result
#                 local_namespace = {}
#                 exec(solution, globals(), local_namespace)
#                 if 'p' in local_namespace:
#                     intermediate = local_namespace['p'](inp.tolist())
                    
#                     if intermediate == out.tolist():
#                         return solution
                    
#                     # Create new task with intermediate as input
#                     new_task = {
#                         'train': [{'input': intermediate, 'output': out.tolist()}],
#                         'test': []
#                     }
                    
#                     # Recursive search
#                     next_solution = recursive_transformation_search(
#                         new_task, 
#                         handlers, 
#                         max_depth, 
#                         current_depth + 1,
#                         analysis
#                     )
                    
#                     if next_solution:
#                         # Combine solutions
#                         return create_recursive_combination(solution, next_solution, handler.__name__)
#         except:
#             pass
            
#     return None

# def generate_solution_enhanced(task_data, pattern_learner=None):
#     """Enhanced solution generation with intelligent selection and combinations"""
#     global DEBUG_LOG, HANDLER_CACHE
#     DEBUG_LOG = []
    
#     # Advanced analysis
#     analysis = analyze_task_advanced(task_data)
    
#     debug_print(f"\n{'='*60}", "INFO")
#     debug_print(f"Advanced Analysis Results:", "INFO")
#     debug_print(f"  Transform types: {analysis['transform_type']}", "INFO")
#     debug_print(f"  Recommended handlers: {len(analysis.get('recommended_handlers', []))}", "INFO")
    
#     # Get all handlers (including new recursive ones)
#     pattern_handlers = get_all_handlers()
    
#     # Phase 0: Try recommended handlers first
#     if analysis.get('recommended_handlers'):
#         debug_print("\n=== PHASE 0: Trying recommended handlers ===", "INFO")
#         for handler in analysis['recommended_handlers']:
#             solution = handler(task_data, analysis)
#             if solution and verify_solution(solution, task_data):
#                 debug_print(f"✓ Recommended handler {handler.__name__} solved it!", "SUCCESS")
#                 return solution
    
#     # Phase 1: Intelligent handler selection
#     debug_print("\n=== PHASE 1: Intelligent handler selection ===", "INFO")
    
#     if pattern_learner:
#         features = pattern_learner.extract_features(task_data)
#         prioritized_handlers = pattern_learner.get_handler_priority(features, pattern_handlers)
#     else:
#         prioritized_handlers = pattern_handlers
    
#     # Try handlers with early stopping
#     for idx, handler in enumerate(prioritized_handlers[:50]):  # Limit first pass
#         solution = handler(task_data, analysis)
        
#         if solution and verify_solution(solution, task_data):
#             debug_print(f"✓ Handler {handler.__name__} solved it!", "SUCCESS")
            
#             if pattern_learner:
#                 features = pattern_learner.extract_features(task_data)
#                 pattern_learner.update_success(handler.__name__, features, True)
                
#             return solution
    
#     # Phase 2: Adaptive combinations
#     debug_print("\n=== PHASE 2: Adaptive combinations ===", "INFO")
    
#     combination_solution = try_adaptive_combinations(task_data, pattern_handlers, analysis)
#     if combination_solution:
#         return combination_solution
    
#     # Phase 3: Recursive search with backtracking
#     debug_print("\n=== PHASE 3: Recursive transformation search ===", "INFO")
    
#     solution = recursive_transformation_search(
#         task_data, 
#         pattern_handlers[:30], 
#         max_depth=3,
#         analysis=analysis
#     )
#     if solution:
#         return solution
    
#     # Phase 4: Try remaining handlers
#     debug_print("\n=== PHASE 4: Exhaustive search ===", "INFO")
    
#     for handler in prioritized_handlers[50:]:
#         solution = handler(task_data, analysis)
#         if solution and verify_solution(solution, task_data):
#             return solution
    
#     # Fallback
#     return """def p(g):
#  return g
# """

# def recursive_transformation_search(task_data, handlers, max_depth=3, current_depth=0, analysis=None):
#     """Recursively search for transformation combinations"""
#     if current_depth >= max_depth:
#         return None
        
#     inp = np.array(task_data['train'][0]['input'])
#     out = np.array(task_data['train'][0]['output'])
    
#     # Try each handler
#     for handler in handlers[:10]:  # Limit to prevent explosion
#         try:
#             solution = handler(task_data, analysis or {})
#             if solution:
#                 # Create a mock execution to get intermediate result
#                 local_namespace = {}
#                 exec(solution, globals(), local_namespace)
#                 if 'p' in local_namespace:
#                     intermediate = local_namespace['p'](inp.tolist())
                    
#                     if intermediate == out.tolist():
#                         return solution
                    
#                     # Create new task with intermediate as input
#                     new_task = {
#                         'train': [{'input': intermediate, 'output': out.tolist()}],
#                         'test': []
#                     }
                    
#                     # Recursive search
#                     next_solution = recursive_transformation_search(
#                         new_task, 
#                         handlers, 
#                         max_depth, 
#                         current_depth + 1,
#                         analysis
#                     )
                    
#                     if next_solution:
#                         # Combine solutions
#                         return create_recursive_combination(solution, next_solution, handler.__name__)
#         except:
#             pass
            
#     return None

# def create_recursive_combination(sol1, sol2, name1):
#     """Create a recursive combination of solutions"""
#     body1 = extract_function_body(sol1)
#     body2 = extract_function_body(sol2)
    
#     return f"""def p(g):
#  # Step 1: {name1}
#  def t1(g):
# {indent_code(body1, 2)}
 
#  # Step 2: Recursive transformation
#  def t2(g):
# {indent_code(body2, 2)}
 
#  intermediate = t1(g)
#  return t2(intermediate)
# """

# def get_all_handlers():
#     """Get all handlers including new ones"""
#     return [
#         # ... all existing handlers ...
#         handle_recursive_pattern_completion,
#         handle_recursive_growth,
#         # Add more recursive handlers here
#     ] + get_existing_handlers()  # This would be your current handler list

# @timed_handler
# def handle_contextual_transformation(task_data, analysis):
#     """Transform based on local context"""
#     debug_print(f"Trying handle_contextual_transformation")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.zeros_like(inp)
            
#             # Transform each cell based on its context
#             for i in range(h):
#                 for j in range(w):
#                     # Get 3x3 context
#                     context = []
#                     for di in range(-1, 2):
#                         for dj in range(-1, 2):
#                             ni, nj = i + di, j + dj
#                             if 0 <= ni < h and 0 <= nj < w:
#                                 context.append(inp[ni,nj])
#                             else:
#                                 context.append(0)
                    
#                     # Apply context-based transformation
#                     context_hash = sum(c * (10 ** idx) for idx, c in enumerate(context)) % 10
#                     test[i,j] = context_hash
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found contextual transformation")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[[0]*w for _ in range(h)]
#  for i in range(h):
#   for j in range(w):
#    ctx=[]
#    for di in range(-1,2):
#     for dj in range(-1,2):
#      ni,nj=i+di,j+dj
#      if 0<=ni<h and 0<=nj<w:
#       ctx.append(g[ni][nj])
#      else:
#       ctx.append(0)
#    ch=sum(c*(10**idx)for idx,c in enumerate(ctx))%10
#    res[i][j]=ch
#  return res
# """
#         debug_print(f"  ✗ Not a contextual transformation")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_contextual_transformation: {str(e)}")
#     return None

# @timed_handler
# def handle_probabilistic_fill(task_data, analysis):
#     """Fill based on probabilistic rules"""
#     debug_print(f"Trying handle_probabilistic_fill")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             # Analyze color probabilities
#             color_probs = defaultdict(lambda: defaultdict(int))
            
#             for i in range(h):
#                 for j in range(w):
#                     if inp[i,j] != 0:
#                         # Count neighbors
#                         for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
#                             ni, nj = i + di, j + dj
#                             if 0 <= ni < h and 0 <= nj < w:
#                                 color_probs[inp[i,j]][inp[ni,nj]] += 1
            
#             # Apply probabilistic rules
#             test = np.copy(inp)
#             for i in range(h):
#                 for j in range(w):
#                     if inp[i,j] == 0:
#                         # Get neighbor colors
#                         neighbor_colors = []
#                         for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
#                             ni, nj = i + di, j + dj
#                             if 0 <= ni < h and 0 <= nj < w and inp[ni,nj] != 0:
#                                 neighbor_colors.append(inp[ni,nj])
                        
#                         if neighbor_colors:
#                             # Choose based on probability
#                             color_scores = defaultdict(int)
#                             for nc in neighbor_colors:
#                                 for target, count in color_probs[nc].items():
#                                     color_scores[target] += count
                            
#                             if color_scores:
#                                 best_color = max(color_scores.items(), key=lambda x: x[1])[0]
#                                 test[i,j] = best_color
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found probabilistic fill")
#                 # Return simplified version
#                 return """def p(g):
#  from collections import defaultdict
#  h,w=len(g),len(g[0])
#  res=[r[:]for r in g]
#  # Simplified probabilistic fill
#  for i in range(h):
#   for j in range(w):
#    if g[i][j]==0:
#     nc=[]
#     for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
#      ni,nj=i+di,j+dj
#      if 0<=ni<h and 0<=nj<w and g[ni][nj]!=0:
#       nc.append(g[ni][nj])
#     if nc:
#      res[i][j]=max(set(nc),key=nc.count)
#  return res
# """
#         debug_print(f"  ✗ Not a probabilistic fill")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_probabilistic_fill: {str(e)}")
#     return None

# def create_recursive_combination(sol1, sol2, name1):
#     """Create a recursive combination of solutions"""
#     body1 = extract_function_body(sol1)
#     body2 = extract_function_body(sol2)
    
#     return f"""def p(g):
#  # Step 1: {name1}
#  def t1(g):
# {indent_code(body1, 2)}
 
#  # Step 2: Recursive transformation
#  def t2(g):
# {indent_code(body2, 2)}
 
#  intermediate = t1(g)
#  return t2(intermediate)
# """

# def get_all_handlers():
#     """Get all handlers including new ones"""
#     return [
#         # ... all existing handlers ...
#         handle_recursive_pattern_completion,
#         handle_recursive_growth,
#         # Add more recursive handlers here
#     ] + get_existing_handlers()  # This would be your current handler list

# def find_transformation(inp, out, handlers):
#     """Find handlers that transform inp to out"""
#     mock_task = {
#         'train': [{'input': inp.tolist(), 'output': out.tolist()}],
#         'test': []
#     }
    
#     matches = []
#     for handler in handlers:
#         try:
#             solution = handler(mock_task, {})
#             if solution and verify_solution(solution, mock_task):
#                 matches.append((handler, solution))
#         except:
#             pass
            
#     return matches

# def recommend_handlers(analysis):
#     """Recommend handlers based on pattern analysis"""
#     recommendations = []
    
#     handler_patterns = {
#         'rotation_90': [handle_rotate_90],
#         'rotation_180': [handle_rotate_180],
#         'rotation_270': [handle_rotate_270],
#         'mirror_horizontal': [handle_flip_horizontal],
#         'mirror_vertical': [handle_flip_vertical],
#         'tiling': [handle_tile_pattern, handle_repeat_pattern],
#         'expansion': [handle_upscale, handle_pad_to_square, handle_border_operations],
#         'reduction': [handle_downscale, handle_crop_to_content, handle_extract_pattern],
#         'color_mapping': [handle_color_mapping, handle_layered_transform, handle_color_cycle],
#     }
    
#     for pattern_type in analysis['transform_type']:
#         if pattern_type in handler_patterns:
#             recommendations.extend(handler_patterns[pattern_type])
            
#     return recommendations

# # Initialize pattern learner
# learner = PatternLearner()

# # Enhanced main execution
# def create_arc_solutions_ultra(input_dir=".", output_dir="submission"):
#     """Ultra-enhanced main execution with learning"""
#     solutions = {}
    
#     os.makedirs(output_dir, exist_ok=True)
    
#     solved_count = 0
#     total_count = 0
    
#     for task_num in range(1, 401):
#         task_id = f"{task_num:03d}"
#         task_file = os.path.join(input_dir, f"task{task_id}.json")
        
#         try:
#             with open(task_file) as f:
#                 task_data = json.load(f)
            
#             # Use enhanced generator with learning
#             solution = generate_solution_enhanced(task_data, learner)
#             solutions[task_id] = solution
            
#             if verify_solution(solution, task_data):
#                 print(f"✅ Task {task_id} solved")
#                 solved_count += 1
                
#                 # Update learner with success
#                 features = learner.extract_features(task_data)
#                 # Extract handler name from solution if possible
#                 for line in solution.split('\n'):
#                     if '# Step 1:' in line or '✓ Found' in DEBUG_LOG:
#                         handler_name = line.split(':')[1].strip()
#                         learner.update_success(handler_name, features, True)
#                         break
#             else:
#                 print(f"❌ Task {task_id} fallback")
            
#             total_count += 1
            
#         except Exception as e:
#             print(f"Error task {task_id}: {str(e)}")
#             solutions[task_id] = """def p(g):
#  return g
# """
#             total_count += 1
    
#     # Save solutions and create submission
#     # ... rest of the code ...
    
# def recommend_handlers(analysis):
#     """Recommend handlers based on pattern analysis"""
#     recommendations = []
    
#     handler_patterns = {
#         'rotation_90': [handle_rotate_90],
#         'rotation_180': [handle_rotate_180],
#         'rotation_270': [handle_rotate_270],
#         'mirror_horizontal': [handle_flip_horizontal],
#         'mirror_vertical': [handle_flip_vertical],
#         'tiling': [handle_tile_pattern, handle_repeat_pattern],
#         'expansion': [handle_upscale, handle_pad_to_square, handle_border_operations],
#         'reduction': [handle_downscale, handle_crop_to_content, handle_extract_pattern],
#         'color_mapping': [handle_color_mapping, handle_layered_transform, handle_color_cycle],
#     }
    
#     for pattern_type in analysis['transform_type']:
#         if pattern_type in handler_patterns:
#             recommendations.extend(handler_patterns[pattern_type])
            
#     return recommendations

# import time
# from functools import lru_cache
# from typing import Dict, List, Tuple, Optional, Callable
# import numpy as np
# from collections import defaultdict, Counter
# import json
# import os
# import zipfile

# # Debug mode configuration
# DEBUG_MODE = True
# DEBUG_LOG = []
# PERFORMANCE_STATS = defaultdict(lambda: {'calls': 0, 'successes': 0, 'total_time': 0})

# def debug_print(msg, level="INFO"):
#     """Enhanced debug printing with levels and timing"""
#     if DEBUG_MODE:
#         timestamp = time.strftime("%H:%M:%S")
#         formatted_msg = f"[{timestamp}] [{level}] {msg}"
#         print(formatted_msg)
#         DEBUG_LOG.append(formatted_msg)

# # Cache for handler results to avoid recomputation
# HANDLER_CACHE = {}

# def cache_key(task_data):
#     """Generate a cache key for task data"""
#     inp = np.array(task_data['train'][0]['input'])
#     out = np.array(task_data['train'][0]['output'])
#     return f"{inp.shape}_{out.shape}_{hash(inp.tobytes())}_{hash(out.tobytes())}"

# # Pattern combination helpers
# def extract_function_body(solution_code):
#     """Extract the function body from a solution string"""
#     lines = solution_code.strip().split('\n')
#     if not lines or not lines[0].startswith('def p(g):'):
#         return ""
#     # Skip the def line and get the body
#     body_lines = []
#     for line in lines[1:]:
#         if line.startswith(' '):
#             body_lines.append(line[1:])  # Remove one space of indentation
#     return '\n'.join(body_lines)

# def indent_code(code, spaces):
#     """Indent code by specified spaces"""
#     return '\n'.join(' ' * spaces + line for line in code.split('\n'))

# # Enhanced pattern handlers with performance tracking
# def timed_handler(func):
#     """Decorator to time handler execution"""
#     def wrapper(task_data, analysis):
#         start_time = time.time()
#         result = None
#         try:
#             result = func(task_data, analysis)
#             success = result is not None
#         except Exception as e:
#             success = False
#             debug_print(f"Handler {func.__name__} failed: {str(e)}", "ERROR")
        
#         elapsed = time.time() - start_time
#         PERFORMANCE_STATS[func.__name__]['calls'] += 1
#         PERFORMANCE_STATS[func.__name__]['total_time'] += elapsed
#         if success:
#             PERFORMANCE_STATS[func.__name__]['successes'] += 1
        
#         return result
#     return wrapper

# # Additional pattern handlers to add before analyze_task

# @timed_handler
# def handle_diagonal_mirror(task_data, analysis):
#     """Mirror along main diagonal or anti-diagonal"""
#     debug_print(f"Trying handle_diagonal_mirror")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape and inp.shape[0] == inp.shape[1]:
#             n = inp.shape[0]
            
#             # Main diagonal mirror
#             test = np.zeros_like(inp)
#             for i in range(n):
#                 for j in range(n):
#                     if inp[i,j] != 0:
#                         test[i,j] = inp[i,j]
#                         test[j,i] = inp[i,j]
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found diagonal mirror pattern (size={n}x{n})")
#                 return """def p(g):
#  n=len(g)
#  res=[r[:]for r in g]
#  for i in range(n):
#   for j in range(n):
#    if g[i][j]!=0:
#     res[j][i]=g[i][j]
#  return res
# """
#         debug_print(f"  ✗ Not a diagonal mirror pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_diagonal_mirror: {str(e)}")
#     return None

# @timed_handler
# def handle_color_inversion(task_data, analysis):
#     """Invert colors (e.g., 9-x for each color x)"""
#     debug_print(f"Trying handle_color_inversion")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             test = 9 - inp
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found color inversion (9-x)")
#                 return """def p(g):
#  return[[9-x for x in r]for r in g]
# """
#         debug_print(f"  ✗ Not a color inversion pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_color_inversion: {str(e)}")
#     return None

# @timed_handler
# def handle_distance_coloring(task_data, analysis):
#     """Color based on Manhattan/Euclidean distance from specific points"""
#     debug_print(f"Trying handle_distance_coloring")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             # Find non-zero points in input
#             seeds = [(i, j) for i in range(h) for j in range(w) if inp[i,j] != 0]
            
#             if seeds:
#                 test = np.zeros_like(inp)
#                 for i in range(h):
#                     for j in range(w):
#                         min_dist = min(abs(i-si) + abs(j-sj) for si, sj in seeds)
#                         test[i,j] = min(min_dist, 9)
                
#                 if np.array_equal(test, out):
#                     debug_print(f"  ✓ Found distance coloring with {len(seeds)} seed points")
#                     return """def p(g):
#  h,w=len(g),len(g[0])
#  seeds=[(i,j)for i in range(h)for j in range(w)if g[i][j]!=0]
#  res=[[0]*w for _ in range(h)]
#  for i in range(h):
#   for j in range(w):
#    md=min(abs(i-si)+abs(j-sj)for si,sj in seeds)
#    res[i][j]=min(md,9)
#  return res
# """
#         debug_print(f"  ✗ Not a distance coloring pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_distance_coloring: {str(e)}")
#     return None

# @timed_handler
# def handle_neighbor_sum_mod(task_data, analysis):
#     """Color based on sum of neighbors modulo some value"""
#     debug_print(f"Trying handle_neighbor_sum_mod")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             for mod in range(2, 10):
#                 test = np.zeros_like(inp)
#                 for i in range(h):
#                     for j in range(w):
#                         neighbor_sum = 0
#                         for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
#                             ni, nj = i + di, j + dj
#                             if 0 <= ni < h and 0 <= nj < w:
#                                 neighbor_sum += inp[ni,nj]
#                         test[i,j] = neighbor_sum % mod
                
#                 if np.array_equal(test, out):
#                     debug_print(f"  ✓ Found neighbor sum mod {mod}")
#                     return f"""def p(g):
#  h,w=len(g),len(g[0])
#  res=[[0]*w for _ in range(h)]
#  for i in range(h):
#   for j in range(w):
#    ns=0
#    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
#     ni,nj=i+di,j+dj
#     if 0<=ni<h and 0<=nj<w:
#      ns+=g[ni][nj]
#    res[i][j]=ns%{mod}
#  return res
# """
#         debug_print(f"  ✗ Not a neighbor sum mod pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_neighbor_sum_mod: {str(e)}")
#     return None

# @timed_handler
# def handle_wave_propagation(task_data, analysis):
#     """Propagate values outward like waves"""
#     debug_print(f"Trying handle_wave_propagation")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.copy(inp)
            
#             # Try different wave propagation patterns
#             for steps in range(1, 4):
#                 current = np.copy(inp)
#                 for _ in range(steps):
#                     next_grid = np.copy(current)
#                     for i in range(h):
#                         for j in range(w):
#                             if current[i,j] == 0:
#                                 for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
#                                     ni, nj = i + di, j + dj
#                                     if 0 <= ni < h and 0 <= nj < w and current[ni,nj] != 0:
#                                         next_grid[i,j] = (current[ni,nj] + 1) % 10
#                                         break
#                     current = next_grid
                
#                 if np.array_equal(current, out):
#                     debug_print(f"  ✓ Found wave propagation with {steps} steps")
#                     return f"""def p(g):
#  h,w=len(g),len(g[0])
#  c=[r[:]for r in g]
#  for _ in range({steps}):
#   n=[r[:]for r in c]
#   for i in range(h):
#    for j in range(w):
#     if c[i][j]==0:
#      for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
#       ni,nj=i+di,j+dj
#       if 0<=ni<h and 0<=nj<w and c[ni][nj]!=0:
#        n[i][j]=(c[ni][nj]+1)%10
#        break
#   c=n
#  return c
# """
#         debug_print(f"  ✗ Not a wave propagation pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_wave_propagation: {str(e)}")
#     return None

# @timed_handler
# def handle_quadrant_operations(task_data, analysis):
#     """Apply different operations to different quadrants"""
#     debug_print(f"Trying handle_quadrant_operations")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             mh, mw = h // 2, w // 2
            
#             test = np.copy(inp)
            
#             # Try different operations per quadrant
#             # Top-left: +1, Top-right: +2, Bottom-left: +3, Bottom-right: +4
#             test[:mh, :mw] = (inp[:mh, :mw] + 1) % 10
#             test[:mh, mw:] = (inp[:mh, mw:] + 2) % 10
#             test[mh:, :mw] = (inp[mh:, :mw] + 3) % 10
#             test[mh:, mw:] = (inp[mh:, mw:] + 4) % 10
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found quadrant operations (+1,+2,+3,+4)")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  mh,mw=h//2,w//2
#  res=[r[:]for r in g]
#  for i in range(mh):
#   for j in range(mw):
#    res[i][j]=(g[i][j]+1)%10
#  for i in range(mh):
#   for j in range(mw,w):
#    res[i][j]=(g[i][j]+2)%10
#  for i in range(mh,h):
#   for j in range(mw):
#    res[i][j]=(g[i][j]+3)%10
#  for i in range(mh,h):
#   for j in range(mw,w):
#    res[i][j]=(g[i][j]+4)%10
#  return res
# """
#         debug_print(f"  ✗ Not a quadrant operations pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_quadrant_operations: {str(e)}")
#     return None

# @timed_handler
# def handle_ring_operations(task_data, analysis):
#     """Apply operations to concentric rings"""
#     debug_print(f"Trying handle_ring_operations")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.copy(inp)
            
#             # Color based on ring distance from edge
#             for i in range(h):
#                 for j in range(w):
#                     ring = min(i, j, h-1-i, w-1-j)
#                     test[i,j] = (inp[i,j] + ring) % 10
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found ring operations")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[]
#  for i in range(h):
#   row=[]
#   for j in range(w):
#    r=min(i,j,h-1-i,w-1-j)
#    row.append((g[i][j]+r)%10)
#   res.append(row)
#  return res
# """
#         debug_print(f"  ✗ Not a ring operations pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_ring_operations: {str(e)}")
#     return None

# @timed_handler
# def handle_pattern_frequency(task_data, analysis):
#     """Replace patterns based on their frequency"""
#     debug_print(f"Trying handle_pattern_frequency")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             # Count 2x2 patterns
#             patterns = defaultdict(list)
#             for i in range(h-1):
#                 for j in range(w-1):
#                     pattern = (inp[i,j], inp[i,j+1], inp[i+1,j], inp[i+1,j+1])
#                     patterns[pattern].append((i, j))
            
#             # Replace most common pattern with specific value
#             if patterns:
#                 most_common = max(patterns.items(), key=lambda x: len(x[1]))
#                 test = np.copy(inp)
                
#                 for i, j in most_common[1]:
#                     test[i:i+2, j:j+2] = 5
                
#                 if np.array_equal(test, out):
#                     debug_print(f"  ✓ Found pattern frequency replacement")
#                     return """def p(g):
#  from collections import defaultdict
#  h,w=len(g),len(g[0])
#  pats=defaultdict(list)
#  for i in range(h-1):
#   for j in range(w-1):
#    pat=(g[i][j],g[i][j+1],g[i+1][j],g[i+1][j+1])
#    pats[pat].append((i,j))
#  if pats:
#   mc=max(pats.items(),key=lambda x:len(x[1]))
#   res=[r[:]for r in g]
#   for i,j in mc[1]:
#    res[i][j]=5
#    res[i][j+1]=5
#    res[i+1][j]=5
#    res[i+1][j+1]=5
#   return res
#  return g
# """
#         debug_print(f"  ✗ Not a pattern frequency replacement")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_pattern_frequency: {str(e)}")
#     return None

# @timed_handler
# def handle_sliding_window_ops(task_data, analysis):
#     """Apply operations using sliding windows"""
#     debug_print(f"Trying handle_sliding_window_ops")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             # 3x3 sliding window - set center to max of window
#             test = np.copy(inp)
#             for i in range(1, h-1):
#                 for j in range(1, w-1):
#                     window_max = 0
#                     for di in range(-1, 2):
#                         for dj in range(-1, 2):
#                             window_max = max(window_max, inp[i+di, j+dj])
#                     test[i,j] = window_max
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found sliding window max operation")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[r[:]for r in g]
#  for i in range(1,h-1):
#   for j in range(1,w-1):
#    wm=0
#    for di in range(-1,2):
#     for dj in range(-1,2):
#      wm=max(wm,g[i+di][j+dj])
#    res[i][j]=wm
#  return res
# """
#         debug_print(f"  ✗ Not a sliding window operation")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_sliding_window_ops: {str(e)}")
#     return None

# @timed_handler
# def handle_cross_product_transform(task_data, analysis):
#     """Transform based on row/column products"""
#     debug_print(f"Trying handle_cross_product_transform")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             # Set each cell to (row_sum * col_sum) % 10
#             row_sums = [sum(inp[i,:]) for i in range(h)]
#             col_sums = [sum(inp[:,j]) for j in range(w)]
            
#             test = np.zeros_like(inp)
#             for i in range(h):
#                 for j in range(w):
#                     test[i,j] = (row_sums[i] * col_sums[j]) % 10
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found cross product transform")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  rs=[sum(g[i])for i in range(h)]
#  cs=[sum(g[i][j]for i in range(h))for j in range(w)]
#  res=[]
#  for i in range(h):
#   row=[]
#   for j in range(w):
#    row.append((rs[i]*cs[j])%10)
#   res.append(row)
#  return res
# """
#         debug_print(f"  ✗ Not a cross product transform")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_cross_product_transform: {str(e)}")
#     return None

# @timed_handler
# def handle_spiral_fill(task_data, analysis):
#     """Fill grid in spiral pattern"""
#     debug_print(f"Trying handle_spiral_fill")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         h, w = out.shape
#         test = np.zeros((h, w), dtype=int)
        
#         # Generate spiral pattern with incrementing values
#         val = 1
#         top, bottom, left, right = 0, h-1, 0, w-1
        
#         while top <= bottom and left <= right:
#             for j in range(left, right+1):
#                 test[top][j] = val % 10
#                 val += 1
#             top += 1
            
#             for i in range(top, bottom+1):
#                 test[i][right] = val % 10
#                 val += 1
#             right -= 1
            
#             if top <= bottom:
#                 for j in range(right, left-1, -1):
#                     test[bottom][j] = val % 10
#                     val += 1
#                 bottom -= 1
            
#             if left <= right:
#                 for i in range(bottom, top-1, -1):
#                     test[i][left] = val % 10
#                     val += 1
#                 left += 1
        
#         if np.array_equal(test, out):
#             debug_print(f"  ✓ Found spiral fill pattern")
#             return f"""def p(g):
#  h,w={h},{w}
#  res=[[0]*w for _ in range(h)]
#  v=1
#  t,b,l,r=0,h-1,0,w-1
#  while t<=b and l<=r:
#   for j in range(l,r+1):
#    res[t][j]=v%10
#    v+=1
#   t+=1
#   for i in range(t,b+1):
#    res[i][r]=v%10
#    v+=1
#   r-=1
#   if t<=b:
#    for j in range(r,l-1,-1):
#     res[b][j]=v%10
#     v+=1
#    b-=1
#   if l<=r:
#    for i in range(b,t-1,-1):
#     res[i][l]=v%10
#     v+=1
#    l+=1
#  return res
# """
#         debug_print(f"  ✗ Not a spiral fill pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_spiral_fill: {str(e)}")
#     return None

# @timed_handler
# def handle_border_operations(task_data, analysis):
#     """Add or manipulate borders"""
#     debug_print(f"Trying handle_border_operations")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         # Add border
#         if out.shape[0] == inp.shape[0] + 2 and out.shape[1] == inp.shape[1] + 2:
#             for border_val in range(1, 10):
#                 test = np.full((inp.shape[0]+2, inp.shape[1]+2), border_val)
#                 test[1:-1, 1:-1] = inp
#                 if np.array_equal(test, out):
#                     debug_print(f"  ✓ Found border addition with value {border_val}")
#                     return f"""def p(g):
#  h,w=len(g),len(g[0])
#  res=[[{border_val}]*(w+2)for _ in range(h+2)]
#  for i in range(h):
#   for j in range(w):
#    res[i+1][j+1]=g[i][j]
#  return res
# """
        
#         # Remove border
#         if inp.shape[0] == out.shape[0] + 2 and inp.shape[1] == out.shape[1] + 2:
#             if np.array_equal(inp[1:-1, 1:-1], out):
#                 debug_print(f"  ✓ Found border removal")
#                 return """def p(g):
#  return[r[1:-1]for r in g[1:-1]]
# """
#         debug_print(f"  ✗ Not a border operation")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_border_operations: {str(e)}")
#     return None

# @timed_handler
# def handle_corner_operations(task_data, analysis):
#     """Special operations on corners"""
#     debug_print(f"Trying handle_corner_operations")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.copy(inp)
            
#             # Set corners to specific value
#             for val in range(1, 10):
#                 test[0,0] = val
#                 test[0,w-1] = val
#                 test[h-1,0] = val
#                 test[h-1,w-1] = val
                
#                 if np.array_equal(test, out):
#                     debug_print(f"  ✓ Found corner operations with value {val}")
#                     return f"""def p(g):
#  h,w=len(g),len(g[0])
#  res=[r[:]for r in g]
#  res[0][0]={val}
#  res[0][w-1]={val}
#  res[h-1][0]={val}
#  res[h-1][w-1]={val}
#  return res
# """
#                 test = np.copy(inp)
#         debug_print(f"  ✗ Not a corner operation")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_corner_operations: {str(e)}")
#     return None

# @timed_handler
# def handle_chess_pattern(task_data, analysis):
#     """Create chess-like patterns"""
#     debug_print(f"Trying handle_chess_pattern")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         h, w = out.shape
        
#         # Standard chess pattern
#         for c1 in range(10):
#             for c2 in range(10):
#                 if c1 != c2:
#                     test = np.zeros((h, w), dtype=int)
#                     for i in range(h):
#                         for j in range(w):
#                             test[i,j] = c1 if (i+j) % 2 == 0 else c2
                    
#                     if np.array_equal(test, out):
#                         debug_print(f"  ✓ Found chess pattern with colors {c1},{c2}")
#                         return f"""def p(g):
#  h,w={h},{w}
#  return[[{c1} if(i+j)%2==0 else {c2} for j in range(w)]for i in range(h)]
# """
#         debug_print(f"  ✗ Not a chess pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_chess_pattern: {str(e)}")
#     return None

# @timed_handler
# def handle_diamond_pattern(task_data, analysis):
#     """Create diamond/rhombus patterns"""
#     debug_print(f"Trying handle_diamond_pattern")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         h, w = out.shape
#         cx, cy = w // 2, h // 2
        
#         test = np.zeros((h, w), dtype=int)
#         for i in range(h):
#             for j in range(w):
#                 manhattan_dist = abs(i - cy) + abs(j - cx)
#                 test[i,j] = manhattan_dist % 10
        
#         if np.array_equal(test, out):
#             debug_print(f"  ✓ Found diamond pattern")
#             return f"""def p(g):
#  h,w={h},{w}
#  cx,cy={cx},{cy}
#  res=[]
#  for i in range(h):
#   row=[]
#   for j in range(w):
#    d=abs(i-cy)+abs(j-cx)
#    row.append(d%10)
#   res.append(row)
#  return res
# """
#         debug_print(f"  ✗ Not a diamond pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_diamond_pattern: {str(e)}")
#     return None

# @timed_handler
# def handle_triangular_numbers(task_data, analysis):
#     """Fill with triangular number patterns"""
#     debug_print(f"Trying handle_triangular_numbers")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         h, w = out.shape
#         test = np.zeros((h, w), dtype=int)
        
#         # Fill with triangular numbers
#         val = 0
#         for i in range(h):
#             for j in range(w):
#                 test[i,j] = val % 10
#                 val += (i * w + j + 1)
        
#         if np.array_equal(test, out):
#             debug_print(f"  ✓ Found triangular numbers pattern")
#             return f"""def p(g):
#  h,w={h},{w}
#  res=[]
#  v=0
#  for i in range(h):
#   row=[]
#   for j in range(w):
#    row.append(v%10)
#    v+=i*w+j+1
#   res.append(row)
#  return res
# """
#         debug_print(f"  ✗ Not a triangular numbers pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_triangular_numbers: {str(e)}")
#     return None

# @timed_handler
# def handle_color_cycle(task_data, analysis):
#     """Cycle colors by fixed amount"""
#     debug_print(f"Trying handle_color_cycle")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             for shift in range(1, 10):
#                 test = np.zeros_like(inp)
#                 for i in range(inp.shape[0]):
#                     for j in range(inp.shape[1]):
#                         if inp[i,j] != 0:
#                             test[i,j] = ((inp[i,j] - 1 + shift) % 9) + 1
#                         else:
#                             test[i,j] = 0
                
#                 if np.array_equal(test, out):
#                     debug_print(f"  ✓ Found color cycle with shift {shift}")
#                     return f"""def p(g):
#  return[[((x-1+{shift})%9)+1 if x!=0 else 0 for x in r]for r in g]
# """
#         debug_print(f"  ✗ Not a color cycle pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_color_cycle: {str(e)}")
#     return None

# @timed_handler
# def handle_connectivity_color(task_data, analysis):
#     """Color based on connectivity"""
#     debug_print(f"Trying handle_connectivity_color")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.zeros_like(inp)
            
#             for i in range(h):
#                 for j in range(w):
#                     if inp[i,j] != 0:
#                         # Count connected neighbors
#                         count = 0
#                         for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
#                             ni, nj = i + di, j + dj
#                             if 0 <= ni < h and 0 <= nj < w and inp[ni,nj] != 0:
#                                 count += 1
#                         test[i,j] = count + 1
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found connectivity coloring")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[[0]*w for _ in range(h)]
#  for i in range(h):
#   for j in range(w):
#    if g[i][j]!=0:
#     c=0
#     for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
#      ni,nj=i+di,j+dj
#      if 0<=ni<h and 0<=nj<w and g[ni][nj]!=0:
#       c+=1
#     res[i][j]=c+1
#  return res
# """
#         debug_print(f"  ✗ Not a connectivity coloring pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_connectivity_color: {str(e)}")
#     return None

# @timed_handler
# def handle_layered_transform(task_data, analysis):
#     """Transform different color layers differently"""
#     debug_print(f"Trying handle_layered_transform")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             test = np.copy(inp)
            
#             # Try shifting each color layer
#             unique_colors = np.unique(inp)
#             shifts = {}
            
#             # Check if it's a simple per-color shift
#             for color in unique_colors:
#                 if color == 0:
#                     continue
                
#                 mask = (inp == color)
#                 out_vals = out[mask]
                
#                 if len(np.unique(out_vals)) == 1:
#                     shifts[int(color)] = int(out_vals[0])
            
#             if shifts:
#                 for i in range(inp.shape[0]):
#                     for j in range(inp.shape[1]):
#                         if inp[i,j] in shifts:
#                             test[i,j] = shifts[inp[i,j]]
                
#                 if np.array_equal(test, out):
#                     shift_str = ','.join(f'{k}:{v}' for k,v in shifts.items())
#                     debug_print(f"  ✓ Found layered transform with mapping {{{shift_str}}}")
#                     return f"""def p(g):
#  m={{{shift_str}}}
#  return[[m.get(x,x)for x in r]for r in g]
# """
#         debug_print(f"  ✗ Not a layered transform")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_layered_transform: {str(e)}")
#     return None

# @timed_handler
# def handle_local_symmetry(task_data, analysis):
#     """Complete local symmetry patterns"""
#     debug_print(f"Trying handle_local_symmetry")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.copy(inp)
            
#             # Try to complete 3x3 symmetric patterns
#             for i in range(1, h-1):
#                 for j in range(1, w-1):
#                     if inp[i,j] == 0:
#                         # Check if surrounding forms symmetric pattern
#                         vals = []
#                         for di, dj in [(-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1)]:
#                             vals.append(inp[i+di, j+dj])
                        
#                         # Check various symmetries
#                         if vals[0] == vals[7] and vals[1] == vals[6] and vals[2] == vals[5] and vals[3] == vals[4]:
#                             # Point symmetric
#                             if vals[0] != 0:
#                                 test[i,j] = vals[0]
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found local symmetry completion")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[r[:]for r in g]
#  for i in range(1,h-1):
#   for j in range(1,w-1):
#    if g[i][j]==0:
#     v=[]
#     for di,dj in[(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]:
#      v.append(g[i+di][j+dj])
#     if v[0]==v[7] and v[1]==v[6] and v[2]==v[5] and v[3]==v[4] and v[0]!=0:
#      res[i][j]=v[0]
#  return res
# """
#         debug_print(f"  ✗ Not a local symmetry pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_local_symmetry: {str(e)}")
#     return None

# @timed_handler
# def handle_shape_detection(task_data, analysis):
#     """Detect and transform specific shapes"""
#     debug_print(f"Trying handle_shape_detection")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.copy(inp)
            
#             # Detect 2x2 squares and fill them
#             for i in range(h-1):
#                 for j in range(w-1):
#                     square = inp[i:i+2, j:j+2]
#                     if np.sum(square > 0) == 3:  # 3 out of 4 filled
#                         # Find the empty spot and fill it
#                         for di in range(2):
#                             for dj in range(2):
#                                 if square[di,dj] == 0:
#                                     # Get the color from neighbors
#                                     color = square[1-di, dj] if square[1-di, dj] != 0 else square[di, 1-dj]
#                                     test[i+di, j+dj] = color
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found shape detection and completion")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[r[:]for r in g]
#  for i in range(h-1):
#   for j in range(w-1):
#    s=[g[i+x][j+y]for x in range(2)for y in range(2)]
#    if sum(1 for x in s if x>0)==3:
#     for di in range(2):
#      for dj in range(2):
#       if g[i+di][j+dj]==0:
#        c=g[i+1-di][j+dj]if g[i+1-di][j+dj]!=0 else g[i+di][j+1-dj]
#        res[i+di][j+dj]=c
#  return res
# """
#         debug_print(f"  ✗ Not a shape detection pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_shape_detection: {str(e)}")
#     return None

# @timed_handler
# def handle_directional_fill(task_data, analysis):
#     """Fill in specific directions until hitting boundary"""
#     debug_print(f"Trying handle_directional_fill")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             # Try filling right from non-zero values
#             test = np.copy(inp)
#             for i in range(h):
#                 for j in range(w):
#                     if inp[i,j] != 0:
#                         # Fill right
#                         for jj in range(j+1, w):
#                             if inp[i,jj] == 0:
#                                 test[i,jj] = inp[i,j]
#                             else:
#                                 break
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found directional fill (rightward)")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[r[:]for r in g]
#  for i in range(h):
#   for j in range(w):
#    if g[i][j]!=0:
#     for jj in range(j+1,w):
#      if g[i][jj]==0:
#       res[i][jj]=g[i][j]
#      else:
#       break
#  return res
# """
#         debug_print(f"  ✗ Not a directional fill pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_directional_fill: {str(e)}")
#     return None

# @timed_handler
# def handle_boundary_trace(task_data, analysis):
#     """Trace boundaries of objects"""
#     debug_print(f"Trying handle_boundary_trace")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.zeros_like(inp)
            
#             # Find boundaries and trace them
#             for i in range(h):
#                 for j in range(w):
#                     if inp[i,j] != 0:
#                         is_boundary = False
#                         # Check 8-connectivity
#                         for di in [-1, 0, 1]:
#                             for dj in [-1, 0, 1]:
#                                 if di == 0 and dj == 0:
#                                     continue
#                                 ni, nj = i + di, j + dj
#                                 if ni < 0 or ni >= h or nj < 0 or nj >= w or inp[ni,nj] == 0:
#                                     is_boundary = True
#                                     break
#                             if is_boundary:
#                                 break
                        
#                         if is_boundary:
#                             test[i,j] = inp[i,j]
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found boundary trace")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[[0]*w for _ in range(h)]
#  for i in range(h):
#   for j in range(w):
#    if g[i][j]!=0:
#     b=False
#     for di in[-1,0,1]:
#      for dj in[-1,0,1]:
#       if di==0 and dj==0:continue
#       ni,nj=i+di,j+dj
#       if ni<0 or ni>=h or nj<0 or nj>=w or g[ni][nj]==0:
#        b=True
#        break
#      if b:break
#     if b:
#      res[i][j]=g[i][j]
#  return res
# """
#         debug_print(f"  ✗ Not a boundary trace pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_boundary_trace: {str(e)}")
#     return None

# @timed_handler
# def handle_color_gradient_fill(task_data, analysis):
#     """Fill with color gradients between points"""
#     debug_print(f"Trying handle_color_gradient_fill")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.copy(inp)
            
#             # Find pairs of same color and gradient fill between them
#             colors = defaultdict(list)
#             for i in range(h):
#                 for j in range(w):
#                     if inp[i,j] != 0:
#                         colors[inp[i,j]].append((i,j))
            
#             for color, positions in colors.items():
#                 if len(positions) == 2:
#                     (y1, x1), (y2, x2) = positions
                    
#                     # Fill between them with gradient
#                     if y1 == y2:  # Horizontal line
#                         for x in range(min(x1, x2), max(x1, x2) + 1):
#                             dist_ratio = abs(x - x1) / max(abs(x2 - x1), 1)
#                             test[y1, x] = int(color * (1 - dist_ratio * 0.5)) % 10
#                     elif x1 == x2:  # Vertical line
#                         for y in range(min(y1, y2), max(y1, y2) + 1):
#                             dist_ratio = abs(y - y1) / max(abs(y2 - y1), 1)
#                             test[y, x1] = int(color * (1 - dist_ratio * 0.5)) % 10
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found color gradient fill")
#                 return """def p(g):
#  from collections import defaultdict
#  h,w=len(g),len(g[0])
#  res=[r[:]for r in g]
#  colors=defaultdict(list)
#  for i in range(h):
#   for j in range(w):
#    if g[i][j]!=0:
#     colors[g[i][j]].append((i,j))
#  for c,pos in colors.items():
#   if len(pos)==2:
#    (y1,x1),(y2,x2)=pos
#    if y1==y2:
#     for x in range(min(x1,x2),max(x1,x2)+1):
#      dr=abs(x-x1)/max(abs(x2-x1),1)
#      res[y1][x]=int(c*(1-dr*0.5))%10
#    elif x1==x2:
#     for y in range(min(y1,y2),max(y1,y2)+1):
#      dr=abs(y-y1)/max(abs(y2-y1),1)
#      res[y][x1]=int(c*(1-dr*0.5))%10
#  return res
# """
#         debug_print(f"  ✗ Not a color gradient fill pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_color_gradient_fill: {str(e)}")
#     return None

# @timed_handler
# def handle_pattern_interpolation(task_data, analysis):
#     """Interpolate patterns between key points"""
#     debug_print(f"Trying handle_pattern_interpolation")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.copy(inp)
            
#             # Simple linear interpolation between non-zero values
#             # Horizontal interpolation
#             for i in range(h):
#                 last_val = 0
#                 last_pos = -1
#                 for j in range(w):
#                     if inp[i,j] != 0:
#                         if last_val != 0 and last_pos >= 0:
#                             # Interpolate between last_pos and j
#                             for jj in range(last_pos + 1, j):
#                                 ratio = (jj - last_pos) / (j - last_pos)
#                                 test[i,jj] = int(last_val + ratio * (inp[i,j] - last_val)) % 10
#                         last_val = inp[i,j]
#                         last_pos = j
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found pattern interpolation")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[r[:]for r in g]
#  for i in range(h):
#   lv=0
#   lp=-1
#   for j in range(w):
#    if g[i][j]!=0:
#     if lv!=0 and lp>=0:
#      for jj in range(lp+1,j):
#       r=(jj-lp)/(j-lp)
#       res[i][jj]=int(lv+r*(g[i][j]-lv))%10
#     lv=g[i][j]
#     lp=j
#  return res
# """
#         debug_print(f"  ✗ Not a pattern interpolation")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_pattern_interpolation: {str(e)}")
#     return None

# @timed_handler
# def handle_region_growing(task_data, analysis):
#     """Grow regions from seed points"""
#     debug_print(f"Trying handle_region_growing")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.copy(inp)
            
#             # Grow each color region by one pixel
#             for color in range(1, 10):
#                 positions = np.argwhere(inp == color)
#                 for y, x in positions:
#                     for dy, dx in [(0,1), (1,0), (0,-1), (-1,0)]:
#                         ny, nx = y + dy, x + dx
#                         if 0 <= ny < h and 0 <= nx < w and inp[ny,nx] == 0:
#                             test[ny,nx] = color
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found region growing")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[r[:]for r in g]
#  for c in range(1,10):
#   pos=[(i,j)for i in range(h)for j in range(w)if g[i][j]==c]
#   for y,x in pos:
#    for dy,dx in[(0,1),(1,0),(0,-1),(-1,0)]:
#     ny,nx=y+dy,x+dx
#     if 0<=ny<h and 0<=nx<w and g[ny][nx]==0:
#      res[ny][nx]=c
#  return res
# """
#         debug_print(f"  ✗ Not a region growing pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_region_growing: {str(e)}")
#     return None

# @timed_handler
# def handle_symmetry_detection_fill(task_data, analysis):
#     """Detect partial symmetry and complete it"""
#     debug_print(f"Trying handle_symmetry_detection_fill")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             # Try vertical symmetry completion
#             test = np.copy(inp)
#             for i in range(h):
#                 for j in range(w//2):
#                     if inp[i,j] != 0 and inp[i,w-1-j] == 0:
#                         test[i,w-1-j] = inp[i,j]
#                     elif inp[i,j] == 0 and inp[i,w-1-j] != 0:
#                         test[i,j] = inp[i,w-1-j]
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found symmetry detection and completion")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[r[:]for r in g]
#  for i in range(h):
#   for j in range(w//2):
#    if g[i][j]!=0 and g[i][w-1-j]==0:
#     res[i][w-1-j]=g[i][j]
#    elif g[i][j]==0 and g[i][w-1-j]!=0:
#     res[i][j]=g[i][w-1-j]
#  return res
# """
#         debug_print(f"  ✗ Not a symmetry detection fill pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_symmetry_detection_fill: {str(e)}")
#     return None

# @timed_handler
# def handle_pixel_sorting(task_data, analysis):
#     """Sort pixels in each row or column"""
#     debug_print(f"Trying handle_pixel_sorting")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             # Try sorting each row
#             test = np.array([sorted(row) for row in inp])
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found row sorting")
#                 return """def p(g):
#  return[sorted(r)for r in g]
# """
            
#             # Try sorting each column
#             test = np.array([sorted(col) for col in inp.T]).T
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found column sorting")
#                 return """def p(g):
#  import numpy as np
#  return np.array([sorted(c)for c in np.array(g).T]).T.tolist()
# """
#         debug_print(f"  ✗ Not a pixel sorting pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_pixel_sorting: {str(e)}")
#     return None

# @timed_handler
# def handle_fractal_pattern(task_data, analysis):
#     """Generate fractal-like patterns"""
#     debug_print(f"Trying handle_fractal_pattern")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         h, w = out.shape
        
#         # Sierpinski-like pattern
#         test = np.zeros((h, w), dtype=int)
#         for i in range(h):
#             for j in range(w):
#                 if i & j == 0:
#                     test[i,j] = 1
        
#         if np.array_equal(test, out):
#             debug_print(f"  ✓ Found fractal pattern (Sierpinski-like)")
#             return f"""def p(g):
#  h,w={h},{w}
#  return[[1 if i&j==0 else 0 for j in range(w)]for i in range(h)]
# """
#         debug_print(f"  ✗ Not a fractal pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_fractal_pattern: {str(e)}")
#     return None

# @timed_handler
# def handle_position_encoding(task_data, analysis):
#     """Encode position information in output"""
#     debug_print(f"Trying handle_position_encoding")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         h, w = out.shape
        
#         # Try various position encodings
#         # Row + Column mod 10
#         test = np.zeros((h, w), dtype=int)
#         for i in range(h):
#             for j in range(w):
#                 test[i,j] = (i + j) % 10
        
#         if np.array_equal(test, out):
#             debug_print(f"  ✓ Found position encoding (i+j mod 10)")
#             return f"""def p(g):
#  h,w={h},{w}
#  return[[(i+j)%10 for j in range(w)]for i in range(h)]
# """
        
#         # Row * Column mod 10
#         test = np.zeros((h, w), dtype=int)
#         for i in range(h):
#             for j in range(w):
#                 test[i,j] = (i * j) % 10
        
#         if np.array_equal(test, out):
#             debug_print(f"  ✓ Found position encoding (i*j mod 10)")
#             return f"""def p(g):
#  h,w={h},{w}
#  return[[(i*j)%10 for j in range(w)]for i in range(h)]
# """
#         debug_print(f"  ✗ Not a position encoding pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_position_encoding: {str(e)}")
#     return None

# @timed_handler
# def handle_path_tracing(task_data, analysis):
#     """Trace paths between connected components"""
#     debug_print(f"Trying handle_path_tracing")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.copy(inp)
            
#             # Find all non-zero positions
#             positions = [(i,j) for i in range(h) for j in range(w) if inp[i,j] != 0]
            
#             # Try connecting adjacent same-color pixels
#             for i, (y1, x1) in enumerate(positions):
#                 for y2, x2 in positions[i+1:]:
#                     if inp[y1,x1] == inp[y2,x2]:
#                         # Connect if adjacent (including diagonally)
#                         if abs(y1-y2) <= 1 and abs(x1-x2) <= 1:
#                             # Draw line between them
#                             if y1 == y2:
#                                 for x in range(min(x1,x2), max(x1,x2)+1):
#                                     test[y1,x] = inp[y1,x1]
#                             elif x1 == x2:
#                                 for y in range(min(y1,y2), max(y1,y2)+1):
#                                     test[y,x1] = inp[y1,x1]
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found path tracing")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[r[:]for r in g]
#  pos=[(i,j)for i in range(h)for j in range(w)if g[i][j]!=0]
#  for i,(y1,x1)in enumerate(pos):
#   for y2,x2 in pos[i+1:]:
#    if g[y1][x1]==g[y2][x2]:
#     if abs(y1-y2)<=1 and abs(x1-x2)<=1:
#      if y1==y2:
#       for x in range(min(x1,x2),max(x1,x2)+1):
#        res[y1][x]=g[y1][x1]
#      elif x1==x2:
#       for y in range(min(y1,y2),max(y1,y2)+1):
#        res[y][x1]=g[y1][x1]
#  return res
# """
#         debug_print(f"  ✗ Not a path tracing pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_path_tracing: {str(e)}")
#     return None

# # NEW HANDLERS START HERE

# @timed_handler
# def handle_mask_arithmetic(task_data, analysis):
#     """Apply arithmetic operations with specific masks"""
#     debug_print(f"Trying handle_mask_arithmetic")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             # Try diagonal mask operations
#             test = np.copy(inp)
#             for i in range(h):
#                 for j in range(w):
#                     if i == j:  # Main diagonal
#                         test[i,j] = (inp[i,j] * 2) % 10
#                     elif i + j == h - 1:  # Anti-diagonal
#                         test[i,j] = (inp[i,j] + 3) % 10
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found mask arithmetic (diagonal *2, anti-diagonal +3)")
#                 return """def p(g):
#  h=len(g)
#  res=[r[:]for r in g]
#  for i in range(h):
#   for j in range(len(g[0])):
#    if i==j:
#     res[i][j]=(g[i][j]*2)%10
#    elif i+j==h-1:
#     res[i][j]=(g[i][j]+3)%10
#  return res
# """
#         debug_print(f"  ✗ Not a mask arithmetic pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_mask_arithmetic: {str(e)}")
#     return None

# @timed_handler
# def handle_wraparound_shift(task_data, analysis):
#     """Shift with wraparound at edges"""
#     debug_print(f"Trying handle_wraparound_shift")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             # Try circular shift right
#             for shift in range(1, w):
#                 test = np.zeros_like(inp)
#                 for i in range(h):
#                     for j in range(w):
#                         test[i,(j+shift)%w] = inp[i,j]
                
#                 if np.array_equal(test, out):
#                     debug_print(f"  ✓ Found wraparound shift (right by {shift})")
#                     return f"""def p(g):
#  h,w=len(g),len(g[0])
#  res=[[0]*w for _ in range(h)]
#  for i in range(h):
#   for j in range(w):
#    res[i][(j+{shift})%w]=g[i][j]
#  return res
# """
#         debug_print(f"  ✗ Not a wraparound shift pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_wraparound_shift: {str(e)}")
#     return None

# @timed_handler
# def handle_1d_interleave(task_data, analysis):
#     """Interleave elements in 1D representation"""
#     debug_print(f"Trying handle_1d_interleave")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         arr_in = to_1d_raster(inp.tolist(), 'row_major')
#         arr_out = to_1d_raster(out.tolist(), 'row_major')
        
#         # Try interleaving first half and second half
#         if len(arr_in) % 2 == 0:
#             half = len(arr_in) // 2
#             interleaved = []
#             for i in range(half):
#                 interleaved.append(arr_in[i])
#                 interleaved.append(arr_in[half + i])
            
#             if interleaved == arr_out:
#                 h, w = out.shape
#                 debug_print(f"  ✓ Found 1D interleave pattern")
#                 return f"""def p(g):
#  arr=[g[i][j]for i in range(len(g))for j in range(len(g[0]))]
#  half=len(arr)//2
#  res=[]
#  for i in range(half):
#   res.append(arr[i])
#   res.append(arr[half+i])
#  h,w={h},{w}
#  out=[[0]*w for _ in range(h)]
#  for i,v in enumerate(res):
#   out[i//w][i%w]=v
#  return out
# """
#         debug_print(f"  ✗ Not a 1D interleave pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_1d_interleave: {str(e)}")
#     return None

# @timed_handler
# def handle_diagonal_stripes(task_data, analysis):
#     """Create diagonal stripe patterns"""
#     debug_print(f"Trying handle_diagonal_stripes")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         h, w = out.shape
        
#         # Try diagonal stripes
#         for stripe_width in range(1, 5):
#             for c1 in range(10):
#                 for c2 in range(10):
#                     if c1 != c2:
#                         test = np.zeros((h, w), dtype=int)
#                         for i in range(h):
#                             for j in range(w):
#                                 if ((i + j) // stripe_width) % 2 == 0:
#                                     test[i,j] = c1
#                                 else:
#                                     test[i,j] = c2
                        
#                         if np.array_equal(test, out):
#                             debug_print(f"  ✓ Found diagonal stripes (width={stripe_width}, colors={c1},{c2})")
#                             return f"""def p(g):
#  h,w={h},{w}
#  res=[]
#  for i in range(h):
#   row=[]
#   for j in range(w):
#    if((i+j)//{stripe_width})%2==0:
#     row.append({c1})
#    else:
#     row.append({c2})
#   res.append(row)
#  return res
# """
#         debug_print(f"  ✗ Not a diagonal stripes pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_diagonal_stripes: {str(e)}")
#     return None

# @timed_handler
# def handle_cellular_rule(task_data, analysis):
#     """Apply custom cellular automaton rules"""
#     debug_print(f"Trying handle_cellular_rule")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.zeros_like(inp)
            
#             # Rule: cell becomes sum of all 8 neighbors mod 10
#             for i in range(h):
#                 for j in range(w):
#                     total = 0
#                     for di in [-1, 0, 1]:
#                         for dj in [-1, 0, 1]:
#                             if di == 0 and dj == 0:
#                                 continue
#                             ni, nj = (i + di) % h, (j + dj) % w  # Wraparound
#                             total += inp[ni,nj]
#                     test[i,j] = total % 10
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found cellular rule (sum of 8 neighbors mod 10)")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[[0]*w for _ in range(h)]
#  for i in range(h):
#   for j in range(w):
#    t=0
#    for di in[-1,0,1]:
#     for dj in[-1,0,1]:
#      if di==0 and dj==0:continue
#      ni,nj=(i+di)%h,(j+dj)%w
#      t+=g[ni][nj]
#    res[i][j]=t%10
#  return res
# """
#         debug_print(f"  ✗ Not a cellular rule pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_cellular_rule: {str(e)}")
#     return None

# @timed_handler
# def handle_prime_mask(task_data, analysis):
#     """Apply operations based on prime number positions"""
#     debug_print(f"Trying handle_prime_mask")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             def is_prime(n):
#                 if n < 2: return False
#                 for i in range(2, int(n**0.5) + 1):
#                     if n % i == 0: return False
#                 return True
            
#             h, w = inp.shape
#             test = np.copy(inp)
            
#             for i in range(h):
#                 for j in range(w):
#                     pos = i * w + j
#                     if is_prime(pos):
#                         test[i,j] = (inp[i,j] + 1) % 10
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found prime mask pattern")
#                 return """def p(g):
#  def ip(n):
#   if n<2:return False
#   for i in range(2,int(n**0.5)+1):
#    if n%i==0:return False
#   return True
#  h,w=len(g),len(g[0])
#  res=[r[:]for r in g]
#  for i in range(h):
#   for j in range(w):
#    if ip(i*w+j):
#     res[i][j]=(g[i][j]+1)%10
#  return res
# """
#         debug_print(f"  ✗ Not a prime mask pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_prime_mask: {str(e)}")
#     return None

# @timed_handler
# def handle_bit_pattern_mask(task_data, analysis):
#     """Apply operations based on bit patterns"""
#     debug_print(f"Trying handle_bit_pattern_mask")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.copy(inp)
            
#             # Apply operation if position has odd number of 1s in binary
#             for i in range(h):
#                 for j in range(w):
#                     pos = i * w + j
#                     if bin(pos).count('1') % 2 == 1:
#                         test[i,j] = (inp[i,j] * 3) % 10
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found bit pattern mask")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[r[:]for r in g]
#  for i in range(h):
#   for j in range(w):
#    if bin(i*w+j).count('1')%2==1:
#     res[i][j]=(g[i][j]*3)%10
#  return res
# """
#         debug_print(f"  ✗ Not a bit pattern mask")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_bit_pattern_mask: {str(e)}")
#     return None

# @timed_handler
# def handle_symmetric_operations(task_data, analysis):
#     """Apply operations that maintain symmetry"""
#     debug_print(f"Trying handle_symmetric_operations")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.copy(inp)
            
#             # Apply operation to maintain vertical symmetry
#             for i in range(h):
#                 for j in range(w//2):
#                     val = (inp[i,j] + inp[i,w-1-j]) % 10
#                     test[i,j] = val
#                     test[i,w-1-j] = val
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found symmetric operations")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[r[:]for r in g]
#  for i in range(h):
#   for j in range(w//2):
#    v=(g[i][j]+g[i][w-1-j])%10
#    res[i][j]=v
#    res[i][w-1-j]=v
#  return res
# """
#         debug_print(f"  ✗ Not a symmetric operations pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_symmetric_operations: {str(e)}")
#     return None

# @timed_handler
# def handle_distance_transform(task_data, analysis):
#     """Transform based on distance from edges"""
#     debug_print(f"Trying handle_distance_transform")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.zeros_like(inp)
            
#             for i in range(h):
#                 for j in range(w):
#                     edge_dist = min(i, j, h-1-i, w-1-j)
#                     test[i,j] = (inp[i,j] + edge_dist * 2) % 10
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found distance transform")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[]
#  for i in range(h):
#   row=[]
#   for j in range(w):
#    ed=min(i,j,h-1-i,w-1-j)
#    row.append((g[i][j]+ed*2)%10)
#   res.append(row)
#  return res
# """
#         debug_print(f"  ✗ Not a distance transform pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_distance_transform: {str(e)}")
#     return None

# @timed_handler
# def handle_modular_grid(task_data, analysis):
#     """Apply modular arithmetic based on grid position"""
#     debug_print(f"Trying handle_modular_grid")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             for mod_h in range(2, 5):
#                 for mod_w in range(2, 5):
#                     test = np.copy(inp)
#                     for i in range(h):
#                         for j in range(w):
#                             factor = (i % mod_h) * mod_w + (j % mod_w)
#                             test[i,j] = (inp[i,j] + factor) % 10
                    
#                     if np.array_equal(test, out):
#                         debug_print(f"  ✓ Found modular grid (mod_h={mod_h}, mod_w={mod_w})")
#                         return f"""def p(g):
#  h,w=len(g),len(g[0])
#  res=[]
#  for i in range(h):
#   row=[]
#   for j in range(w):
#    f=(i%{mod_h})*{mod_w}+(j%{mod_w})
#    row.append((g[i][j]+f)%10)
#   res.append(row)
#  return res
# """
#         debug_print(f"  ✗ Not a modular grid pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_modular_grid: {str(e)}")
#     return None

# @timed_handler
# def handle_pattern_mask_multiply(task_data, analysis):
#     """Multiply by mask pattern"""
#     debug_print(f"Trying handle_pattern_mask_multiply")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             # Try multiplication by position-based mask
#             test = np.zeros_like(inp)
#             for i in range(h):
#                 for j in range(w):
#                     mask_val = 1 + (i + j) % 3
#                     test[i,j] = (inp[i,j] * mask_val) % 10
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found pattern mask multiply")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[]
#  for i in range(h):
#   row=[]
#   for j in range(w):
#    m=1+(i+j)%3
#    row.append((g[i][j]*m)%10)
#   res.append(row)
#  return res
# """
#         debug_print(f"  ✗ Not a pattern mask multiply")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_pattern_mask_multiply: {str(e)}")
#     return None

# @timed_handler
# def handle_recursive_fill(task_data, analysis):
#     """Recursively fill based on neighbors"""
#     debug_print(f"Trying handle_recursive_fill")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.copy(inp)
            
#             # Fill zeros with average of non-zero neighbors
#             changed = True
#             while changed:
#                 changed = False
#                 new_test = np.copy(test)
#                 for i in range(h):
#                     for j in range(w):
#                         if test[i,j] == 0:
#                             neighbors = []
#                             for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
#                                 ni, nj = i + di, j + dj
#                                 if 0 <= ni < h and 0 <= nj < w and test[ni,nj] != 0:
#                                     neighbors.append(test[ni,nj])
#                             if neighbors:
#                                 new_test[i,j] = sum(neighbors) // len(neighbors)
#                                 changed = True
#                 test = new_test
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found recursive fill")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[r[:]for r in g]
#  ch=True
#  while ch:
#   ch=False
#   nr=[r[:]for r in res]
#   for i in range(h):
#    for j in range(w):
#     if res[i][j]==0:
#      n=[]
#      for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
#       ni,nj=i+di,j+dj
#       if 0<=ni<h and 0<=nj<w and res[ni][nj]!=0:
#        n.append(res[ni][nj])
#      if n:
#       nr[i][j]=sum(n)//len(n)
#       ch=True
#   res=nr
#  return res
# """
#         debug_print(f"  ✗ Not a recursive fill pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_recursive_fill: {str(e)}")
#     return None

# @timed_handler
# def handle_gradient_mask(task_data, analysis):
#     """Apply gradient-based masks"""
#     debug_print(f"Trying handle_gradient_mask")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         h, w = out.shape
        
#         # Linear gradient from top-left to bottom-right
#         test = np.zeros((h, w), dtype=int)
#         for i in range(h):
#             for j in range(w):
#                 grad = (i + j) * 9 // (h + w - 2) if h + w > 2 else 0
#                 test[i,j] = grad
        
#         if np.array_equal(test, out):
#             debug_print(f"  ✓ Found gradient mask")
#             return f"""def p(g):
#  h,w={h},{w}
#  res=[]
#  for i in range(h):
#   row=[]
#   for j in range(w):
#    gr=(i+j)*9//(h+w-2)if h+w>2 else 0
#    row.append(gr)
#   res.append(row)
#  return res
# """
#         debug_print(f"  ✗ Not a gradient mask pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_gradient_mask: {str(e)}")
#     return None

# @timed_handler
# def handle_phase_shift_pattern(task_data, analysis):
#     """Apply phase-shifted periodic patterns"""
#     debug_print(f"Trying handle_phase_shift_pattern")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             # Try phase shift based on row
#             test = np.zeros_like(inp)
#             for i in range(h):
#                 for j in range(w):
#                     phase = i % 3
#                     test[i,j] = (inp[i,j] + j + phase) % 10
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found phase shift pattern")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[]
#  for i in range(h):
#   row=[]
#   for j in range(w):
#    ph=i%3
#    row.append((g[i][j]+j+ph)%10)
#   res.append(row)
#  return res
# """
#         debug_print(f"  ✗ Not a phase shift pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_phase_shift_pattern: {str(e)}")
#     return None

# @timed_handler
# def handle_overflow_mask(task_data, analysis):
#     """Apply masks that overflow grid boundaries"""
#     debug_print(f"Trying handle_overflow_mask")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.zeros_like(inp)
            
#             # Virtual larger grid mask
#             for i in range(h):
#                 for j in range(w):
#                     # Treat as if part of larger grid
#                     virtual_i = i + h
#                     virtual_j = j + w
#                     mask_val = (virtual_i * virtual_j) % 10
#                     test[i,j] = (inp[i,j] + mask_val) % 10
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found overflow mask")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[]
#  for i in range(h):
#   row=[]
#   for j in range(w):
#    vi=i+h
#    vj=j+w
#    m=(vi*vj)%10
#    row.append((g[i][j]+m)%10)
#   res.append(row)
#  return res
# """
#         debug_print(f"  ✗ Not an overflow mask pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_overflow_mask: {str(e)}")
#     return None

# @timed_handler
# def handle_1d_block_swap(task_data, analysis):
#     """Swap blocks in 1D representation"""
#     debug_print(f"Trying handle_1d_block_swap")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         arr_in = to_1d_raster(inp.tolist(), 'row_major')
#         arr_out = to_1d_raster(out.tolist(), 'row_major')
        
#         # Try swapping blocks of different sizes
#         for block_size in range(2, len(arr_in)//2 + 1):
#             if len(arr_in) % block_size == 0:
#                 blocks = [arr_in[i:i+block_size] for i in range(0, len(arr_in), block_size)]
#                 # Try reversing blocks
#                 reversed_blocks = blocks[::-1]
#                 test = []
#                 for block in reversed_blocks:
#                     test.extend(block)
                
#                 if test == arr_out:
#                     h, w = out.shape
#                     debug_print(f"  ✓ Found 1D block swap (block_size={block_size})")
#                     return f"""def p(g):
#  arr=[g[i][j]for i in range(len(g))for j in range(len(g[0]))]
#  bs={block_size}
#  blocks=[arr[i:i+bs]for i in range(0,len(arr),bs)]
#  rb=blocks[::-1]
#  res=[]
#  for b in rb:
#   res.extend(b)
#  h,w={h},{w}
#  out=[[0]*w for _ in range(h)]
#  for i,v in enumerate(res):
#   out[i//w][i%w]=v
#  return out
# """
#         debug_print(f"  ✗ Not a 1D block swap pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_1d_block_swap: {str(e)}")
#     return None

# @timed_handler
# def handle_number_sequence_fill(task_data, analysis):
#     """Fill with various number sequences"""
#     debug_print(f"Trying handle_number_sequence_fill")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         h, w = out.shape
        
#         # Fibonacci mod 10
#         test = np.zeros((h, w), dtype=int)
#         a, b = 0, 1
#         for i in range(h):
#             for j in range(w):
#                 test[i,j] = a % 10
#                 a, b = b, (a + b)
        
#         if np.array_equal(test, out):
#             debug_print(f"  ✓ Found Fibonacci sequence fill")
#             return f"""def p(g):
#  h,w={h},{w}
#  res=[]
#  a,b=0,1
#  for i in range(h):
#   row=[]
#   for j in range(w):
#    row.append(a%10)
#    a,b=b,a+b
#   res.append(row)
#  return res
# """
#         debug_print(f"  ✗ Not a number sequence fill pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_number_sequence_fill: {str(e)}")
#     return None

# @timed_handler
# def handle_complex_bit_ops(task_data, analysis):
#     """Complex bit manipulation patterns"""
#     debug_print(f"Trying handle_complex_bit_ops")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.zeros_like(inp)
            
#             # XOR with position and rotate bits
#             for i in range(h):
#                 for j in range(w):
#                     pos = (i * w + j) % 8
#                     val = inp[i,j]
#                     # Rotate bits left by position
#                     rotated = ((val << pos) | (val >> (8 - pos))) & 0xFF
#                     test[i,j] = rotated % 10
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found complex bit operations")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[]
#  for i in range(h):
#   row=[]
#   for j in range(w):
#    pos=(i*w+j)%8
#    v=g[i][j]
#    r=((v<<pos)|(v>>(8-pos)))&0xFF
#    row.append(r%10)
#   res.append(row)
#  return res
# """
#         debug_print(f"  ✗ Not a complex bit ops pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_complex_bit_ops: {str(e)}")
#     return None

# @timed_handler
# def handle_transformation_chain(task_data, analysis):
#     """Chain multiple transformations"""
#     debug_print(f"Trying handle_transformation_chain")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
            
#             # Try: add position, then multiply by 2, then mod 10
#             test = np.zeros_like(inp)
#             for i in range(h):
#                 for j in range(w):
#                     step1 = inp[i,j] + i + j
#                     step2 = step1 * 2
#                     test[i,j] = step2 % 10
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found transformation chain")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[]
#  for i in range(h):
#   row=[]
#   for j in range(w):
#    s1=g[i][j]+i+j
#    s2=s1*2
#    row.append(s2%10)
#   res.append(row)
#  return res
# """
#         debug_print(f"  ✗ Not a transformation chain pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_transformation_chain: {str(e)}")
#     return None

# @timed_handler
# def handle_neighbor_product(task_data, analysis):
#     """Product of neighbors"""
#     debug_print(f"Trying handle_neighbor_product")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.zeros_like(inp)
            
#             for i in range(h):
#                 for j in range(w):
#                     product = 1
#                     count = 0
#                     for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
#                         ni, nj = i + di, j + dj
#                         if 0 <= ni < h and 0 <= nj < w:
#                             if inp[ni,nj] != 0:
#                                 product *= inp[ni,nj]
#                                 count += 1
#                     if count > 0:
#                         test[i,j] = product % 10
#                     else:
#                         test[i,j] = inp[i,j]
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found neighbor product")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[[0]*w for _ in range(h)]
#  for i in range(h):
#   for j in range(w):
#    pr=1
#    c=0
#    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
#     ni,nj=i+di,j+dj
#     if 0<=ni<h and 0<=nj<w:
#      if g[ni][nj]!=0:
#       pr*=g[ni][nj]
#       c+=1
#    if c>0:
#     res[i][j]=pr%10
#    else:
#     res[i][j]=g[i][j]
#  return res
# """
#         debug_print(f"  ✗ Not a neighbor product pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_neighbor_product: {str(e)}")
#     return None

# @timed_handler
# def handle_1d_segment_reverse(task_data, analysis):
#     """Reverse specific segments in 1D"""
#     debug_print(f"Trying handle_1d_segment_reverse")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         arr_in = to_1d_raster(inp.tolist(), 'row_major')
#         arr_out = to_1d_raster(out.tolist(), 'row_major')
        
#         # Try reversing segments between zeros
#         segments = []
#         current = []
#         for val in arr_in:
#             if val == 0:
#                 if current:
#                     segments.append(current[::-1])
#                     current = []
#                 segments.append([0])
#             else:
#                 current.append(val)
#         if current:
#             segments.append(current[::-1])
        
#         test = []
#         for seg in segments:
#             test.extend(seg)
        
#         if test == arr_out:
#             h, w = out.shape
#             debug_print(f"  ✓ Found 1D segment reverse")
#             return f"""def p(g):
#  arr=[g[i][j]for i in range(len(g))for j in range(len(g[0]))]
#  segs=[]
#  cur=[]
#  for v in arr:
#   if v==0:
#    if cur:
#     segs.append(cur[::-1])
#     cur=[]
#    segs.append([0])
#   else:
#    cur.append(v)
#  if cur:
#   segs.append(cur[::-1])
#  res=[]
#  for s in segs:
#   res.extend(s)
#  h,w={h},{w}
#  out=[[0]*w for _ in range(h)]
#  for i,v in enumerate(res):
#   out[i//w][i%w]=v
#  return out
# """
#         debug_print(f"  ✗ Not a 1D segment reverse pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_1d_segment_reverse: {str(e)}")
#     return None

# @timed_handler
# def handle_symmetric_mask_ops(task_data, analysis):
#     """Operations that create symmetric patterns"""
#     debug_print(f"Trying handle_symmetric_mask_ops")
#     try:
#         inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
#         if inp.shape == out.shape:
#             h, w = inp.shape
#             test = np.zeros_like(inp)
            
#             # Create symmetric pattern from input
#             for i in range(h):
#                 for j in range(w):
#                     # Take max of symmetric positions
#                     sym_vals = [inp[i,j]]
#                     if i < h-1-i:
#                         sym_vals.append(inp[h-1-i,j])
#                     if j < w-1-j:
#                         sym_vals.append(inp[i,w-1-j])
#                     if i < h-1-i and j < w-1-j:
#                         sym_vals.append(inp[h-1-i,w-1-j])
#                     test[i,j] = max(sym_vals)
            
#             if np.array_equal(test, out):
#                 debug_print(f"  ✓ Found symmetric mask operations")
#                 return """def p(g):
#  h,w=len(g),len(g[0])
#  res=[[0]*w for _ in range(h)]
#  for i in range(h):
#   for j in range(w):
#    sv=[g[i][j]]
#    if i<h-1-i:
#     sv.append(g[h-1-i][j])
#    if j<w-1-j:
#     sv.append(g[i][w-1-j])
#    if i<h-1-i and j<w-1-j:
#     sv.append(g[h-1-i][w-1-j])
#    res[i][j]=max(sv)
#  return res
# """
#         debug_print(f"  ✗ Not a symmetric mask operations pattern")
#     except Exception as e:
#         debug_print(f"  !! Exception in handle_symmetric_mask_ops: {str(e)}")
#     return None

# def analyze_task(task_data):
#     """Enhanced task analysis with pattern detection"""
#     all_examples = task_data['train'] + task_data['test']
#     analysis = {
#         'color_changes': defaultdict(set),
#         'shape_changes': set(),
#         'symmetry': None,
#         'arithmetic': None,
#         'pattern_types': set(),
#         'color_count': set(),
#         'has_objects': False,
#         'has_lines': False,
#         'has_repeating_patterns': False,
#     }
    
#     for example in all_examples:
#         in_grid = np.array(example['input'])
#         out_grid = np.array(example['output'])
        
#         analysis['shape_changes'].add((in_grid.shape, out_grid.shape))
#         analysis['color_count'].add(len(np.unique(in_grid)))
        
#         # Check for objects
#         if len(np.unique(in_grid)) > 2:
#             analysis['has_objects'] = True
        
#         # Check for lines
#         for i in range(in_grid.shape[0]):
#             if len(np.unique(in_grid[i,:])) == 1 and in_grid[i,0] != 0:
#                 analysis['has_lines'] = True
        
#         for (i,j), val in np.ndenumerate(in_grid):
#             if i < out_grid.shape[0] and j < out_grid.shape[1]:
#                 if in_grid[i,j] != out_grid[i,j]:
#                     analysis['color_changes'][(i,j)].add((int(in_grid[i,j]), int(out_grid[i,j])))
    
#     return analysis

# def verify_solution(solution_code, task_data):
#     """Verify solution against all training examples"""
#     try:
#         # Create a local namespace for execution
#         local_namespace = {}
#         exec(solution_code, globals(), local_namespace)
        
#         if 'p' not in local_namespace:
#             return False
        
#         p = local_namespace['p']
        
#         # Test against all training examples
#         for idx, example in enumerate(task_data['train']):
#             input_grid = example['input']
#             expected = example['output']
#             try:
#                 actual = p(input_grid)
#                 if actual != expected:
#                     debug_print(f"  Failed on training example {idx}", "WARNING")
#                     return False
#             except Exception as e:
#                 debug_print(f"  Exception on training example {idx}: {str(e)}", "ERROR")
#                 return False
#         return True
#     except Exception as e:
#         debug_print(f"  Verification failed: {str(e)}", "ERROR")
#         return False

# # Pattern combination functions
# def try_sequential_combinations(task_data, pattern_handlers, analysis):
#     """Try applying two patterns in sequence"""
#     debug_print("=== Trying sequential pattern combinations ===", "INFO")
    
#     # Get candidate handlers that might work together
#     candidate_handlers = []
    
#     # Quick test each handler to see if it produces valid intermediate results
#     for handler in pattern_handlers[:30]:  # Limit to avoid explosion
#         try:
#             solution = handler(task_data, analysis)
#             if solution and solution != """def p(g):\n return g\n""":
#                 candidate_handlers.append((handler, solution))
#         except:
#             pass
    
#     debug_print(f"Found {len(candidate_handlers)} candidate handlers for combination", "INFO")
    
#     # Try combining pairs
#     for i, (handler1, sol1) in enumerate(candidate_handlers[:10]):
#         for j, (handler2, sol2) in enumerate(candidate_handlers[:10]):
#             if i == j:
#                 continue
            
#             debug_print(f"  Trying {handler1.__name__} → {handler2.__name__}", "DEBUG")
            
#             # Create combined transformation
#             combined_solution = create_sequential_combination(sol1, sol2, handler1.__name__, handler2.__name__)
            
#             if verify_solution(combined_solution, task_data):
#                 debug_print(f"✓ Found sequential combination: {handler1.__name__} → {handler2.__name__}", "SUCCESS")
#                 return combined_solution
    
#     return None

# def create_sequential_combination(sol1, sol2, name1, name2):
#     """Create a solution that applies sol1 then sol2"""
#     body1 = extract_function_body(sol1)
#     body2 = extract_function_body(sol2)
    
#     return f"""def p(g):
#  # Step 1: {name1}
#  def t1(g):
# {indent_code(body1, 2)}
 
#  # Step 2: {name2}
#  def t2(g):
# {indent_code(body2, 2)}
 
#  # Apply both transformations
#  intermediate = t1(g)
#  return t2(intermediate)
# """

# def try_conditional_combinations(task_data, pattern_handlers, analysis):
#     """Try applying patterns based on conditions"""
#     debug_print("=== Trying conditional pattern combinations ===", "INFO")
    
#     # Get shape info
#     inp_shape = list(analysis['shape_changes'])[0][0]
    
#     # Try color-based conditions
#     if analysis['color_count'] and max(analysis['color_count']) > 2:
#         debug_print("  Trying color-based conditional patterns", "DEBUG")
        
#         # Find handlers that work on specific colors
#         color_handlers = []
#         for handler in pattern_handlers[:20]:
#             if 'color' in handler.__name__.lower():
#                 try:
#                     solution = handler(task_data, analysis)
#                     if solution:
#                         color_handlers.append((handler, solution))
#                 except:
#                     pass
        
#         # Try region-based handlers
#         region_handlers = []
#         for handler in pattern_handlers[:20]:
#             if any(word in handler.__name__.lower() for word in ['region', 'quadrant', 'ring']):
#                 try:
#                     solution = handler(task_data, analysis)
#                     if solution:
#                         region_handlers.append((handler, solution))
#                 except:
#                     pass
        
#         # Combine color and region handlers
#         for (ch, cs) in color_handlers[:3]:
#             for (rh, rs) in region_handlers[:3]:
#                 combined = create_conditional_combination(cs, rs, ch.__name__, rh.__name__, inp_shape)
#                 if verify_solution(combined, task_data):
#                     debug_print(f"✓ Found conditional combination: {ch.__name__} + {rh.__name__}", "SUCCESS")
#                     return combined
    
#     return None

# def create_conditional_combination(sol1, sol2, name1, name2, shape):
#     """Create a solution that applies different patterns to different regions"""
#     body1 = extract_function_body(sol1)
#     body2 = extract_function_body(sol2)
#     h, w = shape
    
#     return f"""def p(g):
#  h,w={h},{w}
#  res=[r[:]for r in g]
 
#  # Apply {name1} to top half
#  def t1(g):
# {indent_code(body1, 2)}
 
#  # Apply {name2} to bottom half
#  def t2(g):
# {indent_code(body2, 2)}
 
#  # Split application
#  top_half = [r[:] for r in g[:h//2]]
#  bottom_half = [r[:] for r in g[h//2:]]
 
#  top_result = t1(top_half)
#  bottom_result = t2(bottom_half)
 
#  return top_result + bottom_result
# """

# def generate_solution(task_data):
#     """Enhanced solution generation with pattern combination"""
#     global DEBUG_LOG, HANDLER_CACHE
#     DEBUG_LOG = []  # Clear log for each task
    
#     # Check cache
#     key = cache_key(task_data)
#     if key in HANDLER_CACHE:
#         debug_print("Found cached solution", "INFO")
#         return HANDLER_CACHE[key]
    
#     analysis = analyze_task(task_data)
    
#     debug_print(f"\n{'='*60}", "INFO")
#     debug_print(f"Processing task with shape changes: {analysis['shape_changes']}", "INFO")
#     debug_print(f"Color count: {analysis['color_count']}", "INFO")
#     debug_print(f"Has objects: {analysis['has_objects']}", "INFO")
#     debug_print(f"Has lines: {analysis['has_lines']}", "INFO")
    
#     pattern_handlers = [
#         handle_empty_input,
#         handle_identity,
#         handle_1d_raster_transform,
#         handle_1d_pattern_repeat,
#         handle_1d_sort_colors,
#         handle_1d_run_length_decode,
#         handle_1d_reverse_segments,
#         handle_1d_shuffle_pattern,
#         handle_1d_wave_transform,
#         handle_1d_compression,
#         handle_1d_periodic_transform,
#         handle_1d_fibonacci_transform,
#         handle_1d_prime_positions,
#         handle_1d_alternating_ops,
#         handle_diagonal_linearization,
#         handle_hilbert_curve,
#         handle_morton_order,
#         handle_block_linearization,
#         handle_snake_pattern,
#         handle_radial_linearization,
#         handle_odd_even_rows,
#         handle_odd_even_cols,
#         handle_checkerboard_transform,
#         handle_parity_coloring,
#         handle_alternating_blocks,
#         handle_modular_arithmetic,
#         handle_xor_patterns,
#         handle_binary_operations,
#         handle_bit_manipulation,
#         handle_arithmetic_sequences,
#         handle_geometric_sequences,
#         handle_template_matching,
#         handle_convolution_patterns,
#         handle_morphological_ops,
#         handle_edge_detection_advanced,
#         handle_corner_detection,
#         handle_pattern_completion_advanced,
#         handle_graph_coloring,
#         handle_tree_traversal,
#         handle_component_labeling,
#         handle_minimum_spanning_tree,
#         handle_graph_isomorphism,
#         handle_color_gradients,
#         handle_color_mixing,
#         handle_palette_reduction,
#         handle_color_quantization,
#         handle_color_spreading,
#         handle_color_waves,
#         handle_rotate_90,
#         handle_rotate_180,
#         handle_rotate_270,
#         handle_flip_horizontal,
#         handle_flip_vertical,
#         handle_transpose,
#         handle_rotate_45,
#         handle_shear_transform,
#         handle_perspective_transform,
#         handle_upscale,
#         handle_downscale,
#         handle_pad_to_square,
#         handle_crop_to_content,
#         handle_zoom_center,
#         handle_aspect_ratio_change,
#         handle_repeat_pattern,
#         handle_tile_pattern,
#         handle_mirror_pattern,
#         handle_extract_pattern,
#         handle_apply_mask,
#         handle_pattern_substitution,
#         handle_count_objects,
#         handle_largest_object,
#         handle_move_objects,
#         handle_gravity,
#         handle_connect_same_color,
#         handle_outline_objects,
#         handle_fill_objects,
#         handle_object_intersection,
#         handle_draw_lines,
#         handle_detect_lines,
#         handle_extend_lines,
#         handle_line_intersection,
#         handle_perpendicular_lines,
#         handle_make_symmetric,
#         handle_complete_pattern,
#         handle_find_symmetry_axis,
#         handle_rotational_symmetry,
#         handle_conway_step,
#         handle_cellular_automaton,
#         handle_maze_solve,
#         handle_path_finding,
#         handle_voronoi_regions,
#         handle_fractal_generation,
#         handle_multi_step_transform,
#         handle_conditional_transform,
#         handle_recursive_transform,
#         handle_learned_transform,
#         handle_color_count,
#         handle_color_frequency_sort,
#         handle_replace_most_common,
#         handle_color_propagation,
#         handle_flood_fill,
#         handle_split_grid,
#         handle_merge_grids,
#         handle_overlay_grids,
#         handle_subtract_grids,
#         handle_color_mapping,
#         # Add all the new handlers
#         handle_diagonal_mirror,
#         handle_color_inversion,
#         handle_distance_coloring,
#         handle_neighbor_sum_mod,
#         handle_wave_propagation,
#         handle_quadrant_operations,
#         handle_ring_operations,
#         handle_pattern_frequency,
#         handle_sliding_window_ops,
#         handle_cross_product_transform,
#         handle_spiral_fill,
#         handle_border_operations,
#         handle_corner_operations,
#         handle_chess_pattern,
#         handle_diamond_pattern,
#         handle_triangular_numbers,
#         handle_color_cycle,
#         handle_connectivity_color,
#         handle_layered_transform,
#         handle_local_symmetry,
#         handle_shape_detection,
#         handle_directional_fill,
#         handle_boundary_trace,
#         handle_color_gradient_fill,
#         handle_pattern_interpolation,
#         handle_region_growing,
#         handle_symmetry_detection_fill,
#         handle_pixel_sorting,
#         handle_fractal_pattern,
#         handle_position_encoding,
#         handle_path_tracing,
#         # Add all the new creative handlers
#         handle_mask_arithmetic,
#         handle_wraparound_shift,
#         handle_1d_interleave,
#         handle_diagonal_stripes,
#         handle_cellular_rule,
#         handle_prime_mask,
#         handle_bit_pattern_mask,
#         handle_symmetric_operations,
#         handle_distance_transform,
#         handle_modular_grid,
#         handle_pattern_mask_multiply,
#         handle_recursive_fill,
#         handle_gradient_mask,
#         handle_phase_shift_pattern,
#         handle_overflow_mask,
#         handle_1d_block_swap,
#         handle_number_sequence_fill,
#         handle_complex_bit_ops,
#         handle_transformation_chain,
#         handle_neighbor_product,
#         handle_1d_segment_reverse,
#         handle_symmetric_mask_ops,
#     ]
    
#     debug_print(f"Total handlers available: {len(pattern_handlers)}", "INFO")
    
#     # PHASE 1: Try individual handlers
#     debug_print("\n=== PHASE 1: Trying individual handlers ===", "INFO")
    
#     handlers_tried = 0
#     handlers_with_solutions = 0
#     handlers_with_exceptions = 0
    
#     for idx, handler in enumerate(pattern_handlers):
#         handlers_tried += 1
#         solution = handler(task_data, analysis)
        
#         if solution:
#             handlers_with_solutions += 1
#             debug_print(f"  → Handler #{idx+1} ({handler.__name__}) returned a solution", "DEBUG")
#             if verify_solution(solution, task_data):
#                 debug_print(f"  ✓✓ SOLUTION VERIFIED! Found by handler #{idx+1} ({handler.__name__})", "SUCCESS")
#                 debug_print(f"  Stats: Tried {handlers_tried}/{len(pattern_handlers)} handlers", "INFO")
                
#                 # Cache the solution
#                 HANDLER_CACHE[key] = solution
#                 return solution
#             else:
#                 debug_print(f"  ✗✗ Solution failed verification", "WARNING")
    
#     # PHASE 2: Try pattern combinations
#     debug_print("\n=== PHASE 2: Trying pattern combinations ===", "INFO")
    
#     # Try sequential combinations
#     combination_solution = try_sequential_combinations(task_data, pattern_handlers, analysis)
#     if combination_solution:
#         HANDLER_CACHE[key] = combination_solution
#         return combination_solution
    
#     # Try conditional combinations
#     combination_solution = try_conditional_combinations(task_data, pattern_handlers, analysis)
#     if combination_solution:
#         HANDLER_CACHE[key] = combination_solution
#         return combination_solution
    
#     debug_print(f"\n  SUMMARY: Tried {handlers_tried} handlers, {handlers_with_solutions} returned solutions", "INFO")
#     debug_print("  No valid solution found - returning identity", "WARNING")
    
#     identity_solution = """def p(g):
#  return g
# """
#     HANDLER_CACHE[key] = identity_solution
#     return identity_solution

# # ==================== SECTION 18: MAIN EXECUTION ====================

# def create_arc_solutions_enhanced(input_dir=".", output_dir="submission"):
#     """Enhanced main execution with performance tracking and better reporting"""
#     solutions = {}
    
#     os.makedirs(output_dir, exist_ok=True)
    
#     solved_count = 0
#     total_count = 0
#     start_time = time.time()
    
#     # Task difficulty tracking
#     task_difficulty = defaultdict(int)
    
#     for task_num in range(1, 401):
#         task_id = f"{task_num:03d}"
#         task_file = os.path.join(input_dir, f"task{task_id}.json")
        
#         task_start_time = time.time()
        
#         try:
#             with open(task_file) as f:
#                 task_data = json.load(f)
            
#             solution = generate_solution(task_data)
#             solutions[task_id] = solution
            
#             task_time = time.time() - task_start_time
            
#             if verify_solution(solution, task_data):
#                 print(f"✅ Task {task_id} solved in {task_time:.2f}s")
#                 solved_count += 1
                
#                 # Track which handler solved it
#                 for line in solution.split('\n'):
#                     if 'Step 1:' in line or 'Found' in line:
#                         task_difficulty['easy'] += 1
#                         break
#                 else:
#                     task_difficulty['hard'] += 1
#             else:
#                 print(f"❌ Task {task_id} fallback in {task_time:.2f}s")
#                 task_difficulty['failed'] += 1
            
#             total_count += 1
            
#             # Save debug log if debugging is enabled
#             if DEBUG_MODE and DEBUG_LOG:
#                 debug_file = os.path.join(output_dir, f"debug_task{task_id}.log")
#                 with open(debug_file, "w") as f:
#                     f.write("\n".join(DEBUG_LOG))
                
#         except Exception as e:
#             print(f"Error task {task_id}: {str(e)}")
#             solutions[task_id] = """def p(g):
#  return g
# """
#             total_count += 1
#             task_difficulty['error'] += 1
    
#     # Save solutions
#     for task_id, code in solutions.items():
#         output_file = os.path.join(output_dir, f"task{task_id}.py")
#         with open(output_file, "w") as f:
#             f.write(code)
    
#     # Create submission zip
#     with zipfile.ZipFile("submission.zip", "w") as zipf:
#         for task_id in solutions:
#             file_path = os.path.join(output_dir, f"task{task_id}.py")
#             zipf.write(file_path, f"task{task_id}.py")
    
#     total_time = time.time() - start_time
    
#     # Print summary statistics
#     print(f"\n{'='*60}")
#     print(f"FINAL RESULTS")
#     print(f"{'='*60}")
#     print(f"✅ Solved: {solved_count}/{total_count} ({solved_count/total_count*100:.1f}%)")
#     print(f"⏱️  Total time: {total_time:.1f}s")
#     print(f"⚡ Average time per task: {total_time/total_count:.2f}s")
#     print(f"\nDifficulty breakdown:")
#     for level, count in task_difficulty.items():
#         print(f"  {level}: {count}")
    
#     # Print handler performance stats
#     if PERFORMANCE_STATS:
#         print(f"\nTop performing handlers:")
#         sorted_handlers = sorted(
#             PERFORMANCE_STATS.items(), 
#             key=lambda x: x[1]['successes'], 
#             reverse=True
#         )[:10]
        
#         for handler_name, stats in sorted_handlers:
#             if stats['successes'] > 0:
#                 avg_time = stats['total_time'] / stats['calls'] if stats['calls'] > 0 else 0
#                 print(f"  {handler_name}: {stats['successes']} successes, {avg_time:.3f}s avg")
    
#     print(f"\n📦 Created: submission.zip")

# if __name__ == "__main__":
#     create_arc_solutions_enhanced(input_dir='/kaggle/input/google-code-golf-2025')